# 计算流程

## 1. 整理源数据：将公司名(ID)与数据对应起来
+ dw.qyxx_basic ： regcap
+ qyxx_state_owned_enterprise_background
+ 


## 2. 明确特征的计算函数

+ 合并后的df的字段：

>a	----目标企业ID  
b	----投资方ID  
c	----被投资方ID  
b_degree	----投资方度数   
c_degree	----被投资方度数   
bc_relation	----投资方是否是投资关系   
b_isperson	----投资方是否是人  
c_isperson  ----被投资方是否是人  
a_name	 ----目标企业名称  
b_name  ----投资方名称  
c_name  ----被投资方名称  
b_isSOcompany  ----投资方是否是国企  
c_isSOcompany  ----被投资方是否是国企  
a_isIPOcompany ----目标企业是否是上市  
a_regcap  ----目标企业注册资本  
a_realcap  ----目标企业实收资本  
>a_zhuanli  ----目标企业专利数  
a_shangbiao  ----目标企业商标  
>a_ICP  ----目标企业备案信息  
a_url  ----目标企业网站  
>a_bgxx  ---目标企业变更信息【法定代表人变更次数、股东变更次数、注册资本变更次数、高管变更次数、经营范围变更次数】  
a_bgxx_capital ----目标企业注册资本变更详情    
a_recruit  ----目标企业招聘信息【招聘人员分布】  
a_recruit_industry ----目标企业招聘信息【各行业招聘人员分布】   
a_fzjg ---- 目标公司分支机构数   
>a_zhaobiao  ----目标企业招标数【一年之内】  
a_zhongbiao  ----目标企业中标数【一年之内】  
a_namefrag ---- 目标公司字号    
a_province_black_num ---- 目标公司所在省份黑企业走总数       
a_province_total_num ---- 目标公司所在省份类金融总数    
a_common_interests ----目标公司与黑名单库中任意一家企业存在利益一致行动关系的数量    
a_common_address ----目标公司与黑名单库中任意一家企业地址相同的数量        
b_province ---- 投资方省份    
c_province ---- 被投资方省份    
b_is_black_company ---- 投资方是否是黑企业    
c_is_black_company ---- 被投资方是否是黑企业    
>b_regtime  ----投资方注册时间   
c_regtime  ----被投资方注册时间   
b_ktgg  ----投资方开庭公告数  
b_zgcpwsw  ----投资方裁判文书数  
b_zgcpwsw_specific ----投资方裁判文书数【非法集资】    
b_rmfygg  ----投资方法院公告  
b_lending  ----投资方民间借贷数    
c_ktgg  ----被投资方开庭公告数  
c_zgcpwsw  ----被投资方裁判文书数  
c_zgcpwsw_specific ----被投资方裁判文书数【非法集资】    
c_rmfygg  ----被投资方法院公告  
c_lending  ----被投资方民间借贷数    
b_xzcf  ----投资方行政处罚数  
c_xzcf  ----被投资方行政处罚数  
b_zhixing  ----投资方执行人数  
c_zhixing  ----被投资方执行人数  
b_dishonesty  ----投资方失信被执行人数  
c_dishonesty  ----被投资方失信被执行人数  
b_jyyc  ----投资方经营异常数  
c_jyyc  ----被投资方经营异常数  
b_estatus  ----投资方经营状态  
c_estatus  ----被投资方经营状态   
b_circxzcf  ----投资方银监会行政处罚数  
c_circxzcf  ----被投资方银监会行政处罚数  
>b_opescope  ----投资方经营范围  
c_opescope  ----被投资方经营范围  
b_address  ----投资方地址  
c_address  ----被投资方地址   
b_is_fzjg ----投资方是否是分支机构    
c_is_fzjg ----被投资方是否是分支机构    
b_is_new_finance ----投资方是否是新金融企业    
c_is_new_finance ----被投资方是否是新金融企业    
b_isIPOcompany ----投资方是否是上市公司    
c_isIPOcompany ----被投资方是否是上市公司    



In [1]:
'''
STEP 0 : 重新启一个sparksession，包含各种外部依赖（首先需要给定spark.driver.extraClassPath,与 JDBC的jar包，这里假如一个公司名的解析程序）
'''
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from collections import OrderedDict

spark.stop()

if __name__ == '__main__':
    conf = SparkConf()
    conf.setMaster('yarn-client')
    conf.set("spark.yarn.am.cores", 15)
    conf.set("spark.executor.memory", "50g")
    conf.set("spark.executor.instances", 20)
    conf.set("spark.executor.cores", 10)
    conf.set("spark.python.worker.memory", "3g")
    conf.set("spark.default.parallelism", 3000)
    conf.set("spark.sql.shuffle.partitions", 3000)
    conf.set("spark.broadcast.blockSize", 1024)
    conf.set("spark.sql.crossJoin.enabled", True)
    conf.set("spark.executor.extraJavaOptions","-XX:+PrintGCDetails -XX:+PrintGCTimeStamps")    
    conf.set("spark.submit.pyFiles", u'hdfs://bbdc6ha/user/antifraud/source/keyword_demo/dafei_keyword.py, hdfs://bbdc6ha/user/antifraud/source/company_type_for_capital_risk/wtyh_company_type_20160816.py')
    conf.set("spark.files", u'''hdfs://bbdc6ha/user/antifraud/source/keyword_demo/city,hdfs://bbdc6ha/user/antifraud/source/keyword_demo/1gram.words,hdfs://bbdc6ha/user/antifraud/source/keyword_demo/2gram.words,hdfs://bbdc6ha/user/antifraud/source/keyword_demo/new.work.words,/data5/antifraud/source/GM_release_LR.model''')
    
    
    spark = SparkSession \
        .builder \
        .appName("glf_core_node") \
        .config(conf = conf) \
        .enableHiveSupport() \
        .getOrCreate()    

In [2]:
'''
STEP 1：读取基础原数据
'''
from pyspark.sql import functions as fun
from pyspark.sql import types as tp
from pyspark.sql import Row
import os

#数据参数
relation_version = '20180622'

basic_version = '20180629'
zhuanli_version = '20180629'
shangbiao_version = '20180629'
domain_website_version = '20180629'
bgxx_version = '20180629'
recruit_version = '20180629'
zhaobiao_version = '20180629'
zhongbiao_version = '20180629' 
ktgg_version = '20180629'
zgcpwsw_version = '20180629'
rmfygg_version = '20180629'
xzcf_version = '20180629'
zhixing_version = '20180629'
dishonesty_version = '20180629'
jyyc_version = '20180629'
circxzcf_version = '20180629'
fzjg_version = '20180629'

state_owned_version = '20180117'
black_version = '20170901'
leijinrong_version = relation_version
nf_version = relation_version
so_version = state_owned_version

#输入样本的参数
sample_company_type = 'tar_company'
sample_company_name = 'raw_jl_company_list_20180724_1.data'

#中间结果版本
tid_version = relation_version



def add_col(one_col, two_col):
    '''
    将2个col合成一个元祖
    '''
    return (one_col, two_col)
add_col_udf = fun.udf(add_col, tp.ArrayType(tp.StringType()))

def to_dict(col):
    '''
    转换成字典
    '''
    return dict(col)
to_dict_udf = fun.udf(to_dict, tp.MapType(tp.StringType(), tp.StringType()))

#投资方、被投资方经营状态是否为吊销
def is_not_revoked(col):
    try:
        if u'吊销' not in col and u'注销' not in col:
            return 0
        else:
            return 1
    except:
        return 1
is_not_revoked_udf = fun.udf(is_not_revoked, tp.IntegerType())

def has_keyword(opescope):
    '''
    关键字匹配
    '''
    keywords_list_1 = [
        u'民间借贷', u'民间融资', u'资产管理', u'养老', u'艺术品', u'生态农业',
        u'养生', u'新能源', u'生物科技', u'环保科技', u'资产管理', u'投资管理', 
        u'基金管理', u'支付业务', u'互联网支付', u'电子支付', u'货币兑换', 
        u'移动电话支付', u'银行卡收单', u'预付卡受理', u'金融信息服务', 
        u'电子商务', u'投资咨询', u'投资管理', u'基金募集', u'基金销售',
        u'资产管理', u'征信', u'经济贸易咨询', u'财务咨询', u'风险投资', 
        u'资产经营', u'众筹', u'财富管理',  u'纳米']
    keywords_list_2 = [
        u'投资', u'咨询', u'贸易', u'租赁', u'保理', u'交易场所', u'小额贷款', 
        u'担保', u'金融信息服务', u'网络科技', u'信息科技', u'信息技术']

    keyword_dict = dict(
        zip(keywords_list_1, len(keywords_list_1) * ['k_1']) + 
        zip(keywords_list_1, len(keywords_list_2) * ['k_2']))
    
    if opescope is not None:
        for each_keyword, keyword_type in keyword_dict.iteritems():
            if each_keyword in opescope:
                return keyword_type
        return 'k_0'
    else:
        return 'k_0'
has_keyword_udf = fun.udf(has_keyword, tp.StringType())

def get_change_num(s1, s2):
    '''解析注册资本'''
    
    try:
        import re
        s1_analysis = re.search('[\d\.]+', s1).group()
        s2_analysis = re.search('[\d\.]+', s2).group()
        return round(float(s2_analysis) - float(s1_analysis), 1)
    except:
        return 0.
get_change_num_udf = fun.udf(get_change_num, tp.FloatType())

def get_company_namefrag(iterator):
    '''
    构建DAG；这里因为涉及到加载词典，只能用mapPartition，不然IO开销太大
    '''
    try:
        from dafei_keyword import KeywordExtr
        _obj = KeywordExtr("city", "1gram.words", "2gram.words", "new.work.words")
        keyword_list = []
        for row in iterator:
            keyword_list.append((row.company_name, _obj.clean(row.company_name)))
        return keyword_list
    except Exception, e:
        return e

#原始样本
sample_df = spark.read.csv(
    ("/user/antifraud/jinli/rawdata/{company_type}/"
     "/{company_name}").format(
        company_type=sample_company_type,
        company_name=sample_company_name
    ))\
.withColumnRenamed('_c0', 'company_name') \
.distinct()


#原始关联方
relation_df = spark.sql(
    '''SELECT 
    bbd_qyxx_id                          a,
    source_bbd_id                       b,
    destination_bbd_id                c,
    company_name                     a_name,
    source_name                         b_name,
    destination_name                  c_name,
    source_degree                       b_degree,
    destination_degree               c_degree,
    relation_type                         bc_relation,
    source_isperson                    b_isperson,
    destination_isperson            c_isperson
    FROM 
    dw.off_line_relations 
    WHERE 
    dt='{version}'  
    AND
    source_degree <= 3
    AND
    destination_degree <= 3
    '''.format(version=relation_version)
)

#国企列表
#url = "jdbc:mysql://10.10.10.12:3306/bbd_higgs?characterEncoding=UTF-8"
#prop = {"user": "reader", "password":"", "driver": "com.mysql.jdbc.Driver"} 
#table = "qyxx_state_owned_enterprise_background"
#so_df = spark.read.jdbc(url=url, table=table, properties=prop)
#os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/qyxx_state_owned_enterprise_background/{version}".format(version=so_version))
#so_df.write.parquet("/user/antifraud/jinli/rawdata/qyxx_state_owned_enterprise_background/{version}".format(version=so_version))
so_df = spark.sql(
    '''
    SELECT
    bbd_qyxx_id,
    company_name,
    company_type,
    bbd_uptime,
    bbd_dotime
    FROM
    dw.qyxx_state_owned_enterprise_background
    WHERE
    dt='{version}'  
    '''.format(version=state_owned_version)
)
os.system(
    ("hadoop fs -rmr "
     "{path}/"
     "qyxx_state_owned_enterprise_background"
     "/{version}").format(version=state_owned_version,
                         path='/user/antifraud/jinli/rawdata'))
so_df.write.parquet(
    ("{path}/"
     "qyxx_state_owned_enterprise_background"
     "/{version}").format(version=state_owned_version,
                          path='/user/antifraud/jinli/rawdata'))



#基础工商信息
basic_df = spark.sql(
    '''
    SELECT
    bbd_qyxx_id,
    company_name,
    ipo_company,
    regcap_amount,
    realcap_amount,
    esdate,
    operate_scope,
    trim(address) address,
    enterprise_status,
    company_province
    FROM
    dw.qyxx_basic
    WHERE
    dt='{version}'  
    '''.format(version=basic_version)
).select(
    'bbd_qyxx_id',
    'company_name',
    'ipo_company',
    'regcap_amount',
    'realcap_amount',
    'esdate',
    has_keyword_udf('operate_scope').alias('operate_scope'),
    'address',
    is_not_revoked_udf('enterprise_status').alias('enterprise_status'),
    'company_province'
).cache()
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/basic/{version}".format(version=basic_version))
basic_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/basic/{version}".format(version=basic_version))


#专利信息
zhuanli_count_df = spark.sql(
    '''
    SELECT 
    bbd_qyxx_id,
    count(*) zhuanli_num
    FROM
    dw.qyxx_wanfang_zhuanli
    WHERE
    dt='{version}' 
    GROUP BY 
    bbd_qyxx_id
    '''.format(version=zhuanli_version)
)
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/zhuanli/{version}".format(version=zhuanli_version))
zhuanli_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/zhuanli/{version}".format(version=zhuanli_version))

#商标信息
shangbiao_count_df = spark.sql(
    '''
    SELECT 
    bbd_qyxx_id,
    count(*) shangbiao_num
    FROM
    dw.xgxx_shangbiao
    WHERE
    dt='{version}' 
    GROUP BY 
    bbd_qyxx_id
    '''.format(version=shangbiao_version)
)
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/shangbiao/{version}".format(version=shangbiao_version))
shangbiao_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/shangbiao/{version}".format(version=shangbiao_version))

#域名与网址
domain_website_df = spark.sql(
    '''
    SELECT 
    bbd_qyxx_id,
    homepage_url,
    domain_name, 
    record_license
    FROM
    dw.domain_name_website_info
    WHERE
    dt='{version}' 
    '''.format(version=domain_website_version)
)
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/domain_website/{version}".format(version=domain_website_version))
domain_website_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/domain_website/{version}".format(version=domain_website_version))


#变更信息
bgxx_df = spark.sql(
    '''
    SELECT 
    bbd_qyxx_id,
    change_items,
    count(*) change_num
    FROM
    dw.qyxx_bgxx
    WHERE
    dt='{version}' 
    AND
    (change_items like '%高管%'  OR change_items like '%法定代表人%'  OR 
    change_items like '%股东%'  OR change_items like '%注册资本%'  OR
    change_items like '%经营范围%' )
    GROUP BY 
    bbd_qyxx_id, change_items
    '''.format(version=bgxx_version)
)
bgxx_df = bgxx_df.withColumn('tid_tuple', add_col_udf('change_items', 'change_num')) \
    .groupBy('bbd_qyxx_id') \
    .agg({'tid_tuple': 'collect_list'}) \
    .withColumnRenamed('collect_list(tid_tuple)', 'tid_list') \
    .withColumn('bgxx_dict', to_dict_udf('tid_list')) \
    .select('bbd_qyxx_id', 'bgxx_dict')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/bgxx/{version}".format(version=bgxx_version))
bgxx_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/bgxx/{version}".format(version=bgxx_version))



#注册资本变更
bgxx_capital_df = spark.sql(
    '''
    SELECT 
    bbd_qyxx_id,
    change_date,
    content_before_change, 
    content_after_change
    FROM
    dw.qyxx_bgxx
    WHERE
    dt='{version}' 
    AND
    change_items like '%注册资本%' 
    '''.format(version=bgxx_version)
)
bgxx_capital_df = bgxx_capital_df.withColumn(
    'tid_tuple', 
    add_col_udf(bgxx_capital_df.change_date.astype(tp.StringType()), 
                         get_change_num_udf('content_before_change', 
                                                            'content_after_change').alias('change_info'))
) \
    .groupBy('bbd_qyxx_id') \
    .agg({'tid_tuple': 'collect_list'}) \
    .withColumnRenamed('collect_list(tid_tuple)', 'tid_list') \
    .select('bbd_qyxx_id', 'tid_list')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/bgxx_capital/{version}".format(version=bgxx_version))
bgxx_capital_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/bgxx_capital/{version}".format(version=bgxx_version))


#招聘信息
#招聘学历分布
recruit_df = spark.sql(
    '''
    SELECT 
    bbd_qyxx_id,
    sum(bbd_recruit_num) bbd_recruit_num,
    education_required
    FROM
    dw.recruit
    WHERE
    dt='{version}' 
    GROUP BY
    bbd_qyxx_id, education_required
    '''.format(version=recruit_version)
)
recruit_df = recruit_df.withColumn('tid_tuple', add_col_udf('education_required', 'bbd_recruit_num')) \
    .groupBy('bbd_qyxx_id') \
    .agg({'tid_tuple': 'collect_list'}) \
    .withColumnRenamed('collect_list(tid_tuple)', 'tid_list') \
    .withColumn('recruit_dict', to_dict_udf('tid_list')) \
    .select('bbd_qyxx_id', 'recruit_dict')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/recruit/{version}".format(version=recruit_version))
recruit_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/recruit/{version}".format(version=recruit_version))

#招聘行业分布
recruit_industry_df = spark.sql(
    '''
    SELECT 
    bbd_qyxx_id,
    sum(bbd_recruit_num) bbd_recruit_num,
    bbd_industry
    FROM
    dw.recruit
    WHERE
    dt='{version}' 
    GROUP BY
    bbd_qyxx_id, bbd_industry
    '''.format(version=recruit_version)
)
recruit_industry_df = recruit_industry_df.withColumn('tid_tuple', add_col_udf('bbd_industry', 'bbd_recruit_num')) \
    .groupBy('bbd_qyxx_id') \
    .agg({'tid_tuple': 'collect_list'}) \
    .withColumnRenamed('collect_list(tid_tuple)', 'tid_list') \
    .withColumn('recruit_dict', to_dict_udf('tid_list')) \
    .select('bbd_qyxx_id', 'recruit_dict')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/recruit_industry/{version}".format(version=recruit_version))
recruit_industry_df.repartition(
    10
).write.parquet(
    "/user/antifraud/jinli/rawdata/recruit_industry/{version}".format(version=recruit_version)
)


#招标信息
zhaobiao_df = spark.sql(
    '''
    SELECT 
    bbd_qyxx_id,
    pubdate
    FROM
    dw.shgy_zhaobjg
    WHERE
    dt='{version}' 
    '''.format(version=zhaobiao_version)
)
zhaobiao_count_df = zhaobiao_df.where(
    fun.date_add('pubdate', 365) > fun.current_date()) \
.where(zhaobiao_df.bbd_qyxx_id.isNotNull()) \
.groupBy('bbd_qyxx_id') \
.count() \
.withColumnRenamed('count', 'zhaobiao_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/zhaobiao/{version}".format(version=zhaobiao_version))
zhaobiao_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/zhaobiao/{version}".format(version=zhaobiao_version))

#中标信息
zhongbiao_df = spark.sql(
    '''
    SELECT 
    bbd_qyxx_id,
    pubdate
    FROM
    dw.shgy_zhaobjg
    WHERE
    dt='{version}' 
    '''.format(version=zhongbiao_version)
)
zhongbiao_count_df = zhongbiao_df.where(
        fun.date_add('pubdate', 365) > fun.current_date()) \
    .where(zhongbiao_df.bbd_qyxx_id.isNotNull()) \
    .groupBy('bbd_qyxx_id') \
    .count() \
    .withColumnRenamed('count', 'zhongbiao_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/zhongbiao/{version}".format(version=zhongbiao_version))
zhongbiao_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/zhongbiao/{version}".format(version=zhongbiao_version))

#开庭公告
ktgg_df = spark.sql(
    '''
    SELECT 
    bbd_qyxx_id,
    action_cause
    FROM
    dw.ktgg
    WHERE
    dt='{version}'
    AND
    bbd_qyxx_id is not null
    AND
    action_cause != 'NULL'
    '''.format(version=ktgg_version)
)
ktgg_count_df = ktgg_df.groupBy('bbd_qyxx_id')\
    .count() \
    .withColumnRenamed('count', 'ktgg_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/ktgg/{version}".format(version=ktgg_version))
ktgg_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/ktgg/{version}".format(version=ktgg_version))


#裁判文书
zgcpwsw_df = spark.sql(
    '''
    SELECT 
    bbd_qyxx_id,
    action_cause
    FROM
    dw.zgcpwsw
    WHERE
    dt='{version}'
    AND
    bbd_qyxx_id is not null
    AND
    action_cause != 'NULL'
    '''.format(version=zgcpwsw_version)
)
zgcpwsw_count_df = zgcpwsw_df.groupBy('bbd_qyxx_id') \
    .count() \
    .withColumnRenamed('count', 'zgcpwsw_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/zgcpwsw/{version}".format(version=zgcpwsw_version))
zgcpwsw_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/zgcpwsw/{version}".format(version=zgcpwsw_version))

#非法集资案件的裁判文书
zgcpwsw_specific_df = spark.sql(
    '''
    SELECT
    bbd_qyxx_id,
    action_cause,
    litigant_type,
    caseout_come 
    FROM
    dw.zgcpwsw 
    WHERE
    dt='{version}'
    AND
    (action_cause like '%非法吸收公众存款%' or action_cause like '%集资诈骗%')
    '''.format(version=zgcpwsw_version)
)
zgcpwsw_specific_count_df = zgcpwsw_specific_df.groupBy('bbd_qyxx_id') \
    .count() \
    .withColumnRenamed('count', 'zgcpwsw_specific_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/zgcpwsw_specific/{version}".format(version=zgcpwsw_version))
zgcpwsw_specific_count_df.repartition(10).write.parquet(
    "/user/antifraud/jinli/rawdata/zgcpwsw_specific/{version}".format(version=zgcpwsw_version))


#法院公告
rmfygg_df = spark.sql(
    '''
    SELECT 
    bbd_qyxx_id,
    '' action_cause
    FROM
    dw.rmfygg
    WHERE
    dt='{version}'
    AND
    bbd_qyxx_id is not null
    '''.format(version=rmfygg_version)
)
rmfygg_count_df = rmfygg_df.groupBy('bbd_qyxx_id') \
    .count() \
    .withColumnRenamed('count', 'rmfygg_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/rmfygg/{version}".format(version=rmfygg_version))
rmfygg_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/rmfygg/{version}".format(version=rmfygg_version))


#民间借贷
def filter_machine(col):
    if u'民间借贷' in col:
        return True
    else:
        return False
filter_machine_udf = fun.udf(filter_machine, tp.BooleanType())
lawsuit_df = ktgg_df.union(zgcpwsw_df).union(rmfygg_df)
lawsuit_count_df = lawsuit_df.where(filter_machine_udf('action_cause')) \
    .groupBy('bbd_qyxx_id') \
    .count() \
    .withColumnRenamed('count', 'lawsuit_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/lawsuit/{version}".format(version=rmfygg_version))
lawsuit_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/lawsuit/{version}".format(version=rmfygg_version))


#行政处罚
xzcf_df = spark.sql(
    '''
    SELECT
    bbd_qyxx_id
    FROM
    dw.Xzcf
    WHERE
    dt='{version}'
    '''.format(version=xzcf_version)
)
xzcf_count_df = xzcf_df.groupBy('bbd_qyxx_id') \
    .count() \
    .withColumnRenamed('count', 'xzcf_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/xzcf/{version}".format(version=xzcf_version))
xzcf_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/xzcf/{version}".format(version=xzcf_version))


#被执行
zhixing_df = spark.sql(
    '''
    SELECT
    bbd_qyxx_id
    FROM
    dw.zhixing
    WHERE
    dt='{version}'
    '''.format(version=zhixing_version)
)
zhixing_count_df = zhixing_df.groupBy('bbd_qyxx_id') \
    .count() \
    .withColumnRenamed('count', 'zhixing_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/zhixing/{version}".format(version=zhixing_version))
zhixing_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/zhixing/{version}".format(version=zhixing_version))


#失信被执行
dishonesty_df = spark.sql(
    '''
    SELECT
    bbd_qyxx_id
    FROM
    dw.dishonesty
    WHERE
    dt='{version}'
    '''.format(version=dishonesty_version)
)
dishonesty_count_df = dishonesty_df.groupBy('bbd_qyxx_id') \
    .count() \
    .withColumnRenamed('count', 'dishonesty_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/dishonesty/{version}".format(version=dishonesty_version))
dishonesty_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/dishonesty/{version}".format(version=dishonesty_version))


#经营异常
jyyc_df = spark.sql(
    '''
    SELECT
    bbd_qyxx_id
    FROM
    dw.qyxg_jyyc
    WHERE
    dt='{version}'
    '''.format(version=jyyc_version)
)
jyyc_count_df = jyyc_df.groupBy('bbd_qyxx_id') \
    .count() \
    .withColumnRenamed('count', 'jyyc_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/jyyc/{version}".format(version=jyyc_version))
jyyc_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/jyyc/{version}".format(version=jyyc_version))


#银监会行政处罚
circxzcf_df = spark.sql(
    '''
    SELECT
    bbd_qyxx_id
    FROM
    dw.qyxg_circxzcf
    WHERE
    dt='{version}'
    '''.format(version=circxzcf_version)
)
circxzcf_count_df = circxzcf_df.groupBy('bbd_qyxx_id') \
    .count() \
    .withColumnRenamed('count', 'circxzcf_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/circxzcf/{version}".format(version=circxzcf_version))
circxzcf_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/circxzcf/{version}".format(version=circxzcf_version))


#分支机构
fzjg_df = spark.sql(
    '''
    SELECT
    bbd_qyxx_id,
    name fzjg_name,
    1 is_fzjg
    FROM
    dw.qyxx_fzjg_merge
    WHERE
    dt='{version}'
    '''.format(version=fzjg_version)
)
fzjg_count_df = fzjg_df.groupBy('bbd_qyxx_id') \
    .count() \
    .withColumnRenamed('count', 'fzjg_num')
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/fzjg/{version}".format(version=fzjg_version))
fzjg_count_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/fzjg/{version}".format(version=fzjg_version))

os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/fzjg_name/{version}".format(version=fzjg_version))
fzjg_name_df = fzjg_df.select(
    'fzjg_name',
    'is_fzjg'
).repartition(
    10
).write.parquet(
    "/user/antifraud/jinli/rawdata/fzjg_name/{version}".format(version=fzjg_version)
)


#公司字号
namefrag_df = sample_df.rdd.repartition(100).mapPartitions(get_company_namefrag).map(
            lambda r: Row(company_name=r[0], namefrag=r[1])).toDF()
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/namefrag/{version}".format(version=relation_version))
namefrag_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/namefrag/{version}".format(version=relation_version))


#黑企业名单
black_df = spark.sql(
    "select company_name, 'black' company_type  from dw.qyxg_leijinrong_blacklist where dt={version}".format(version=black_version)
)
#黑企业省份分布
black_province_df  = black_df.join(
    basic_df,
    basic_df.company_name == black_df.company_name
).select(
    black_df.company_name,
    basic_df.company_province
).where(
    basic_df.company_province != 'NULL'
).groupBy(
    'company_province'
).agg(
    {"company_name": "count"}
).withColumnRenamed(
    'count(company_name)', 'province_black_num'
)
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/black_province/{version}".format(version=black_version))
black_province_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/black_province/{version}".format(version=black_version))


#类金融名单
leijinrong_df = spark.read.parquet(
        "/user/antifraud/hongjing2/dataflow/step_one/raw/ljr_sample/{version}".format(version=leijinrong_version)
)
#类金融企业省份分布
leijinrong_province_df  = leijinrong_df.join(
    basic_df,
    basic_df.company_name == leijinrong_df.company_name,
).select(
    basic_df.company_name,
    basic_df.company_province
).where(
    basic_df.company_province != 'NULL'
).groupBy(
    'company_province'
).agg(
    {"company_name": "count"}
).withColumnRenamed(
    'count(company_name)', 'province_leijinrong_num'
)
os.system("hadoop fs -rmr /user/antifraud/jinli/rawdata/leijinrong_province/{version}".format(version=leijinrong_version))
leijinrong_province_df.repartition(10).write.parquet("/user/antifraud/jinli/rawdata/leijinrong_province/{version}".format(version=leijinrong_version))
print "SUCESS !!"

SUCESS !!


In [3]:
'''
STEP 2.0：新版关联方处理（投资与被投资关系）

2.0：读取所以输入数据
2.1：解析关联方，获取全量公司列表 
    (sample_df + relation_df) -> tid_df
2.2：将所有公司信息计算出来 
    tid_df -> (tid_company_list_df + ...) -> tid_company_info_df
2.3：根据样本列表构建属性图 
    (sample_df + tid_df + tid_company_info_df) -> tid_company_merge_df

'''
from pyspark.sql import functions as fun
from pyspark.sql import  types as tp

is_new_finance_udf = fun.udf(lambda r: True, tp.BooleanType())

#2.0 输入数据读取
so_df = spark.read.parquet("/user/antifraud/jinli/rawdata/qyxx_state_owned_enterprise_background/{version}".format(version=so_version))
basic_df = spark.read.parquet("/user/antifraud/jinli/rawdata/basic/{version}".format(version=basic_version)).cache()
zhuanli_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/zhuanli/{version}".format(version=zhuanli_version))
shangbiao_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/shangbiao/{version}".format(version=shangbiao_version))
domain_website_df = spark.read.parquet("/user/antifraud/jinli/rawdata/domain_website/{version}".format(version=domain_website_version))
bgxx_df = spark.read.parquet("/user/antifraud/jinli/rawdata/bgxx/{version}".format(version=bgxx_version))
bgxx_capital_df = spark.read.parquet("/user/antifraud/jinli/rawdata/bgxx_capital/{version}".format(version=bgxx_version))
recruit_df = spark.read.parquet("/user/antifraud/jinli/rawdata/recruit/{version}".format(version=recruit_version))
recruit_industry_df = spark.read.parquet("/user/antifraud/jinli/rawdata/recruit_industry/{version}".format(version=recruit_version))
zhaobiao_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/zhaobiao/{version}".format(version=zhaobiao_version))
zhongbiao_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/zhongbiao/{version}".format(version=zhongbiao_version))
ktgg_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/ktgg/{version}".format(version=ktgg_version))
zgcpwsw_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/zgcpwsw/{version}".format(version=zgcpwsw_version))
zgcpwsw_specific_count_df = spark.read.parquet(
    "/user/antifraud/jinli/rawdata/zgcpwsw_specific/{version}".format(version=zgcpwsw_version))
rmfygg_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/rmfygg/{version}".format(version=rmfygg_version))
lawsuit_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/lawsuit/{version}".format(version=rmfygg_version))
xzcf_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/xzcf/{version}".format(version=xzcf_version))
zhixing_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/zhixing/{version}".format(version=zhixing_version))
dishonesty_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/dishonesty/{version}".format(version=dishonesty_version))
jyyc_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/jyyc/{version}".format(version=jyyc_version))
circxzcf_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/circxzcf/{version}".format(version=circxzcf_version))
fzjg_count_df = spark.read.parquet("/user/antifraud/jinli/rawdata/fzjg/{version}".format(version=fzjg_version))
fzjg_name_df = spark.read.parquet("/user/antifraud/jinli/rawdata/fzjg_name/{version}".format(version=fzjg_version))
namefrag_df = spark.read.parquet("/user/antifraud/jinli/rawdata/namefrag/{version}".format(version=relation_version))
black_province_df = spark.read.parquet("/user/antifraud/jinli/rawdata/black_province/{version}".format(version=black_version))
leijinrong_province_df = spark.read.parquet("/user/antifraud/jinli/rawdata/leijinrong_province/{version}".format(version=leijinrong_version))
new_finance_df = spark.read.parquet(
    "/user/antifraud/hongjing2/dataflow/step_one/raw/ljr_sample/{version}".format(version=nf_version)
).withColumn(
    'is_new_finance', is_new_finance_udf('bbd_qyxx_id')
)



#2.1 解析关联方，获取全量公司列表
from  pyspark.sql.types import StructType, StructField, StringType, IntegerType

def is_invest(edge_iter):
    '''
    将关联图中的各种关系变成：投资/非投资关系
    '''
    for each_edge in edge_iter:
        if each_edge.bc_relation == 'INVEST':
            return Row(
                a=each_edge.a,
                b=each_edge.b,
                c=each_edge.c,
                b_degree=each_edge.b_degree,
                c_degree=each_edge.c_degree,
                bc_relation=each_edge.bc_relation,
                b_isperson=each_edge.b_isperson,
                c_isperson=each_edge.c_isperson,
                a_name=each_edge.a_name,
                b_name=each_edge.b_name,
                c_name=each_edge.c_name)
    else:
        return Row(
            a=edge_iter.data[0].a,
            b=edge_iter.data[0].b,
            c=edge_iter.data[0].c,
            b_degree=edge_iter.data[0].b_degree,
            c_degree=edge_iter.data[0].c_degree,
            bc_relation='UNINVEST',
            b_isperson=edge_iter.data[0].b_isperson,
            c_isperson=edge_iter.data[0].c_isperson,
            a_name=edge_iter.data[0].a_name,
            b_name=edge_iter.data[0].b_name,
            c_name=edge_iter.data[0].c_name)
            
#关联方
tid_df = sample_df.join(
    relation_df,
    fun.trim(relation_df.a_name) == fun.trim(sample_df.company_name),
    'left_outer'
).select(
    'a','b','c','b_degree','c_degree','bc_relation' ,'b_isperson','c_isperson','a_name','b_name','c_name'
)

glf_schema = StructType([
        StructField('a',StringType(),True),
        StructField('b',StringType(),True),
        StructField('c',StringType(),True),
        StructField('b_degree',IntegerType(),True),
        StructField('c_degree',IntegerType(),True),
        StructField('bc_relation',StringType(),True),
        StructField('b_isperson',IntegerType(),True),
        StructField('c_isperson',IntegerType(),True),
        StructField('a_name',StringType(),True),
        StructField('b_name',StringType(),True),
        StructField('c_name',StringType(),True)])

tid_df = tid_df.rdd.map(lambda r: ((r.a, r.b, r.c), r)) \
    .groupByKey().mapValues(is_invest) \
    .map(lambda r: r[1]) \
    .toDF(schema=glf_schema) \
    .cache()
    
tid_company_list_df = tid_df.select(
    'a', 'a_name'
).union(
    tid_df.where(
        tid_df.b_isperson == 0
    ).select(
        'b', 'b_name'
    )
).union(
    tid_df.where(
        tid_df.c_isperson == 0
    ).select(
        'c', 'c_name'
    )
).withColumnRenamed(
    'a', 'bbd_qyxx_id'
).withColumnRenamed(
    'a_name', 'company_name'
).dropDuplicates(
    ['bbd_qyxx_id']
).cache()

#2.2 合并所有公司的相关信息
#国企
tid_company_info_df = tid_company_list_df.join(
    so_df,
    tid_company_list_df.bbd_qyxx_id == so_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_list_df.bbd_qyxx_id,
    tid_company_list_df.company_name,
    fun.when(so_df.company_type.isNotNull(), True).otherwise(False).alias('isSOcompany')
)

#基本信息
tid_company_info_df = tid_company_info_df.join(
    basic_df,
    basic_df.bbd_qyxx_id == tid_company_info_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany',
    fun.when(basic_df.ipo_company != 'NULL', True).otherwise(False).alias('isIPOcompany'),
    basic_df.realcap_amount.alias('realcap'),
    basic_df.regcap_amount.alias('regcap'),
    basic_df.esdate.alias('regtime'),
    basic_df.operate_scope.alias('opescope'),
    basic_df.address.alias('address'),
    basic_df.enterprise_status.alias('estatus'),
    basic_df.company_province.alias('province')    
)

#专利
tid_company_info_df = tid_company_info_df.join(
    zhuanli_count_df,
    tid_company_info_df.bbd_qyxx_id == zhuanli_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    zhuanli_count_df.zhuanli_num.alias('zhuanli')
)

#商标
tid_company_info_df = tid_company_info_df.join(
    shangbiao_count_df,
    tid_company_info_df.bbd_qyxx_id == shangbiao_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli',
    shangbiao_count_df.shangbiao_num.alias('shangbiao')
)

#域名与网址
tid_company_info_df = tid_company_info_df.join(
    domain_website_df,
    tid_company_info_df.bbd_qyxx_id == domain_website_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao',
    domain_website_df.homepage_url.alias('url'),
    domain_website_df.record_license.alias('ICP')
)

#变更信息
tid_company_info_df = tid_company_info_df.join(
    bgxx_df,
    tid_company_info_df.bbd_qyxx_id == bgxx_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP',
    bgxx_df.bgxx_dict.alias('bgxx')
)

#招聘信息
tid_company_info_df = tid_company_info_df.join(
    recruit_df,
    tid_company_info_df.bbd_qyxx_id == recruit_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 
    recruit_df.recruit_dict.alias('recruit')
)

#招标信息
tid_company_info_df = tid_company_info_df.join(
    zhaobiao_count_df,
    tid_company_info_df.bbd_qyxx_id == zhaobiao_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    zhaobiao_count_df.zhaobiao_num.alias('zhaobiao') 
)

#中标信息
tid_company_info_df = tid_company_info_df.join(
    zhongbiao_count_df,
    tid_company_info_df.bbd_qyxx_id == zhongbiao_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao',
    zhongbiao_count_df.zhongbiao_num.alias('zhongbiao') 
)

#开庭公告
tid_company_info_df = tid_company_info_df.join(
    ktgg_count_df,
    tid_company_info_df.bbd_qyxx_id == ktgg_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 
    ktgg_count_df.ktgg_num.alias('ktgg') 
)

#裁判文书
tid_company_info_df = tid_company_info_df.join(
    zgcpwsw_count_df,
    tid_company_info_df.bbd_qyxx_id == zgcpwsw_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg',
    zgcpwsw_count_df.zgcpwsw_num.alias('zgcpwsw') 
)

#法院公告
tid_company_info_df = tid_company_info_df.join(
    rmfygg_count_df,
    tid_company_info_df.bbd_qyxx_id == rmfygg_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg',
    'zgcpwsw',
    rmfygg_count_df.rmfygg_num.alias('rmfygg') 
)

#民间借贷
tid_company_info_df = tid_company_info_df.join(
    lawsuit_count_df,
    tid_company_info_df.bbd_qyxx_id == lawsuit_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg',
    'zgcpwsw', 'rmfygg',
    lawsuit_count_df.lawsuit_num.alias('lending') 
)

#行政处罚
tid_company_info_df = tid_company_info_df.join(
    xzcf_count_df,
    tid_company_info_df.bbd_qyxx_id == xzcf_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg',
    'zgcpwsw', 'rmfygg', 'lending',
    xzcf_count_df.xzcf_num.alias('xzcf') 
)

#被执行
tid_company_info_df = tid_company_info_df.join(
    zhixing_count_df,
    tid_company_info_df.bbd_qyxx_id == zhixing_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf', 
    zhixing_count_df.zhixing_num.alias('zhixing') 
)

#失信被执行
tid_company_info_df = tid_company_info_df.join(
    dishonesty_count_df,
    tid_company_info_df.bbd_qyxx_id == dishonesty_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 
    dishonesty_count_df.dishonesty_num.alias('dishonesty') 
)

#经营异常
tid_company_info_df = tid_company_info_df.join(
    jyyc_count_df,
    tid_company_info_df.bbd_qyxx_id == jyyc_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 
    jyyc_count_df.jyyc_num.alias('jyyc')
)

#银监会处罚
tid_company_info_df = tid_company_info_df.join(
    circxzcf_count_df,
    tid_company_info_df.bbd_qyxx_id == circxzcf_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 'jyyc',
    circxzcf_count_df.circxzcf_num.alias('circxzcf')
)

#分支机构
tid_company_info_df = tid_company_info_df.join(
    fzjg_count_df,
    tid_company_info_df.bbd_qyxx_id == fzjg_count_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 'jyyc',
    'circxzcf', 
    fzjg_count_df.fzjg_num.alias('fzjg')
)

#公司字号
tid_company_info_df = tid_company_info_df.join(
    namefrag_df,
    tid_company_info_df.company_name == namefrag_df.company_name,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 'jyyc',
    'circxzcf', 'fzjg',
    namefrag_df.namefrag.alias('namefrag')
)

#某省份黑企业数
tid_company_info_df = tid_company_info_df.join(
    black_province_df,
    tid_company_info_df.province == black_province_df.company_province,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 'jyyc',
    'circxzcf', 'fzjg', 'namefrag',
    black_province_df.province_black_num
)

#某省份类金融企业数
tid_company_info_df = tid_company_info_df.join(
    leijinrong_province_df,
    tid_company_info_df.province == leijinrong_province_df.company_province,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 'jyyc',
    'circxzcf', 'fzjg', 'namefrag',
    'province_black_num',
    leijinrong_province_df.province_leijinrong_num
)

#是否是黑企业
tid_company_info_df = tid_company_info_df.join(
    black_df,
    tid_company_info_df.company_name == black_df.company_name,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 'jyyc',
    'circxzcf', 'fzjg', 'namefrag',
    'province_black_num', 'province_leijinrong_num',
    fun.when(
        black_df.company_type == 'black', True
    ).otherwise(False).alias('is_black_company')
)

#是否是分支机构
tid_company_info_df = tid_company_info_df.join(
    fzjg_name_df,
    fzjg_name_df.fzjg_name == tid_company_info_df.company_name,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 'jyyc',
    'circxzcf', 'fzjg', 'namefrag',
    'province_black_num', 'province_leijinrong_num',
    'is_black_company',
    fzjg_name_df.is_fzjg
)

#招聘行业数据
tid_company_info_df = tid_company_info_df.join(
    recruit_industry_df,
    tid_company_info_df.bbd_qyxx_id == recruit_industry_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 'jyyc',
    'circxzcf', 'fzjg', 'namefrag',
    'province_black_num', 'province_leijinrong_num',
    'is_black_company', 'is_fzjg',
    recruit_industry_df.recruit_dict.alias('recruit_industry')
)

#注册资本变更详情
tid_company_info_df = tid_company_info_df.join(
    bgxx_capital_df,
    bgxx_capital_df.bbd_qyxx_id == tid_company_info_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 'jyyc',
    'circxzcf', 'fzjg', 'namefrag',
    'province_black_num', 'province_leijinrong_num',
    'is_black_company', 'is_fzjg', 'recruit_industry',
    bgxx_capital_df.tid_list.alias('bgxx_capital')
)

#是否是新金融企业
tid_company_info_df = tid_company_info_df.join(
    new_finance_df,
    new_finance_df.bbd_qyxx_id == tid_company_info_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 'jyyc',
    'circxzcf', 'fzjg', 'namefrag',
    'province_black_num', 'province_leijinrong_num',
    'is_black_company', 'is_fzjg', 'recruit_industry',
    'bgxx_capital',
    new_finance_df.is_new_finance.alias('is_new_finance')
)

#非法集资案件数
tid_company_info_df = tid_company_info_df.join(
    zgcpwsw_specific_count_df,
    zgcpwsw_specific_count_df.bbd_qyxx_id == tid_company_info_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 'jyyc',
    'circxzcf', 'fzjg', 'namefrag',
    'province_black_num', 'province_leijinrong_num',
    'is_black_company', 'is_fzjg', 'recruit_industry',
    'bgxx_capital', 'is_new_finance',
    zgcpwsw_specific_count_df.zgcpwsw_specific_num.alias('zgcpwsw_specific')
).cache()

#某公司是否与黑名单库中任意一家企业存在利益一致行动关系（是1否0）
#是否与黑名单库中任意一家企业地址相同（是1否0）
def is_common_interests(frag, company_name):
    '''
    目标工公司是否与企业存在利益一致行动关系
    '''
    def is_similarity(str_1, str_2):
        '''
        判断两个字符串是否有连续2个字相同
        '''
        try:
            token_1 = [
                str_1[index] + str_1[index+1] 
                for index,data in enumerate(str_1) 
                if index < len(str_1) - 1]
            is_similarity = sum([
                    1 for each_token in token_1 
                    if each_token in str_2])
            return 1 if is_similarity > 0 else 0        
        except:
            return 0
    return is_similarity(frag, company_name)
is_common_interests_udf = fun.udf(is_common_interests, tp.IntegerType())
    
def is_common_address(address1, address2):
    '''
    是否有相同的address
    '''
    return 1 if address1 == address2 else 0
is_common_address_udf = fun.udf(is_common_address, tp.IntegerType())
    
    
all_company_namefrag_df = tid_company_info_df.select(
    'company_name'
).rdd.repartition(
    100
).mapPartitions(
    get_company_namefrag
).map(
    lambda r: Row(company_name=r[0], namefrag=r[1])
).toDF(
).cache()
    
all_company_address_df = all_company_namefrag_df.join(
    basic_df,
    basic_df.company_name == all_company_namefrag_df.company_name,
    'left_outer'
).select(
    basic_df.bbd_qyxx_id,
    all_company_namefrag_df.namefrag,
    basic_df.address.alias('all_company_address')
).dropDuplicates(
    ['bbd_qyxx_id']
).cache()

black_address_df = black_df.join(
    basic_df,
    basic_df.company_name == black_df.company_name,
    'left_outer'
).select(
    black_df.company_name,
    basic_df.address.alias('black_address')
).dropDuplicates(
    ['company_name']
).cache()



os.system("hadoop fs -rmr /user/antifraud/jinli/tmpdata/{version}".format(version=tid_version))
all_company_address_df.repartition(10).write.parquet("/user/antifraud/jinli/tmpdata/{version}/all_company_address_df".format(version=tid_version))
black_address_df.repartition(10).write.parquet("/user/antifraud/jinli/tmpdata/{version}/black_address_df".format(version=tid_version))
tid_company_info_df.repartition(10).write.parquet("/user/antifraud/jinli/tmpdata/{version}/tid_company_info_df".format(version=tid_version))
tid_df.repartition(10).write.parquet("/user/antifraud/jinli/tmpdata/{version}/tid_df".format(version=tid_version))
print 'SUCESS !!'

SUCESS !!


In [4]:
'''
STEP 2.1：为了计算与企业相关的指标，必须将中间结果落地
'''

from pyspark.sql import functions as fun
from pyspark.sql import  types as tp
from pyspark.sql import  Row

def is_common_interests(frag, company_name):
    '''
    目标工公司是否与企业存在利益一致行动关系
    '''
    def is_similarity(str_1, str_2):
        '''
        判断两个字符串是否有连续2个字相同
        '''
        try:
            token_1 = [
                str_1[index] + str_1[index+1] 
                for index,data in enumerate(str_1) 
                if index < len(str_1) - 1]
            is_similarity = sum([
                    1 for each_token in token_1 
                    if each_token in str_2])
            return 1 if is_similarity > 0 else 0        
        except:
            return 0
    return is_similarity(frag, company_name)
is_common_interests_udf = fun.udf(is_common_interests, tp.IntegerType())
    
def is_common_address(address1, address2):
    '''
    是否有相同的address
    '''
    return 1 if address1 == address2 else 0
is_common_address_udf = fun.udf(is_common_address, tp.IntegerType())


all_company_address_df = spark.read.parquet(
    "/user/antifraud/jinli/tmpdata/{version}/all_company_address_df".format(version=tid_version)
).dropDuplicates(
    ['bbd_qyxx_id']
)
black_address_df = spark.read.parquet("/user/antifraud/jinli/tmpdata/{version}/black_address_df".format(version=tid_version))


os.system(
    "hadoop fs -rmr /user/antifraud/jinli/{version}/some_black_info_df".format(version=tid_version)
)
some_black_info_df = all_company_address_df.join(
    black_address_df
).select(
    'bbd_qyxx_id',
    is_common_interests_udf('namefrag', 'company_name').alias('is_common_interests'),
    is_common_address_udf('all_company_address', 'black_address').alias('is_common_address')
).cache(
).where(
    'is_common_interests = 1 or  is_common_address = 1'
).dropDuplicates(
    ['bbd_qyxx_id']
).repartition(
    10
).write.parquet(
    "/user/antifraud/jinli/tmpdata/{version}/some_black_info_df".format(version=tid_version)
)

print 'SUCESS !!'

SUCESS !!


In [5]:
'''
STEP 2.2：合并最终结果
'''


some_black_info_df = spark.read.parquet(
    "/user/antifraud/jinli/tmpdata/{version}/some_black_info_df".format(version=tid_version)
)
tid_company_info_df = spark.read.parquet("/user/antifraud/jinli/tmpdata/{version}/tid_company_info_df".format(version=tid_version))
tid_df = spark.read.parquet("/user/antifraud/jinli/tmpdata/{version}/tid_df".format(version=tid_version))
#原始样本
#sample_company_type = 'tar_company'
#sample_company_name = 'raw_jl_company_list_20170531_1.data'
sample_df = spark.read.csv(
    ("/user/antifraud/jinli/rawdata/{company_type}/"
     "/{company_name}").format(
        company_type=sample_company_type,
        company_name=sample_company_name
    ))\
.withColumnRenamed('_c0', 'company_name') \
.distinct()


tid_company_info_df = tid_company_info_df.join(
    some_black_info_df,
    some_black_info_df.bbd_qyxx_id == tid_company_info_df.bbd_qyxx_id,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id,
    tid_company_info_df.company_name,
    'isSOcompany', 'isIPOcompany', 'realcap',
    'regcap', 'regtime', 'opescope',
    'address', 'estatus', 'province',
    'zhuanli', 'shangbiao', 'url',
    'ICP', 'bgxx', 'recruit', 
    'zhaobiao', 'zhongbiao', 'ktgg', 
    'zgcpwsw', 'rmfygg', 'lending', 'xzcf',
    'zhixing', 'dishonesty', 'jyyc',
    'circxzcf', 'fzjg', 'namefrag',
    'province_black_num', 'province_leijinrong_num',
    'is_black_company', 'is_fzjg', 'recruit_industry',
    'bgxx_capital', 'is_new_finance',
    'zgcpwsw_specific', 'is_common_interests', 
    'is_common_address'
)


#结果输出
tid_company_info_df = tid_company_info_df.dropDuplicates(
    ['bbd_qyxx_id']
)


#2.3 根据样本列表构建属性图
##获取关联方
tid_company_merge_df = sample_df.join(
    tid_df,
    fun.trim(tid_df.a_name) == fun.trim(sample_df.company_name),
    'left_outer'
).select(
    'a', 'b', 'c',
    'b_degree', 'c_degree', 'bc_relation',
    'b_isperson', 'c_isperson',
    sample_df.company_name.alias('a_name'),
    'b_name', 'c_name'
)
##目标公司信息
tid_company_merge_df = tid_company_merge_df.join(
    tid_company_info_df,
    tid_company_info_df.company_name == tid_company_merge_df.a_name,
    'left_outer'
).select(
    tid_company_info_df.bbd_qyxx_id.alias('a'), 
    'b', 'c',
    'b_degree', 'c_degree', 'bc_relation',
    'b_isperson', 'c_isperson', 'a_name',
    'b_name', 'c_name',
    tid_company_info_df.isIPOcompany.alias('a_isIPOcompany'), 
    tid_company_info_df.realcap.alias('a_realcap'),
    tid_company_info_df.regcap.alias('a_regcap'), 
    tid_company_info_df.zhuanli.alias('a_zhuanli'), 
    tid_company_info_df.shangbiao.alias('a_shangbiao'), 
    tid_company_info_df.url.alias('a_url'), 
    tid_company_info_df.ICP.alias('a_ICP'), 
    tid_company_info_df.bgxx.alias('a_bgxx'), 
    tid_company_info_df.recruit.alias('a_recruit'), 
    tid_company_info_df.zhaobiao.alias('a_zhaobiao'), 
    tid_company_info_df.zhongbiao.alias('a_zhongbiao'), 
    tid_company_info_df.fzjg.alias('a_fzjg'), 
    tid_company_info_df.namefrag.alias('a_namefrag'), 
    tid_company_info_df.province_black_num.alias('a_province_black_num'), 
    tid_company_info_df.province_leijinrong_num.alias('a_province_leijinrong_num'),
    tid_company_info_df.recruit_industry.alias('a_recruit_industry'),
    tid_company_info_df.bgxx_capital.alias('a_bgxx_capital'),
    tid_company_info_df.is_common_interests.alias('a_is_common_interests'),
    tid_company_info_df.is_common_address.alias('a_is_common_address')
)
#投资方信息
tid_company_merge_df = tid_company_merge_df.join(
    tid_company_info_df,
    tid_company_info_df.bbd_qyxx_id == tid_company_merge_df.b,
    'left_outer'
).select(
    'a', 'b', 'c',
    'b_degree', 'c_degree', 'bc_relation',
    'b_isperson', 'c_isperson', 'a_name',
    'b_name', 'c_name', 'a_isIPOcompany',
    'a_realcap', 'a_regcap', 'a_zhuanli',
    'a_shangbiao', 'a_url', 'a_ICP',
    'a_bgxx', 'a_recruit', 'a_zhaobiao',
    'a_zhongbiao', 'a_fzjg', 'a_namefrag',
    'a_province_black_num', 'a_province_leijinrong_num',
    'a_recruit_industry', 'a_bgxx_capital', 
    'a_is_common_interests', 'a_is_common_address',
    tid_company_info_df.isSOcompany.alias('b_isSOcompany'),
    tid_company_info_df.isSOcompany.alias('b_isIPOcompany'),
    tid_company_info_df.is_black_company.alias('b_is_black_company'),
    tid_company_info_df.regtime.alias('b_regtime'),
    tid_company_info_df.ktgg.alias('b_ktgg'),
    tid_company_info_df.zgcpwsw.alias('b_zgcpwsw'),
    tid_company_info_df.rmfygg.alias('b_rmfygg'),
    tid_company_info_df.lending.alias('b_lending'),
    tid_company_info_df.xzcf.alias('b_xzcf'),
    tid_company_info_df.zhixing.alias('b_zhixing'),
    tid_company_info_df.dishonesty.alias('b_dishonesty'),
    tid_company_info_df.jyyc.alias('b_jyyc'),
    tid_company_info_df.estatus.alias('b_estatus'),
    tid_company_info_df.circxzcf.alias('b_circxzcf'),
    tid_company_info_df.opescope.alias('b_opescope'),
    tid_company_info_df.address.alias('b_address'),
    tid_company_info_df.province.alias('b_province'),
    tid_company_info_df.is_fzjg.alias('b_is_fzjg'),
    tid_company_info_df.is_new_finance.alias('b_is_new_finance'), 
    tid_company_info_df.zgcpwsw_specific.alias('b_zgcpwsw_specific'),
    tid_company_info_df.is_common_interests.alias('b_is_common_interests'),
    tid_company_info_df.is_common_address.alias('b_is_common_address')
)
#被投资方信息
tid_company_merge_df = tid_company_merge_df.join(
    tid_company_info_df,
    tid_company_info_df.bbd_qyxx_id == tid_company_merge_df.c,
    'left_outer'
).select(
    'a', 'b', 'c',
    'b_degree', 'c_degree', 'bc_relation',
    'b_isperson', 'c_isperson', 'a_name',
    'b_name', 'c_name', 'a_isIPOcompany',
    'a_realcap', 'a_regcap', 'a_zhuanli',
    'a_shangbiao', 'a_url', 'a_ICP',
    'a_bgxx', 'a_recruit', 'a_zhaobiao',
    'a_zhongbiao', 'a_fzjg', 'a_namefrag',    
    'a_province_black_num', 'a_province_leijinrong_num',
    'a_recruit_industry', 'a_bgxx_capital',
    'a_is_common_interests', 'a_is_common_address',
    'b_isSOcompany' , 'b_isIPOcompany', 'b_is_black_company', 'b_regtime',
    'b_ktgg', 'b_zgcpwsw', 'b_rmfygg', 'b_lending',   
    'b_xzcf' , 'b_zhixing', 'b_dishonesty',   
    'b_jyyc' , 'b_estatus', 'b_circxzcf',   
    'b_opescope', 'b_address', 'b_province', 'b_is_fzjg',
    'b_is_new_finance', 'b_zgcpwsw_specific',
    'b_is_common_interests', 'b_is_common_address',
    tid_company_info_df.isSOcompany.alias('c_isSOcompany'),
    tid_company_info_df.isSOcompany.alias('c_isIPOcompany'),
    tid_company_info_df.is_black_company.alias('c_is_black_company'),
    tid_company_info_df.regtime.alias('c_regtime'),
    tid_company_info_df.ktgg.alias('c_ktgg'),
    tid_company_info_df.zgcpwsw.alias('c_zgcpwsw'),
    tid_company_info_df.rmfygg.alias('c_rmfygg'),
    tid_company_info_df.lending.alias('c_lending'),
    tid_company_info_df.xzcf.alias('c_xzcf'),
    tid_company_info_df.zhixing.alias('c_zhixing'),
    tid_company_info_df.dishonesty.alias('c_dishonesty'),
    tid_company_info_df.jyyc.alias('c_jyyc'),
    tid_company_info_df.estatus.alias('c_estatus'),
    tid_company_info_df.circxzcf.alias('c_circxzcf'),
    tid_company_info_df.opescope.alias('c_opescope'),
    tid_company_info_df.address.alias('c_address'),
    tid_company_info_df.province.alias('c_province'),
    tid_company_info_df.is_fzjg.alias('c_is_fzjg'),
    tid_company_info_df.is_new_finance.alias('c_is_new_finance'),
    tid_company_info_df.zgcpwsw_specific.alias('c_zgcpwsw_specific'),
    tid_company_info_df.is_common_interests.alias('c_is_common_interests'),
    tid_company_info_df.is_common_address.alias('c_is_common_address')
)

os.system("hadoop fs -rmr /user/antifraud/jinli/tiddata/relation_company_info_merge/{version}".format(version=tid_version))
tid_company_merge_df.repartition(10).write.parquet("/user/antifraud/jinli/tiddata/relation_company_info_merge/{version}".format(version=tid_version))
print "SUCESS ！！"

SUCESS ！！


In [6]:
'''
STEP 3：使用DataFrame完成数据计算（静态风险）
'''
import datetime
import requests
import json
import time
import re
import os
from operator import itemgetter, methodcaller
from collections import Counter, defaultdict, OrderedDict

from pandas.io.json import json_normalize
import networkx as nx
import numpy as np



class FeatureConstruction(object):
    '''
    计算特征的函数集
    '''      

    def __fault_tolerant(func):
        '''
        一个用于容错的装饰器
        '''
        @classmethod
        def wappen(cls, *args, **kwargs):
            try:
                return func(cls, *args, **kwargs)
            except Exception, e:
                return (
                    "{func_name} has a errr : {excp}"
                ).format(func_name=func.__name__, excp=e)
        return wappen    
    
    @__fault_tolerant    
    def create_graph_udf(cls, relations, is_directed):
        '''
        根据关联方的结构创建有向、无向图
        '''
        def init_graph(edge_list, node_list, is_directed=0):
            #网络初始化
            G = nx.DiGraph() if is_directed == 1 else nx.Graph()    
            #增加带属性的节点
            for node in node_list:
                G.add_node(node[0], attr_dict=node[1])
            #增加带属性的边
            G.add_edges_from(edge_list)
            return G
            
        #生成一个图
        company_correlative_edges = [
            (row.b, row.c, {'is_invest': row.bc_relation}) for row in relations]
                                     
        company_correlative_nodes = [(
                row.b, 
                dict(
                    is_human=row.b_isperson,
                    is_black=row.b_is_black_company,
                    distance = row.b_degree,
                    name = row.b_name,
                    isSOcompany = row.b_isSOcompany,
                    isIPOcompany = row.b_isIPOcompany,
                    esdate = row.b_regtime,
                    ktgg = row.b_ktgg,
                    zgcpwsw = row.b_zgcpwsw,
                    rmfygg = row.b_rmfygg,
                    lending = row.b_lending,
                    xzcf = row.b_xzcf,
                    zhixing = row.b_zhixing,
                    dishonesty = row.b_dishonesty,
                    jyyc = row.b_jyyc,
                    circxzcf = row.b_circxzcf,
                    opescope = row.b_opescope,
                    address = row.b_address,
                    estatus = row.b_estatus,
                    province = row.b_province,
                    is_fzjg=row.b_is_fzjg,
                    is_new_finance=row.b_is_new_finance,
                    zgcpwsw_specific=row.b_zgcpwsw_specific,
                    common_interests=row.b_is_common_interests,
                    common_address=row.b_is_common_address
                ))  for row in relations] + [(
                row.c, 
                dict(
                    is_human=row.c_isperson,
                    is_black=row.c_is_black_company,
                    distance = row.c_degree,
                    name = row.c_name,
                    isSOcompany = row.c_isSOcompany,
                    isIPOcompany = row.c_isIPOcompany,
                    esdate = row.c_regtime,
                    ktgg = row.c_ktgg,
                    zgcpwsw = row.c_zgcpwsw,
                    rmfygg = row.c_rmfygg,
                    lending = row.c_lending,
                    xzcf = row.c_xzcf,
                    zhixing = row.c_zhixing,
                    dishonesty = row.c_dishonesty,
                    jyyc = row.c_jyyc,
                    circxzcf = row.c_circxzcf,
                    opescope = row.c_opescope,
                    address = row.c_address,
                    estatus = row.c_estatus,
                    province = row.c_province,
                    is_fzjg=row.c_is_fzjg,
                    is_new_finance=row.c_is_new_finance,
                    zgcpwsw_specific=row.c_zgcpwsw_specific,
                    common_interests=row.c_is_common_interests,
                    common_address=row.c_is_common_address
                )) for row in relations]
        
        if is_directed == 1:
            g = init_graph(
                company_correlative_edges,
                company_correlative_nodes, is_directed = 1) 
        else:
            g = init_graph(
                company_correlative_edges, 
                company_correlative_nodes, is_directed = 0)
        return g

    @__fault_tolerant
    def get_feature_1(cls):
        '''
        企业背景风险
        '''
        one_relation_set = [
            node for node, attr in cls.DIG.nodes_iter(data=True) if attr['distance'] == 1]
        shareholder = [
            src_node for src_node, des_node, edge_attr 
            in cls.DIG.edges_iter(one_relation_set, data=True) 
            if des_node == cls.tarcompany 
            and edge_attr['is_invest'] == 'INVEST']
        os_shareholder = [
            node for node in shareholder 
            if cls.DIG.node[node]['isSOcompany'] is True 
            and cls.DIG.node[node]['is_human'] == 0]
        anti_os_shareholder = [
            node for node in shareholder 
            if cls.DIG.node[node]['isSOcompany'] is False 
            and cls.DIG.node[node]['is_human'] == 0]
        nature_shareholder = [
            node for node in shareholder 
            if cls.DIG.node[node]['is_human'] == 1]
        new_finance = [
            node for ndoe in shareholder
            if cls.DIG.node[node]['is_new_finance']]
        ipo_company = [
            node for ndoe in shareholder
            if cls.DIG.node[node]['isIPOcompany']]
        out_degree_obj = cls.DIG.out_degree()
        if isinstance(out_degree_obj, dict):
            out_degree_list = out_degree_obj.values()
            out_degree_list.sort()
        else:
            out_degree_list = [0]
        
        is_ipo_tarcompany = 1 if cls.resultiterable.data[0].a_isIPOcompany is True else 0
        is_common_interests = 1 if cls.resultiterable.data[0].a_is_common_interests else 0
        is_common_address = 1 if cls.resultiterable.data[0].a_is_common_address else 0
        
        x = len(os_shareholder)
        y = len(anti_os_shareholder)
        z = len(nature_shareholder)
        w = len(new_finance)
        n = len(ipo_company)
        r_i = is_ipo_tarcompany
        r_1 = is_common_interests
        r_2 = is_common_address
        r_3 = out_degree_list[-1]
        r_4 = sum(out_degree_list[-3:])
        
        r = (0.2*x + 0.5*y + z) * 100. * (2 - r_i) / (2*w + 0.001)
        
        return dict(
            x=x,
            y=y,
            z=z,
            n=n,
            w=w,
            r_1=r_1,
            r_2=r_2,
            r_3=r_3,
            r_4=r_4,
            r_i=r_i,
            r=round(r, 2) if r else 100
        )


    @__fault_tolerant
    def get_feature_2(cls):
        '''
        1、资本风险
        2、目标公司任意6个月内注册资本最大增额
        '''
        def get_date_density(difference_list, timedelta):
            '''计算最大连续时间段，并返回他们的index'''
            time_density_list = []
            result = ''
            for index, date in enumerate(difference_list):
                if date < timedelta:
                    s = 0
                    indexes = [index]
                    i = 1
                    while(s < timedelta and i <= len(difference_list) - index):
                        indexes.append(index+i)
                        i += 1
                        s = sum(difference_list[index:index+i])
                    time_density_list.append(indexes)
                else:
                    continue
            if time_density_list:
                ma = 0
                for each_data in time_density_list:
                    if len(each_data) > ma:
                        result = each_data
                        ma = len(each_data)
                    else:
                        continue
            return time_density_list
    
        def get_date_list(date_list):
            '''获得时间差序列'''
            date_strp_list = sorted(date_list)
            if len(date_strp_list) > 1:
                #构建时间差的序列，例如：[256, 4, 5, 1, 2, 33, 6, 5, 4, 73]
                date_difference_list = [
                    (date_strp_list[index + 1] - date_strp_list[index]).days 
                    for index in range(len(date_strp_list) -1)]
            else:
                date_difference_list = []
            return date_difference_list

                
        captial_list = cls.resultiterable.data[0].a_bgxx_capital
        result = []
        
        if captial_list and len(captial_list) > 1:
            captial_list = sorted(captial_list, key=lambda r: r[0])
            date_list = map(
                lambda r: datetime.datetime.strptime(
                    r[0], "%Y-%m-%d"), captial_list)
            max_date_indexes = get_date_density(get_date_list(date_list), 180)
            for each_index in max_date_indexes:  
                result.append(
                    abs(
                        float(captial_list[each_index[-1]][1]) - 
                        float(captial_list[each_index[0]][1] )))
        elif captial_list and len(captial_list) ==1:
            result.append(float(captial_list[0][1]))
            
            
        x = cls.resultiterable.data[0].a_regcap 
        y = cls.resultiterable.data[0].a_realcap
        
        x = x / 10000 if x != 'NULL' and x is not None else 0
        y = y / 10000 if y != 'NULL' and y is not None else 0
        
        if 0 <= x < 500:
            c_i = 100
        elif 500 <= x < 1000:
            c_i = 60
        elif 1000 <= x < 5000:
            c_i = 30
        elif 5000 <= x:
            c_i = 10
        
        p_i = 1 if y / (x + 0.001) >= 0.5 else 0
        
        return  dict(
            x=x,
            x_1=max(result) if result else 0,
            y=y,
            c=c_i * (1 - p_i/2.)
        )
    
    @__fault_tolerant
    def get_feature_3(cls):
        '''
        资本背景风险
        '''
        def get_relation_set(distance):
            return [
                attr['name']
                for node, attr in cls.DIG.nodes_iter(data=True)
                if attr['distance'] == distance]

        from wtyh_company_type_20160816 import my_dict
        import numpy as np
        
        ratio = np.array([1, 1, 0.75, 0.5])
        reducing_score = np.array([-20, -40, -30, -15])
        
        #记录目标企业关联方中的所有企业类别
        matrx = np.zeros([4, 4], dtype=int)
        
        #计算关联方的数据
        degree_zero_list = get_relation_set(0)
        degree_one_list = get_relation_set(1)
        degree_two_list = get_relation_set(2)
        degree_three_list = get_relation_set(3)
        
        for degree_zero_company in degree_zero_list:
            if my_dict.has_key(degree_zero_company):
                row_index = my_dict[degree_zero_company]
                col_index = 0
                matrx[row_index][col_index] += 1

        
        for degree_one_company in degree_one_list:
            if my_dict.has_key(degree_one_company):
                row_index = my_dict[degree_one_company]
                col_index = 1
                matrx[row_index][col_index] += 1    

        
        for degree_two_company in degree_two_list:
            if my_dict.has_key(degree_two_company):
                row_index = my_dict[degree_two_company]
                col_index = 2
                matrx[row_index][col_index] += 1    

        
        for degree_three_company in degree_three_list:
            if my_dict.has_key(degree_three_company):
                row_index = my_dict[degree_three_company]
                col_index = 3
                matrx[row_index][col_index] += 1

        
        #资本背景风险有一个特殊的判断条件：必须小于等于0
        B = reducing_score.dot(matrx).dot(ratio) 
        B = B if B >= -30 else -30
        
        #各度的条件
        data = matrx.sum(axis=1).tolist()
        
        return dict(
            z=B + 30,
            x_0=data[0],
            x_1=data[1],
            x_2=data[2],
            x_3=data[3],
        )
    
    

    
    @__fault_tolerant
    def get_feature_4(cls):
        '''
        知识产权风险（专利和商标）
        '''
        k_1 = cls.resultiterable.data[0].a_zhuanli
        k_2 = cls.resultiterable.data[0].a_shangbiao 
        k_1 = k_1 if k_1 != 'NULL' and k_1 is not None else 0
        k_2 = k_2 if k_1 != 'NULL' and k_2 is not None else 0
        k_3 = k_1 + k_2
        
        return dict(
            k_1=k_1,
            k_2=k_2,
            k=k_3
        )
        
    @__fault_tolerant
    def get_feature_5(cls):
        '''
        域名备案风险
        '''
        icp = cls.resultiterable.data[0].a_ICP
        url = cls.resultiterable.data[0].a_url
        
        c_i = 1 if icp != 'NULL' and icp is not None else 0   
        try:
            if url is not None and url != 'NULL':
                status_code = requests.get(url, allow_redirects=False).status_code 
                p_i = 1 if status_code == 200 else 0
            else:
                p_i = 0
        except:
            p_i = 0
        
        l = c_i * p_i
        
        return dict(
            c_i=c_i,
            p_i=p_i,
            l=l
        )
        
    @__fault_tolerant
    def get_feature_6(cls):
        '''
        工商变更风险
        '''
        def get_bgxx_symbol(bgxx_name):
            if u'法定代表人' in bgxx_name:
                return 'c_1'
            elif u'股东' in bgxx_name:
                return 'c_2'
            elif u'注册资本' in bgxx_name:
                return 'c_3'
            elif u'高管' in bgxx_name:
                return 'c_4'
            elif u'经营范围' in bgxx_name:
                return 'c_5'
            else:
                return 'c_6'
        
        default_result = OrderedDict([
                ('c_1', 0),
                ('c_2', 0),
                ('c_3', 0),
                ('c_4', 0),
                ('c_5', 0),
                ('c_6', 0),
                ('z', 0)
            ]
        )
        
        if cls.resultiterable.data[0].a_bgxx is not None:
            for bgxx_name, bgxx_num in cls.resultiterable.data[0].a_bgxx.iteritems():
                default_result[get_bgxx_symbol(bgxx_name)] += int(bgxx_num)
            default_result['z'] = np.dot(default_result.values(), [2, 1, 1, 1, 2, 0, 0])
                
        default_result.pop('c_6')
        
        return dict(default_result)
        
    @__fault_tolerant
    def get_feature_7(cls):
        '''
        1、人才结构风险
        2、目标公司招聘行业分布
        '''        
        def get_recruit_num(each_recruit):
            if u'本科' in each_recruit:
                return 'e_2'
            elif u'硕士' in each_recruit:
                return 'e_3'
            elif u'博士' in each_recruit:
                return 'e_3'
            else:
                return 'e_1'

        default_result = OrderedDict([
                ('e_1', 0),
                ('e_2', 0),
                ('e_3', 0),
                ('e_4', 0),
                ('e_5', 0),
                ('e_6', 0),
                ('e', 0),
                ('z', 0)
            ]
        )

        recruit_data = cls.resultiterable.data[0].a_recruit_industry
        if recruit_data:
            for key, value in recruit_data.iteritems():
                if u'金融' in key:
                    default_result['e_4'] += int(value)
                if (u'租赁' or u'商务') in key:
                    default_result['e_5'] += int(value)
                if (u'信息传输' or u'软件' or u'信息技术') in key:
                    default_result['e_6'] += int(value)
        
        if cls.resultiterable.data[0].a_recruit is not None:
            for education_name, education_num in cls.resultiterable.data[0].a_recruit.iteritems():
                default_result[get_recruit_num(education_name)] += int(education_num)
            default_result['e'] = sum(
                map(int ,cls.resultiterable.data[0].a_recruit.values()))
            default_result['z'] = default_result['e_1'] / default_result['e'] * 15.  
            
        return dict(default_result)
        
    @__fault_tolerant
    def get_feature_8(cls):
        '''
        公司运营持续风险
        '''
        import math
        
        esdate_relation_set = [
            (datetime.date.today() - attr['esdate']).days 
            for node, attr in cls.DIG.nodes_iter(data=True) 
            if 0 < attr['distance'] <= 3 
            and attr['is_human'] == 0 
            and attr['esdate']]
        t_2 = round(np.average(esdate_relation_set), 2)
        
        if (cls.DIG.node.get(cls.tarcompany, 0) and 
                    cls.DIG.node[cls.tarcompany]['esdate']):
            t_1 = (datetime.date.today() - cls.DIG.node[cls.tarcompany]['esdate']).days 
        else :
            t_1 = 0
       
        leagal_person_set = [
            (datetime.date.today() - attr['esdate']).days 
            for node, attr in cls.DIG.nodes_iter(data=True) 
            if attr['distance'] == 1
            and attr['is_human'] == 0 
            and attr['esdate']
        ]
        t_3 = round(np.average(leagal_person_set), 2)
    
        return dict(
            t_2=t_2 if not math.isnan(t_2) else 0,
            t_1=t_1,
            t_3=t_3 if not math.isnan(t_3) else 0,
            y=t_1
        )

        
    @__fault_tolerant
    def get_feature_9(cls):
        '''
        招标和中标风险
        '''
        n_1 = cls.resultiterable.data[0].a_zhaobiao
        n_2 = cls.resultiterable.data[0].a_zhongbiao
        n_1 = n_1 if n_1 is not None else 0
        n_2 = n_2 if n_2 is not None else 0
        
        n = n_1 + n_2
        
        return  dict(
            n_1=n_1,
            n_2=n_2,
            n=n
        )
        
    @__fault_tolerant
    def get_feature_xgxx(cls):
        '''
        【计算基础数据】
        法律诉讼风险：开庭公告、裁判文书、非法集资裁判文书、法院公告、民间借贷
        行政处罚
        被执行风险
        异常经营风险：经营异常、吊销&注销
        银监会行政处罚
        ''' 
        def get_certain_distance_all_info(distance, document_types):
            '''
            总条数
            '''
            all_array = []
            #处理某一个distance不存在节点的情况
            all_array.append([0]*len(document_types))            
            for node, attr in cls.DIG.nodes_iter(data=True):
                if attr['is_human'] == 0  and attr['distance'] == distance:
                    each_array = map(lambda x: x if x else 0, 
                                                  [attr[each_document] 
                                                      for each_document in document_types])
                    all_array.append(each_array)
                else:
                    continue
            documents_num = np.sum(all_array, axis=0)
            return documents_num
        
        def get_certain_distance_add_info(distance, document_types):
            '''
            出现次数
            '''
            all_array = []
            #处理某一个distance不存在节点的情况
            all_array.append([0]*len(document_types))            
            for node, attr in cls.DIG.nodes_iter(data=True):
                if attr['is_human'] == 0  and attr['distance'] == distance:
                    each_array = map(lambda x: 1 if x else 0, 
                                                  [attr[each_document] 
                                                      for each_document in document_types])
                    all_array.append(each_array)
                else:
                    continue
            documents_num = np.sum(all_array, axis=0)
            return documents_num        
        
        matrx = dict()
        xgxx_type = ['ktgg', 'zgcpwsw', 'rmfygg', 
                             'lending', 'xzcf', 'zhixing', 
                             'dishonesty', 'jyyc', 'circxzcf', 
                             'estatus', 'zgcpwsw_specific']
        
        xgxx_add_type = ['ktgg_1', 'zgcpwsw_1', 'rmfygg_1', 
                             'lending_1', 'xzcf_1', 'zhixing_1', 
                             'dishonesty_1', 'jyyc_1',
                             'estatus_1', 'zgcpwsw_specific_1']   

        for each_distance in xrange(0, 4):
            xgxx_num_list = get_certain_distance_all_info(each_distance, 
                                                                                      xgxx_type)
            xgxx_add_num_list = get_certain_distance_add_info(each_distance, 
                                                                                                xgxx_type)
            matrx[each_distance] = dict(zip(xgxx_type, xgxx_num_list)+
                                                         zip(xgxx_add_type, xgxx_add_num_list))
            
        return matrx
    
    @__fault_tolerant
    def filter_xgxx_type(cls, tar_xgxx):
        '''
        过滤相关信息
        '''
        risk = dict()
        for each_distance, xgxx_statistics in cls.xgxx_distribution.iteritems():
            tar_xgxx_statistics = dict([
                    (each_xgxx_type, each_xgxx_num)
                    for each_xgxx_type, each_xgxx_num 
                    in xgxx_statistics.iteritems() 
                    if each_xgxx_type in tar_xgxx])
            risk[each_distance] = tar_xgxx_statistics
        return risk
    
    @__fault_tolerant
    def get_feature_10(cls):
        '''
        1、法律诉讼风险：开庭公告、裁判文书、法院公告、民间借贷
        2、某度关联方中非法集资裁判文书总数
        '''
        tar_xgxx = ['ktgg', 'zgcpwsw', 'rmfygg', 'lending', 'zgcpwsw_specific',
                           'ktgg_1', 'zgcpwsw_1', 'rmfygg_1', 
                           'lending_1', 'zgcpwsw_specific_1']
        risk = cls.filter_xgxx_type(tar_xgxx)
        
        risk['z'] = round(
            np.dot(
                map(sum, [
                        risk[each_distance].values() 
                        for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)
        
        return risk
            
    @__fault_tolerant
    def get_feature_11(cls):
        '''
        行政处罚风险
        '''
        tar_xgxx = ['xzcf', 'xzcf_1']
        risk = cls.filter_xgxx_type(tar_xgxx)
        
        risk['z'] = round(
            np.dot(
                map(sum, [risk[each_distance].values() 
                        for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)
        
        return risk
    
    @__fault_tolerant
    def get_feature_12(cls):
        '''
        被执行风险：被执行、失信被执行
        '''
        tar_xgxx = ['zhixing', 'dishonesty', 'zhixing_1', 
                           'dishonesty_1']
        risk = cls.filter_xgxx_type(tar_xgxx)        
        
        risk['z'] = round(
            np.dot(
                map(
                    lambda x: x['zhixing'] + 2. * x['dishonesty'], 
                    [risk[each_distance] for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)     
        
        return risk
        
    @__fault_tolerant
    def get_feature_13(cls):
        '''
        异常经营风险
        '''
        tar_xgxx = ['jyyc', 'estatus', 'jyyc_1', 'estatus_1']
        risk = cls.filter_xgxx_type(tar_xgxx)        
        
        risk['z'] = round(
            np.dot(
                map(
                    lambda x: x['jyyc'] + 2. * x['estatus'], 
                    [risk[each_distance] for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)
        
        return risk        
        
    @__fault_tolerant
    def get_feature_14(cls):
        '''
        银监会行政处罚
        '''
        tar_xgxx = ['circxzcf']
        risk = cls.filter_xgxx_type(tar_xgxx)        
        
        risk['z'] = round(
            np.dot(
                map(
                    lambda x: x['circxzcf'], 
                    [risk[each_distance] for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)
     
        return risk            
        
    @__fault_tolerant
    def get_feature_15(cls):
        '''
        实际控制人风险
        '''
        def get_degree_distribution(is_human):
            some_person_sets = [
                node
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if attr['is_human'] == is_human
                and attr['distance'] <= 3]
            person_out_degree = cls.DIG.out_degree(some_person_sets).values()
            if not person_out_degree:
                person_out_degree.append(0)
            return person_out_degree
        
        nature_person_distribution = get_degree_distribution(1)
        legal_person_distribution = get_degree_distribution(0)
        
        nature_max_control = max(nature_person_distribution)
        legal_max_control = max(legal_person_distribution)        
        nature_avg_control = round(np.average(nature_person_distribution), 2)
        legal_avg_control = round(np.average(legal_person_distribution), 2)
        
        total_legal_num = len([
                node 
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if attr['is_human'] == 0])
        
        risk = round(((
                    2*(nature_max_control + legal_max_control) + 
                    (nature_avg_control + legal_avg_control)) /
                (2*total_legal_num + 0.001)), 2)
        
        return dict(
            x_1=nature_max_control,
            x_2=legal_max_control,
            y_1=nature_avg_control,
            y_2=legal_avg_control,
            z=total_legal_num,
            r=risk
        )
    
    @__fault_tolerant
    def get_feature_16(cls):
        '''
        公司扩张路径风险
        '''
        def get_node_set(is_human):
            return Counter([
                    attr['distance']
                    for node, attr in cls.DIG.nodes_iter(data=True)
                    if attr['is_human'] == is_human])
        
        def get_node_set_two(company_type):
            return Counter([
                    attr['distance']
                    for node, attr in cls.DIG.nodes_iter(data=True)
                    if attr['is_human'] == 0
                    and attr[company_type]])
        
        
        nature_person_distribution = get_node_set(1)
        legal_person_distribution = get_node_set(0)
        so_company_distribution = get_node_set_two('isSOcompany')
        ipo_company_distribution = get_node_set_two('isIPOcompany')
        
        nature_person_num = [
            nature_person_distribution.get(each_distance, 0)
            for each_distance in range(1, 4)]
        legal_person_num = [
            legal_person_distribution.get(each_distance, 0)
            for each_distance in range(1, 4)]
        so_company_num = [
            so_company_distribution.get(each_distance, 0)
            for each_distance in range(1, 4)]
        ipo_company_num = [
            ipo_company_distribution.get(each_distance, 0)
            for each_distance in range(1, 4)]
        
        risk = round(np.sum(
                np.divide(
                    [
                        np.divide(
                            np.sum(nature_person_num[:each_distance]), 
                            np.sum(legal_person_num[:each_distance]), 
                            dtype=float)
                        for each_distance in range(1, 4)], 
                    np.array([1, 2, 3], dtype=float))), 2)
        
        return dict(
            x_1=legal_person_num[0],
            x_2=legal_person_num[1],
            x_3=legal_person_num[2],
            y_1=nature_person_num[0],
            y_2=nature_person_num[1],
            y_3=nature_person_num[2],
            v_1=so_company_num[0],
            v_2=so_company_num[1],
            v_3=so_company_num[2],
            w_1=ipo_company_num[0],
            w_2=ipo_company_num[1],
            w_3=ipo_company_num[2],
            r=risk if risk < 10000 else 0
        )

    @__fault_tolerant
    def get_feature_17(cls):
        '''
        关联方中心集聚风险
        '''
        one_relation_set = [
            node for node, attr in cls.DIG.nodes_iter(data=True) 
            if attr['distance'] == 1]
        legal_person_shareholder = len([
                src_node 
                for src_node, des_node, edge_attr 
                in cls.DIG.edges_iter(one_relation_set, data=True) 
                if des_node == cls.tarcompany 
                and edge_attr['is_invest'] == 'INVEST'
                and cls.DIG.node[src_node]['is_human'] == 0])
        legal_person_subsidiary = len([
                des_node 
                for src_node, des_node, edge_attr 
                in cls.DIG.edges_iter(data=True) 
                if src_node == cls.tarcompany 
                and edge_attr['is_invest'] == 'INVEST'
                and cls.DIG.node[des_node]['is_human'] == 0])
        
        risk =(
            legal_person_subsidiary -
            legal_person_shareholder )    
        
        return dict(
            x=legal_person_subsidiary,
            y=legal_person_shareholder,
            r=risk
        )

    @__fault_tolerant
    def get_feature_18(cls):
        '''
        分支机构过度扩张风险
        '''
        risk = cls.resultiterable.data[0].a_fzjg
        risk = risk if risk is not None else 0
        
        return dict(
            d=risk, 
            z=risk
        )
    
    @__fault_tolerant
    def get_feature_19(cls):
        '''
        关联方结构稳定风险
        '''
        def get_relation_num():
            return Counter([
                    attr['distance']
                    for node, attr in cls.DIG.nodes(data=True)])
        
        #目标企业各个度的节点的数量
        relations_num = get_relation_num()
        relation_three_num = relations_num.get(3, 0)
        relation_two_num = relations_num.get(2, 0)
        relation_one_num = relations_num.get(1, 0)
        relation_zero_num = relations_num.get(0, 1)
        
        x = np.array([
                relation_zero_num, 
                relation_one_num, 
                relation_two_num, 
                relation_three_num]).astype(float)
        
        y_2 = x[2] / (x[1]+x[2])
        y_3 = x[3] / (x[1]+x[2]+x[3])
        risk = y_2/2 + y_3/3
        
        return dict(
            x_1=x[1],
            x_2=x[2],
            x_3=x[3],
            y_2=y_2,
            y_3=y_3,
            z=risk if risk else 0
        )        
    
    @__fault_tolerant
    def get_feature_20(cls):
        '''
        1、潜在违规融资风险
        2、某度关联方中新金融企业数量
        '''    
        def get_relation_info(distance):
            return sum([
                1
                for node, attr in cls.DIG.nodes_iter(data=True)
                if attr['distance'] == distance 
                and attr['is_new_finance']
            ])
        
        def get_relation_risk_num(keyword_type):
            return Counter([attr['distance']
                    for node, attr in cls.DIG.nodes_iter(data=True) 
                    if attr['is_human'] == 0 and 
                    attr['opescope'] == keyword_type])
        
        k_2_num = get_relation_risk_num('k_2')
        k_1_num = get_relation_risk_num('k_1')
        
        x = np.array([
                k_2_num.get(each_distance, 0) 
                for each_distance in xrange(1, 4)])
        y = np.array([
                k_1_num.get(each_distance, 0) 
                for each_distance in xrange(1, 4)])
    
        risk = round(
            np.sum(
                np.divide(np.add(x, y), [1, 2, 3], dtype=float)), 2)
    
        return dict(
            x_1=x[0],
            x_2=x[1],
            x_3=x[2],
            y_1=y[0],
            y_2=y[1],
            y_3=y[2],
            z_1=get_relation_info(1),
            z_2=get_relation_info(2),
            z_3=get_relation_info(3),
            g=risk
        )
    
    @__fault_tolerant
    def get_feature_21(cls):
        '''
        关联方地址集中度风险
        '''
        legal_person_address = [
            attr['address'] 
            for node, attr in cls.DIG.nodes_iter(data=True) 
            if attr['distance'] <= 3 
            and attr['is_human'] == 0]
        c = Counter(
            filter(
                lambda x: x is not None and len(x) >= 21,  
                legal_person_address))
        n = c.most_common(1)
        n= n[0][1] if len(n) > 0 else 1
        risk = n -1
                 
        return dict(
            n=risk,
            y=risk
        )
    
    @__fault_tolerant
    def get_feature_22(cls):
        '''
        利益一致行动法人风险
        '''
        def is_similarity(str_1, str_2):
            '''
            判断两个字符串是否有连续2个字相同
            '''
            try:
                token_1 = [
                    str_1[index] + str_1[index+1] 
                    for index,data in enumerate(str_1) 
                    if index < len(str_1) - 1]
                is_similarity = sum([
                        1 for each_token in token_1 
                        if each_token in str_2])
                return True if is_similarity > 0 else False        
            except:
                return False
            
        #目标公司的名称frag
        tar_company = cls.resultiterable.data[0].a_name
        tar_company_frag = cls.resultiterable.data[0].a_namefrag
        #三度以内，所有关联方的节点集合(不包含自身)
        relation_set = [
                attr['name'] 
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if attr['distance'] <= 3]
        #relation_set.remove(tar_company)
        common_interests_list = [
            1 
            for node_name in relation_set 
            if is_similarity(tar_company_frag, node_name)]
        
        risk = sum(common_interests_list)
        
        return dict(
            d=risk, 
            z=risk
        )        
        
    @__fault_tolerant
    def get_feature_23(cls):
        '''
        关联方信誉风险
        '''
        def get_black_num(distance):
            '''
            某一度黑企业的总数
            '''
            return sum([
                    1
                    for node, attr in cls.DIG.nodes_iter(data=True)
                    if (attr['is_black'] and attr['distance'] == distance 
                            and attr['is_human'] == 0)
            ])
        
        def get_common_interests_num(distance):
            '''
            与黑名单库中任意一家企业存在利益一致行动关系
            '''
            return sum([
                    1
                    for node, attr in cls.DIG.nodes_iter(data=True)
                    if attr['distance'] == distance 
                    and attr['common_interests']
            ])
        
        def get_common_address_num(distance):
            '''
            与黑名单库中任意一家企业地址相同
            '''
            return sum([
                    1
                    for node, attr in cls.DIG.nodes_iter(data=True)
                    if attr['distance'] == distance 
                    and attr['common_address']
            ])
        
        black_relation_set =[
            get_black_num(each_distance) 
            for each_distance in range(1, 4)]
        common_interests_set=[
            get_common_interests_num(each_distance) 
            for each_distance in range(1, 4)]
        common_address_set=[
            get_common_address_num(each_distance) 
            for each_distance in range(1, 4)]
        
        risk = np.dot(black_relation_set, [1, 1/2., 1/3.])
        
        return dict(
            b_1=black_relation_set[0],
            b_2=black_relation_set[1],
            b_3=black_relation_set[2],
            c_1=common_interests_set[0],
            c_2=common_interests_set[1],
            c_3=common_interests_set[2],
            d_1=common_address_set[0],
            d_2=common_address_set[1],
            d_3=common_address_set[2],
            y=risk
        )
    
    @__fault_tolerant
    def get_feature_24(cls):
        '''
        短期逐利风险
        '''
        
        def get_max_established(distance, timedelta):
            '''
            获取在某distance关联方企业中，某企业在任意timedelta内投资成立公司数量的最大值
            '''
            #用于获取最大连续时间数，这里i的取值要仔细琢磨一下，这里输入一个时间差序列与时间差
            def get_date_density(difference_list, timedelta):
                time_density_list = []
                for index, date in enumerate(difference_list):
                    if date < timedelta:
                        s = 0
                        i = 1
                        while(s < timedelta and i <= len(difference_list) - index):
                            i += 1
                            s = sum(difference_list[index:index+i])
                        time_density_list.append(i)
                    else:
                        continue
                return max(time_density_list) if len(time_density_list) >0 else 0            

            #distance所有节点集合
            relation_set = [
                node 
                for node, attr in cls.DIG.nodes(data=True) 
                if attr['distance'] == distance]
            investment_dict = defaultdict(lambda : [])     
            
            if len(cls.DIG.edge) > 1:
                for src_node, des_node, edge_attr in (
                        cls.DIG.edges_iter(relation_set, data=True)):
                    if (edge_attr.get('is_invest', 0) == 'INVEST' 
                            and cls.DIG.node[des_node]['distance'] == distance
                            and cls.DIG.node[des_node]['esdate'] is not None 
                            and cls.DIG.node[src_node]['is_human'] == 0):
                        #将所有节点投资的企业的成立时间加进列表中
                        investment_dict[src_node].append(cls.DIG.node[des_node]['esdate'])
             
            #目标企业所有节点所投资的企业时间密度字典
            all_date_density_dict = {}
            
            for node, date_list in investment_dict.iteritems():
                #构建按照时间先后排序的序列
                date_strp_list = sorted(date_list)
                if len(date_strp_list) > 1:
                    #构建时间差的序列，例如：[256, 4, 5, 1, 2, 33, 6, 5, 4, 73]
                    date_difference_list = [
                        (date_strp_list[index + 1] - date_strp_list[index]).days 
                        for index in range(len(date_strp_list) -1)]
                    #计算某法人节点在timedelta天之内有多少家公司成立
                    es_num = get_date_density(date_difference_list, timedelta)
                    if all_date_density_dict.has_key(es_num):
                        all_date_density_dict[es_num].append(node)
                    else:
                        all_date_density_dict[es_num] = [node]              
                else:
                    continue
            keys = all_date_density_dict.keys()        
            max_num = max(keys) if len(keys) > 0 else 0
            
            return max_num
        
        x = [
            get_max_established(each_distance, 180) 
            for each_distance in xrange(0, 4)]
        
        legal_person_num = sum([
            1 
            for node, attr in cls.DIG.nodes(data=True) 
            if not attr['is_human']])
        
        risk = round(
            np.sum(
                np.divide(x, [1, 2, 3, 4], dtype=float)), 2) / legal_person_num * 15
        
        if  risk == 0:
            r = 10
        elif 0 < risk <= 0.7:
            r = 30
        elif 0.7 < risk <= 1:
            r = 60  
        elif 1 < risk:
            r = 100       
        else:
            r = 10
        
        return dict(
            x_0=x[0],
            x_1=x[1],
            x_2=x[2],
            x_3=x[3],
            w=legal_person_num,
            z=risk
        )
    
    @__fault_tolerant
    def get_feature_25(cls):
        '''
        目标公司在任意6个月内分支机构数量的最大值
        '''
        
        def get_max_established(distance, timedelta):
            '''
            获取在某distance关联方企业中，某企业在任意timedelta内投资成立公司（分支机构）数量的最大值
            '''
            #用于获取最大连续时间数，这里i的取值要仔细琢磨一下，这里输入一个时间差序列与时间差
            def get_date_density(difference_list, timedelta):
                time_density_list = []
                for index, date in enumerate(difference_list):
                    if date < timedelta:
                        s = 0
                        i = 1
                        while(s < timedelta and i <= len(difference_list) - index):
                            i += 1
                            s = sum(difference_list[index:index+i])
                        time_density_list.append(i)
                    else:
                        continue
                return max(time_density_list) if len(time_density_list) >0 else 0            

            #distance所有节点集合
            relation_set = [
                node 
                for node, attr in cls.DIG.nodes(data=True) 
                if attr['distance'] == distance]
            investment_dict = defaultdict(lambda : [])     
            
            if len(cls.DIG.edge) > 1:
                for src_node, des_node, edge_attr in (
                        cls.DIG.edges_iter(relation_set, data=True)):
                    if (edge_attr.get('is_invest', 0) == 'INVEST' 
                            and cls.DIG.node[des_node]['distance'] == distance
                            and cls.DIG.node[des_node]['esdate'] is not None 
                            and cls.DIG.node[src_node]['is_human'] == 0
                            and cls.DIG.node[des_node]['is_fzjg']):
                        #将所有节点投资的企业（分支机构）的成立时间加进列表中
                        investment_dict[src_node].append(cls.DIG.node[des_node]['esdate'])
             
            #目标企业所有节点所投资的企业时间密度字典
            all_date_density_dict = {}
            
            for node, date_list in investment_dict.iteritems():
                #构建按照时间先后排序的序列
                date_strp_list = sorted(date_list)
                if len(date_strp_list) > 1:
                    #构建时间差的序列，例如：[256, 4, 5, 1, 2, 33, 6, 5, 4, 73]
                    date_difference_list = [
                        (date_strp_list[index + 1] - date_strp_list[index]).days 
                        for index in range(len(date_strp_list) -1)]
                    #计算某法人节点在timedelta天之内有多少家公司成立
                    es_num = get_date_density(date_difference_list, timedelta)
                    if all_date_density_dict.has_key(es_num):
                        all_date_density_dict[es_num].append(node)
                    else:
                        all_date_density_dict[es_num] = [node]              
                else:
                    continue
            keys = all_date_density_dict.keys()        
            max_num = max(keys) if len(keys) > 0 else 0
            
            return max_num
        
        x = [
            get_max_established(each_distance, 180) 
            for each_distance in xrange(0, 4)]
        
        return dict(
            x_0=x[0],
            x_1=x[1],
            x_2=x[2],
            x_3=x[3]
        )
    
    
    @classmethod
    def get_some_feature(cls, resultiterable, feature_nums):
        #创建类变量
        cls.resultiterable = resultiterable
        cls.tarcompany = cls.resultiterable.data[0].a
        cls.DIG = cls.create_graph_udf(cls.resultiterable, 1)
        cls.xgxx_distribution = cls.get_feature_xgxx()
        
        feature_list = [
            ('feature_{0}'.format(feature_index), 
                     methodcaller('get_feature_{0}'.format(feature_index))(cls))
            for feature_index in feature_nums]
        feature_list.append(('bbd_qyxx_id', cls.tarcompany))
        feature_list.append(('company_name', cls.resultiterable.data[0].a_name))
        
        return dict(feature_list) #json.dumps(dict(feature_list), ensure_ascii=False)      
    
    @classmethod
    def get_some_feature_test(cls, resultiterable, dig, feature_nums):
        #创建类变量
        cls.resultiterable = resultiterable
        cls.tarcompany = cls.resultiterable.data[0].a
        cls.DIG =  cls.create_graph_udf(cls.resultiterable, 1)
        cls.xgxx_distribution = cls.get_feature_xgxx()
        
       
        #eval('cls.get_feature_{0}()'.format(feature_index))
        feature_list = [
            ('feature_{0}'.format(feature_index), 
                     operator.methodcaller('get_feature_{0}'.format(feature_index))(cls))
            for feature_index in feature_nums]
        feature_list.append(('bbd_qyxx_id', cls.tarcompany))
        feature_list.append(('company_name', cls.resultiterable.data[0].a_name))
        
        return dict(feature_list)
    
def spark_data_flow(tidversion):
    '''
    dataflow
    '''
    #输入
    #算法设计直接用rdd
    tid_df = spark.read.parquet("/user/antifraud/jinli/tiddata/relation_company_info_merge/{version}".format(version=tidversion))
    tid_rdd = tid_df.rdd
        
    #最终计算流程
    tid_rdd_2 = tid_rdd.map(lambda row: (row.a_name, row)) \
        .groupByKey() \
        .repartition(600) \
        .filter(lambda r: len(r[1].data) <= 100000) \
        .cache()


    import signal
    def handler(signum, frame):
        raise AssertionError

    def fault_tolerant(func):
        def wappen(*args):
            try:
                return func(*args)
            except Exception, e:
                return e
        return wappen

    @fault_tolerant
    def time_out(data):
        signal.signal(signal.SIGALRM, handler)
        signal.alarm(60)
        result = FeatureConstruction.get_some_feature(data, 
                                                      [_ for _ in range(1, 26)])
        signal.alarm(0)
        return result        
        
    feature_list = tid_rdd_2.mapValues(
        time_out
    ).map(
        itemgetter(1)
    )
    
    return feature_list

def collect_to_driver(version, out_path, tidversion):
    '''
    格式化输出
    '''
    src_time = time.time()
    pd_df = json_normalize(spark_data_flow(tidversion).collect())
    os.system("rm /data5/antifraud/Jinli/data/prddata/prd_jinli_feature_score_distribution_{version}.xlsx".format(version=version))
    pd_df.to_excel("{path}/prd_jinli_feature_score_distribution_{version}.xlsx".format(path=OUTPATH, version=version))
    des_time = time.time()
    
    print 'SUCCESS !! \n 耗时：{0}'.format(des_time - src_time)

def to_json(version, tidversion):
    '''
    导成json
    '''
    src_time = time.time()
    pd_rdd = spark_data_flow(tidversion)
    os.system("hadoop fs -rmr /user/antifraud/jinli/prddata/prd_jinli_feature_score_distribution_{version}".format(version=version))
    pd_rdd.repartition(
        10
    ).map(
        lambda r: json.dumps(r, ensure_ascii=False)    
    ).saveAsTextFile(
        "/user/antifraud/jinli/prddata/prd_jinli_feature_score_distribution_{version}".format(version=version)
    )
    des_time = time.time()
    
    print 'SUCCESS !! \n 耗时：{0}'.format(des_time - src_time)    
    
if __name__ == '__main__':  
    #输入参数
    VERSION = "20180724_01"
    OUTPATH = "/data5/antifraud/Jinli/data/prddata"
    
    #collect_to_driver(
    #    version=VERSION,
    #    out_path=OUTPATH,
    #     tidversion='20170117'
    # )

    to_json(version=VERSION,
                 tidversion=relation_version)
    #print spark_data_flow('20170117').take(3)

SUCCESS !! 
 耗时：30.3952291012


In [48]:
'''
使用mllib来做标准化
'''
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler
from  pyspark.sql.types import StructType, StructField, StringType, IntegerType

input_df = spark.read.parquet(
    "/user/antifraud/hongjing2/dataflow/step_two/raw/nf_feature_merge/20170221"
)

def get_vectors(row):
    if row['feature_26']:
        return (Vectors.dense([
                0,
                row['feature_2']['c'],
                row['feature_3']['z'],
                0,
                row['feature_5']['l'],
                row['feature_6']['z'],
                row['feature_7']['z'],
                row['feature_8']['y'],
                row['feature_9']['n'],
                row['feature_10']['z'],
                row['feature_11']['z'],
                0,
                row['feature_13']['z'],
                0,
                row['feature_15']['r'],
                row['feature_16']['r'],
                row['feature_17']['r'],
                row['feature_18']['z'],
                0,
                row['feature_20']['g'],
                row['feature_21']['y'],
                row['feature_22']['z'],
                row['feature_23']['y'],
                row['feature_24']['z'],
                row['feature_26']['a_1'],
                0,
                0,
                row['feature_26']['a_4'],
                row['feature_26']['a_5'],
                row['feature_26']['a_6'],
                0,
                row['feature_26']['b_1'],
                row['feature_26']['b_2'],
                row['feature_26']['b_3'],
                0,
                0,
                row['feature_26']['c_1'],
                0,
                row['feature_26']['d_2'],
            ]),        
            row['bbd_qyxx_id'],
            row['company_name'])
    else:
        return (Vectors.dense([
                0,
                row['feature_2']['c'],
                row['feature_3']['z'],
                0,
                row['feature_5']['l'],
                row['feature_6']['z'],
                row['feature_7']['z'],
                row['feature_8']['y'],
                row['feature_9']['n'],
                row['feature_10']['z'],
                row['feature_11']['z'],
                0,
                row['feature_13']['z'],
                0,
                row['feature_15']['r'],
                row['feature_16']['r'],
                row['feature_17']['r'],
                row['feature_18']['z'],
                0,
                row['feature_20']['g'],
                row['feature_21']['y'],
                row['feature_22']['z'],
                row['feature_23']['y'],
                row['feature_24']['z'],
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
                0,
            ]), 
            row['bbd_qyxx_id'],
            row['company_name'])
        

tid_vector_df = input_df.rdd.map(
    get_vectors
).toDF(
).withColumnRenamed(
    '_1', 'features'
).withColumnRenamed(
    '_2', 'bbd_qyxx_id'
).withColumnRenamed(
    '_3', 'company_name'
)

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=True)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(tid_vector_df)

# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(tid_vector_df)
scaledData.show()

+--------------------+--------------------+-------------------+--------------------+
|            features|         bbd_qyxx_id|       company_name|      scaledFeatures|
+--------------------+--------------------+-------------------+--------------------+
|[0.0,100.0,30.0,0...|673c99d996ca44d18...|       上海上豪投资管理有限公司|[0.0,0.8490061918...|
|[0.0,60.0,30.0,0....|9dd8f06edac743e1a...|      上海会钜富投资管理有限公司|[0.0,-0.264613252...|
|[0.0,100.0,22.5,0...|15eae5aa74da451ab...|       上海勤合投资咨询有限公司|[0.0,0.8490061918...|
|[0.0,100.0,30.0,0...|44f9c3403cf64206a...|   上海奔瑞投资合伙企业（有限合伙）|[0.0,0.8490061918...|
|[0.0,100.0,30.0,0...|a809a0a3cc6d48668...|       上海密彗投资咨询有限公司|[0.0,0.8490061918...|
|[0.0,100.0,30.0,0...|10deda38649b4c92a...|   上海心云投资管理中心（有限合伙）|[0.0,0.8490061918...|
|[0.0,30.0,30.0,0....|ab2944d05f1840e98...|       上海旗绩投资管理有限公司|[0.0,-1.099827836...|
|[0.0,100.0,30.0,0...|84ceb3198b84477ab...|     上海梓珖投资中心（有限合伙）|[0.0,0.8490061918...|
|[0.0,10.0,30.0,0....|9a9bd7bd216b482fb...|       上海满加投资咨询有限公司|[0

In [66]:
input_df = spark.read.parquet("/user/antifraud/hongjing2/dataflow/step_two/tid/nf_feature_preprocessing/20170221")


In [67]:
def get_label_probability(iter_data):
    '''计算判黑概率'''

    from sklearn.externals import joblib
    
    company_names = []
    bbd_qyxx_ids = []
    data_set = []
    
    for each_row in iter_data:
        data = each_row['scaledFeatures'].values
        company_names.append(each_row['company_name'])
        bbd_qyxx_ids.append(each_row['bbd_qyxx_id'])
        data_set.append(data)
        
    data_set = filter(lambda x: 1 if len(x) == 39  else 0, data_set)
    lr = joblib.load("GM_release_LR.model")
    raw_prob = lr.predict_proba(data_set)
    
    return zip(bbd_qyxx_ids, company_names, raw_prob[:, 1], data_set)

In [ ]:
#step_one 计算判黑概率
raw_rdd = input_df.rdd.repartition(
    100
).mapPartitions(
    get_label_probability
).cache()

data = raw_rdd.collect()

In [ ]:
'''
特征计算,打分
'''
import json
import math
import numpy as np

input_df = spark.read.parquet("/user/antifraud/hongjing2/dataflow/step_two/tid/nf_feature_preprocessing/20170221")

def sigmoid(x):
    return 1. / (1 + math.exp(-x))

def get_first_grade_indexes(row):
    '''计算一级指标'''
    vector = row[4]
    F1_weight = [78.995978788237665, 19.643289163698, 1.3607320480643235]
    F2_weight = [26.060202553046778, 32.455344310155851, 
                         35.422074212291704, 6.062378924505663]
    F3_weight = [8.5671628439652956, 67.099469406473432, 24.33336774956128]
    F4_weight = [5.4992316645363379, 0.73619670729601905, 
                         16.465178242190262, 6.8112990078436821, 
                         27.336531281175152, 5.9500466959961198, 
                         0.94871599993642608, 35.054955599386361, 1.1978448016396601]
    F5_weight = [2.1642169432775109, 7.2205710674551362, 
                         2.7554815418792837, 1.8386921453073, 
                         4.9583378078768945, 0.24537414679201996,
                         0.14077225761444445, 75.358040614215426, 5.3185134755819945]
    
    #综合实力风险
    GM_company_strength_risk = np.dot(F1_weight, map(sigmoid,
                                                     [vector[1], vector[2], vector[4]]))
    #经营行为风险
    GM_behavior_risk = np.dot(F2_weight, map(sigmoid,
                                             [vector[5], vector[6], vector[7], vector[8]]))
    #企业诚信风险
    GM_credit_risk = np.dot(F3_weight, map(sigmoid,
                                           [vector[9], vector[10], vector[12]]))
    #静态关联方风险
    GM_static_relationship_risk = np.dot(F4_weight, map(sigmoid,
                                                        [vector[14], vector[15], vector[16],
                                                         vector[17], vector[19], vector[20],
                                                         vector[21], row[4][22], vector[23]]))
    #动态关联方风险
    GM_dynamic_relationship_risk = np.dot(F5_weight, map(sigmoid,
                                                         [vector[24], vector[27], vector[28],
                                                          vector[29], vector[31], vector[32],
                                                          vector[33], vector[36], vector[38]]))
    return dict(
            bbd_qyxx_id=row[0],
            company_name=row[1],
            total_score=row[3],
            GM_company_strength_risk=round(GM_company_strength_risk, 1),
            GM_behavior_risk=round(GM_behavior_risk, 1),
            GM_credit_risk=round(GM_credit_risk, 1),
            GM_static_relationship_risk=round(GM_static_relationship_risk, 1),
            GM_dynamic_relationship_risk=round(GM_dynamic_relationship_risk, 1)
        )


def change_prob_score(row, quantile_one, quantile_two):
    '''根据分位点，将判黑概率值转化成分'''
    raw_prob = sigmoid(row[2])
    quantile_one = sigmoid(quantile_one)
    quantile_two = sigmoid(quantile_two)
    
    if raw_prob >= quantile_one:
        score = (raw_prob - quantile_one) * 40. / (0.75 - quantile_one) + 50
    elif quantile_two <= raw_prob <= quantile_one:
        score = ((raw_prob - quantile_two) * 50. / (quantile_one - quantile_two) + 
                    (quantile_one - raw_prob) * 30. / (quantile_one - quantile_two))
    elif raw_prob < quantile_two:
        score = (raw_prob - quantile_two) * 30. / quantile_two + 30
    
    return (row[0], row[1], row[2], round(score, 1), row[3])
    
def get_label_probability(iter_data):
    '''计算判黑概率'''

    from sklearn.externals import joblib
    
    company_names = []
    bbd_qyxx_ids = []
    data_set = []
    
    for each_row in iter_data:
        data = each_row['scaledFeatures'].values
        company_names.append(each_row['company_name'])
        bbd_qyxx_ids.append(each_row['bbd_qyxx_id'])
        data_set.append(data)
        
    data_set = filter(lambda x: 1 if len(x) == 39 else 0, data_set)
    lr = joblib.load("GM_release_LR.model")
    raw_prob = lr.predict_proba(data_set)
    
    return zip(bbd_qyxx_ids, company_names, raw_prob[:, 1], data_set)


def get_label_probability_2(each_row):
    '''计算判黑概率'''
    from sklearn.externals import joblib
    data = each_row['scaledFeatures'].values
    lr = joblib.load("GM_release_LR.model")
    raw_prob = lr.predict_proba(data)
    return (each_row['bbd_qyxx_id'], each_row['company_name'], raw_prob[0, 1])


#step_one 计算判黑概率
raw_rdd = input_df.rdd.repartition(
    100
).mapPartitions(
    get_label_probability
).cache()


#step_two 将判黑概率转换成分数
#计算分位点
score_distribution = raw_rdd.map(
    lambda r: r[2]
).collect()
score_distribution.sort(reverse=True)
#取5%与50%的分位点
top_five_index = int(len(score_distribution) * 0.05) - 1
top_fifty_index = int(len(score_distribution) * 0.5) - 1
#得到分位点
Y_1 = score_distribution[top_five_index]
Y_2 = score_distribution[top_fifty_index]
#得到结果
tid_rdd = raw_rdd.map(
    lambda r: change_prob_score(r, Y_1, Y_2)
).cache()

#step_three 计算一级指标的得分
prd_rdd = tid_rdd.map(
    get_first_grade_indexes
)

#输出
codec = "org.apache.hadoop.io.compress.GzipCodec"

prd_rdd.map(
    lambda r: json.dumps(r, ensure_ascii=False)
).coalesce(
    10
)

In [86]:
'''
打标签
'''

from pyspark.sql import Row

def get_tags(row):
    
    def get_behavior_risk(risk_name=u"经营行为风险"):
        tags = []
        if row['feature_6']['c_1'] >= 2:
            tags.append(u'法定代表人变更频繁')
        if row['feature_6']['c_2'] >= 2:
            tags.append(u'股东变更频繁')
        if row['feature_6']['c_3'] >= 2:
            tags.append(u'注册资本变更频繁')
        if row['feature_6']['c_4'] >= 2:
            tags.append(u'高管变更频繁')
        if row['feature_6']['c_5'] >= 2:
            tags.append(u'经营范围变更频繁')
        if row['feature_18']['d'] >= 2:
            tags.append(u'分支机构数量较多')
        if row['feature_22']['z'] >= 2:
            tags.append(u'企业利益一致行动法人较多')
        return {
            risk_name: tags
        }
    
    def get_company_strength_risk(risk_name=u'综合实力风险'):
        tags = []
        if row['feature_8']['t_1'] and row['feature_8']['t_1'] <= 700:
            tags.append(u'公司成立时间较短')
        if (row['feature_7']['e'] and 
                row['feature_7']['e_1'] / row['feature_7']['e'] >= 0.3):
            tags.append(u'大专及大专以下或不限专业招聘比例较高')
        if row['feature_17']['x'] >= 3:
            tags.append(u'对外投资公司数量较多')
        return {
            risk_name: tags
        }
    
    def get_credit_risk(risk_name=u'企业诚信风险'):
        tags = []
        if (row['feature_10']['0']['ktgg'] + 
                row['feature_10']['0']['rmfygg'] +
                row['feature_10']['0']['zgcpwsw']) >= 10:
            tags.append(u'企业诉讼文书数量较多')
        if row['feature_10']['0']['lending']:
            tags.append(u'企业存在“民间借贷”类诉讼文书')
        if row['feature_11']['0']['xzcf']:
            tags.append(u'企业受到行政处罚')
        if row['feature_12']['0']['zhixing']:
            tags.append(u'企业存在被执行人信息')
        if row['feature_12']['0']['dishonesty']:
            tags.append(u'企业存在失信被执行人信息')
        if row['feature_13']['0']['jyyc']:
            tags.append(u'企业存在经营异常信息')
        return {
            risk_name: tags
        }
    
    def get_all_nums(feature_name, distances=['0', '1', '2', '3']):
        return sum(
            map(
                sum, 
                [v.asDict().values() 
                     for k ,v in row[feature_name].asDict().iteritems() 
                     if k in distances]))
    def get_some_nums(feature_name, value_name, distances=['0', '1', '2', '3']):
        return sum([
                v.asDict().get(value_name, 0)
                for k, v in row[feature_name].asDict().iteritems()
                if k in distances])         
    def get_static_relationship_risk(risk_name=u'静态关联方风险'):
        tags = []
        if get_all_nums('feature_10', distances=['0', '1', '2']) >= 15:
            tags.append(u'二度内关联方诉讼文书数量较多')
        if get_some_nums('feature_10', 'lending') >= 3:
            tags.append(u'三度内关联方存在较多“民间借贷”类诉讼文书')
        if get_some_nums('feature_11', 'xzcf', distances=['0', '1', '2']):
            tags.append(u'二度内关联方存在受行政处罚企业')
        if get_some_nums('feature_12', 'zhixing'):
            tags.append(u'三度内关联方存在被执行人企业数量较多')
        if get_some_nums('feature_12', 'dishonesty'):
            tags.append(u'三度内关联方存在失信被执行人企业')
        if get_some_nums('feature_13', 'estatus', distances=['0', '1', '2']):
            tags.append(u'二度内关联方存在吊销企业')
        if get_some_nums('feature_13', 'jyyc', distances=['0', '1', '2']):
            tags.append(u'二度内关联方存在经营异常企业')
        if row['feature_8']['t_2'] and row['feature_8']['t_2'] <= 365:
            tags.append(u'二度内法人关联方平均成立时间较短')
        if row['feature_15']['x_1'] >= 10:
            tags.append(u'二度内单个关联自然人中最大投资企业数量较多')
        if row['feature_15']['x_2'] >= 25:
            tags.append(u'二度内单个关联法人最大投资企业数量较多')
        if ((row['feature_16']['y_1'] + row['feature_16']['y_2']) /
               (row['feature_16']['y_1'] + row['feature_16']['y_2'] +
                row['feature_16']['x_1'] + row['feature_16']['x_2'] + 0.01)) >= 0.9:
            tags.append(u'二度内关联方中自然人比例较高')
        if row['feature_21']['n'] >= 2:
            tags.append(u'关联企业中地址相同公司较多')
        if (row['feature_23']['b_1'] + row['feature_23']['b_2']) :
            tags.append(u'二度内关联方中存在黑名单企业')
        return {
            risk_name: tags
        }
            
    def get_dynamic_relationship_risk(risk_name=u'动态关联方风险'):
        tags = []
        if row['feature_26']['a_1'] > 2 or row['feature_26']['a_1'] < -0.7:
            tags.append(u'近三个月内自然人关联节点变动较大')
        if row['feature_26']['a_4'] > 0.1 or row['feature_26']['a_4'] < -0.1:
            tags.append(u'近三个月内对外投资公司数量变动较大')
        if row['feature_26']['a_5'] > 0.5:
            tags.append(u'近三个月内公司分支机构数量变动较大')
        if row['feature_26']['a_6'] > 0.5:
            tags.append(u'近三个月内利益一致行动法人的数量变动较大')
        if row['feature_26']['c_1'] > 1 or row['feature_26']['c_1'] < -0.5:
            tags.append(u'近三个月内低学历招聘人数变动较大')
        if row['feature_26']['d_2'] > 2 or row['feature_26']['d_2'] < -0.7:
            tags.append(u'近三个月内3度及3度以下核心自然人（前3）控制节点总数变动较大')
        return {
            risk_name: tags
        }
    
    result = {}
    result.update(get_behavior_risk())
    result.update(get_company_strength_risk())
    result.update(get_credit_risk())
    result.update(get_static_relationship_risk())    
    if row['feature_26']:
        result.update(get_dynamic_relationship_risk())
    else:
        result.update({u'动态关联方风险': []})
        
    final_out_dict = {
        '--': result
    }
    
    return json.dumps(final_out_dict, ensure_ascii=False)

RELATION_VERSION = '20170221'

raw_nf_feature_df = spark.read.parquet(
    ("/user/antifraud/hongjing2/dataflow/step_two/raw"
     "/nf_feature_merge/{version}").format(version=RELATION_VERSION))
nf_info_df = spark.read.parquet(
    ("/user/antifraud/hongjing2/dataflow/step_three/raw"
     "/nf_info_merge/{version}").format(version=RELATION_VERSION))

tid_nf_feature_df = raw_nf_feature_df.rdd.map(
    lambda r: Row(
        company_name=r['company_name'],
        bbd_qyxx_id=r['bbd_qyxx_id'],
        risk_tags=get_tags(r)
    )
).toDF().cache()

tid_nf_feature_df.show()

+--------------------+-------------------+--------------------+
|         bbd_qyxx_id|       company_name|           risk_tags|
+--------------------+-------------------+--------------------+
|673c99d996ca44d18...|       上海上豪投资管理有限公司|{"--": {"静态关联方风险"...|
|9dd8f06edac743e1a...|      上海会钜富投资管理有限公司|{"--": {"静态关联方风险"...|
|15eae5aa74da451ab...|       上海勤合投资咨询有限公司|{"--": {"静态关联方风险"...|
|44f9c3403cf64206a...|   上海奔瑞投资合伙企业（有限合伙）|{"--": {"静态关联方风险"...|
|a809a0a3cc6d48668...|       上海密彗投资咨询有限公司|{"--": {"静态关联方风险"...|
|10deda38649b4c92a...|   上海心云投资管理中心（有限合伙）|{"--": {"静态关联方风险"...|
|ab2944d05f1840e98...|       上海旗绩投资管理有限公司|{"--": {"静态关联方风险"...|
|84ceb3198b84477ab...|     上海梓珖投资中心（有限合伙）|{"--": {"静态关联方风险"...|
|9a9bd7bd216b482fb...|       上海满加投资咨询有限公司|{"--": {"静态关联方风险"...|
|f205d49d436e4169b...|       上海玖健投资管理有限公司|{"--": {"静态关联方风险"...|
|0adeaa5dd8404c0d9...|         上海盈太投资有限公司|{"--": {"静态关联方风险"...|
|04c0515d2cbb43508...|   上海福沿投资合伙企业（有限合伙）|{"--": {"静态关联方风险"...|
|339ebf91f8ea4938b...|     上海缘传园林景观工程有限公

In [97]:
RELATION_VERSION = '20170221'

df = spark.read.parquet(
    '/user/antifraud/hongjing2/dataflow/step_three/tid/nf_feature_tags/20170221'
)
df.take(10)

[Row(company_name=u'\u4e0a\u6d77\u4e0a\u8c6a\u6295\u8d44\u7ba1\u7406\u6709\u9650\u516c\u53f8', bbd_qyxx_id=u'673c99d996ca44d18e171f01246d12ad', risk_tags=u'{"--": {"\u9759\u6001\u5173\u8054\u65b9\u98ce\u9669": [], "\u7ecf\u8425\u884c\u4e3a\u98ce\u9669": ["\u4f01\u4e1a\u5229\u76ca\u4e00\u81f4\u884c\u52a8\u6cd5\u4eba\u8f83\u591a"], "\u7efc\u5408\u5b9e\u529b\u98ce\u9669": [], "\u52a8\u6001\u5173\u8054\u65b9\u98ce\u9669": [], "\u4f01\u4e1a\u8bda\u4fe1\u98ce\u9669": []}}', risk_index=30.0, risk_composition=u'{"--": {"\u9759\u6001\u5173\u8054\u65b9\u98ce\u9669": 49.6, "\u4f01\u4e1a\u8bda\u4fe1\u98ce\u9669": 47.0, "total_score": 30.0, "\u7efc\u5408\u5b9e\u529b\u98ce\u9669": 67.4, "\u52a8\u6001\u5173\u8054\u65b9\u98ce\u9669": 48.2, "\u7ecf\u8425\u884c\u4e3a\u98ce\u9669": 52.6}}', province=u'\u4e0a\u6d77\u5e02', city=u'\u4e0a\u6d77\u5e02', county=u'\u95f5\u884c\u533a', is_black=None),
 Row(company_name=u'\u4e0a\u6d77\u4f1a\u949c\u5bcc\u6295\u8d44\u7ba1\u7406\u6709\u9650\u516c\u53f8', bbd_qyxx_i

In [68]:
print Y_1
print Y_2

0.0168715545885
0.000775719110953


In [107]:
min(score_distribution)

2.6138134506932581e-16

In [109]:
sigmoid(min(score_distribution))

0.5000000000000001

In [118]:
def sigmoid(x):
    return 1. / (1 + math.exp(-x))

#130W家公司最小的概率
raw_prob = sigmoid(min(score_distribution)*50000)
#50%分位点
quantile_two = sigmoid(0.000775719110953*50000)

#原始值
raw_prob * 30. / quantile_two

15.000000000098018

In [98]:
df = spark.read.json("/user/antifraud/hongjing2/dataflow/step_two/prd/nf_feature_risk_score/20170221")
data = df.rdd.map(lambda r: r.total_score).collect()


In [125]:
data.sort(reverse=True)


15.0

In [120]:
(sigmoid(50.2 / 100.) * 100 -62) * 15 / 10. + 50

50.43938350763932

0.5000000000032673

In [36]:
from sklearn.externals import joblib
import random

lr = joblib.load("/data5/antifraud/Hongjing2/data/inputdata/model/GM_release_LR.model")

result = lr.predict_proba([[0 for _ in range(39)]])
result[0, 1]

0.0015930328110282772

In [90]:
data = input_df.rdd.map(lambda r: r.scaledFeatures.values).take(3)
lr.predict_proba(data)

array([[  9.99263620e-01,   7.36380271e-04],
       [  9.99314607e-01,   6.85393289e-04],
       [  9.97008961e-01,   2.99103903e-03]])

In [115]:
from pyspark.sql import functions as fun
from pyspark.sql import types as tp

TR = True

def is_has(col, s=TR):
    if 'has' in col and s:
        return True
    else:
        return False
is_has_udf = fun.udf(is_has, tp.BooleanType())


input_df.where(is_has_udf('feature_1')).take(10)

[]

In [16]:
tid_df = spark.read.parquet("/user/antifraud/jinli/tiddata/relation_company_info_merge/{version}".format(version='20170403')).limit(100000)
tid_rdd = tid_df.rdd
    
#最终计算流程
tid_rdd_2 = tid_rdd.map(lambda row: (row.a_name, row)) \
    .groupByKey() \
    .repartition(600) \
    .filter(lambda r: len(r[1].data) <= 100000) \
    .cache()

feature_list = tid_rdd_2.mapValues(
    lambda v: FeatureConstruction.get_some_feature(v, [_ for _ in range(3, 4)])
).map(
    itemgetter(1)
).take(10)

feature_list

[{'bbd_qyxx_id': u'111146dbf706468899add1ac18174eed',
  'company_name': u'\u5317\u4eac\u535a\u745e\u5b89\u5eb7\u79d1\u6280\u53d1\u5c55\u6709\u9650\u516c\u53f8',
  'feature_3': {'x_0': 0, 'x_1': 0, 'x_2': 0, 'x_3': 0, 'z': 30.0}},
 {'bbd_qyxx_id': u'23cbb08e51fc4df0aa3a46acd5e3320e',
  'company_name': u'\u5317\u4eac\u6b23\u8f89\u7535\u529b\u6295\u8d44\u6709\u9650\u516c\u53f8',
  'feature_3': {'x_0': 0, 'x_1': 0, 'x_2': 0, 'x_3': 0, 'z': 30.0}},
 {'bbd_qyxx_id': u'1f33b89ebe614ef1bd9f64c1b213d9a7',
  'company_name': u'\u5317\u4eac\u878d\u5c1a\u667a\u5178\u6295\u8d44\u7ba1\u7406\u6709\u9650\u516c\u53f8',
  'feature_3': {'x_0': 0, 'x_1': 0, 'x_2': 0, 'x_3': 0, 'z': 30.0}},
 {'bbd_qyxx_id': u'9ff2229092e749d0944a68b0c9545039',
  'company_name': u'\u541b\u6d77\u65f6\u4ee3\u6295\u8d44\u6709\u9650\u516c\u53f8',
  'feature_3': {'x_0': 0, 'x_1': 0, 'x_2': 0, 'x_3': 0, 'z': 30.0}},
 {'bbd_qyxx_id': u'ed3f6b62e6ae4b9e82ffc593c312a79c',
  'company_name': u'\u5317\u4eac\u94f6\u6f6e\u6295\u8d44\u54a8

In [1]:
'''
动态风险：需要2个时间节点的中间数据
'''
import datetime
import requests
import json
import time
import re
import os
from operator import itemgetter
from collections import Counter, defaultdict, OrderedDict

from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pandas.io.json import json_normalize
import networkx as nx
import numpy as np


class DynamicFeatureConstruction(FeatureConstruction):
    '''
    计算特征的函数集
    '''      

    def __fault_tolerant(func):
        '''
        一个用于容错的装饰器
        '''
        @classmethod
        def wappen(cls, *args, **kwargs):
            try:
                return func(cls, *args, **kwargs)
            except Exception, e:
                return (
                    "{func_name} has a errr : {excp}"
                ).format(func_name=func.__name__, excp=e)
        return wappen    
    
    @classmethod
    def get_feature_25(cls):
        '''
        平台稳态运营风险
        '''
        nature_person_num = sum([
                1
                for node, attr in  cls.DIG.nodes_iter(data=True)
                if (attr['is_human'] == 1
                        and attr['distance'] <= 3)
        ])
        legal_person_num = sum([
                1
                for node, attr in  cls.DIG.nodes_iter(data=True)
                if (attr['is_human'] == 0
                        and attr['distance'] <= 3)
        ])
        return dict(
            nature_person_num=nature_person_num,
            legal_person_num=legal_person_num
        )
    
    @classmethod
    def get_feature_26(cls):
        '''
        平台核心资本运作风险
        '''
        some_person_sets = [
            node
            for node, attr in cls.DIG.nodes_iter(data=True) 
            if (attr['is_human'] == 1
                    and attr['distance'] <= 3)]
        person_out_degree = cls.DIG.out_degree(
            some_person_sets).values()
        person_out_degree.sort(reverse=True)
        if not person_out_degree:
            person_out_degree.append(0)

        return dict(
            kernel_control_num=sum(person_out_degree[:3])
        )
    
    @classmethod
    def get_feature_27(cls):
        '''
        可持续性风险
        '''
        def is_similarity(str_1, str_2):
            '''
            判断两个字符串是否有连续2个字相同
            '''
            try:
                token_1 = [
                    str_1[index] + str_1[index+1] 
                    for index,data in enumerate(str_1) 
                    if index < len(str_1) - 1]
                is_similarity = sum([
                        1 for each_token in token_1 
                        if each_token in str_2])
                return True if is_similarity > 0 else False        
            except:
                return False
            
        #目标公司的名称frag
        tar_company = cls.resultiterable.data[0].a_name
        tar_company_frag = cls.resultiterable.data[0].a_namefrag
        #三度以内，所有关联方的节点集合(不包含自身)
        relation_set = [
                attr['name']
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if attr['distance'] <= 3]
        #relation_set.remove(tar_company)
        common_interests_num = sum([
                1 
                for node_name in relation_set 
                if is_similarity(tar_company_frag, node_name)
        ])
        total_legal_person_num = sum([
                1
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if (attr['distance'] <= 3 and 
                       attr['is_human'] == 0)
        ])
        if total_legal_person_num:
            ratio = round(
                common_interests_num / total_legal_person_num, 2
            )
        else:
            ratio = 0
            
        return dict(
            ratio=ratio
        )
        
    @classmethod
    def get_feature_28(cls):
        '''
        平台跨区域舞弊风险
        '''
        province_list = filter(
            None, [
                attr['province']
                for node, attr in cls.DIG.nodes_iter(data=True)
                if (attr['distance'] <= 3 and 
                       attr['is_human'] == 0)]
        )
        province_distribution = Counter(province_list)
        province_top4_num = sum(
            map(lambda (k, v): v, province_distribution.most_common(4)))
        
        return dict(
            province_top4_num=province_top4_num
        )
    
    
    @classmethod
    def get_some_feature(cls, resultiterable, feature_nums):
        #创建类变量
        cls.resultiterable = resultiterable
        cls.tarcompany = cls.resultiterable.data[0].a
        cls.DIG = cls.create_graph_udf(cls.resultiterable, 1)
        cls.G = cls.create_graph_udf(cls.resultiterable, 0)
        
        feature_list = [
            ('feature_{0}'.format(feature_index), 
                     eval('cls.get_feature_{0}()'.format(feature_index)))
            for feature_index in feature_nums]
        feature_list.append(('bbd_qyxx_id', cls.tarcompany))
        feature_list.append(('company_name', cls.resultiterable.data[0].a_name))
        
        return dict(feature_list) #json.dumps(dict(feature_list), ensure_ascii=False)      

    
def get_dynamic_risk(data):
    '''
    计算动态风险
    '''
    old_data, new_data = data
    #平台稳态运营风险
    if old_data['feature_25']['nature_person_num']:
        nature_person_risk = ((new_data['feature_25']['nature_person_num'] - 
                                            old_data['feature_25']['nature_person_num']) / 
                                            old_data['feature_25']['nature_person_num'])
    else:
        nature_person_risk = 0.
    if old_data['feature_25']['legal_person_num']:
        legal_person_risk = ((new_data['feature_25']['legal_person_num'] - 
                                          old_data['feature_25']['legal_person_num']) / 
                                          old_data['feature_25']['legal_person_num'])
    else:
        legal_person_risk = 0.
    feature_25 = 5. * (nature_person_risk + legal_person_risk)
    
    #平台核心资本运作风险
    if old_data['feature_26']['kernel_control_num']:
        feature_26 = (15. * 
                              new_data['feature_26']['kernel_control_num'] / 
                              old_data['feature_26']['kernel_control_num'])
    else:
        feature_26 = 0.
        
    #可持续性风险
    if old_data['feature_27']['ratio']:
        feature_27 = (4. * 
                              new_data['feature_27']['ratio'] / 
                              old_data['feature_27']['ratio'])
    else:
        feature_27 = 0.
        
    #平台跨区域舞弊风险
    if old_data['feature_28']['province_top4_num']:
        feature_28 = (15. * 
                              new_data['feature_28']['province_top4_num'] / 
                              old_data['feature_28']['province_top4_num'])
    else:
        feature_28 = 0.
        
    return dict(
        feature_25=round(feature_25, 2),
        feature_26=round(feature_26, 2),
        feature_27=round(feature_27, 2),
        feature_28=round(feature_28, 2)
    )
    
def spark_data_flow(tid_old_version, tid_new_version):
    '''
    dataflow
    '''
    #输入
    #算法设计直接用rdd
    tid_old_df = spark.read.parquet(
        "/user/antifraud/jinli/tiddata/relation_company_info_merge/{version}".format(version=tid_old_version))
    tid_old_rdd = tid_old_df.rdd
    tid_new_df = spark.read.parquet(
        "/user/antifraud/jinli/tiddata/relation_company_info_merge/{version}".format(version=tid_new_version))
    tid_new_rdd = tid_new_df.rdd    
        
    #最终计算流程
    tid_new_rdd_2 = tid_new_rdd.map(lambda row: (row.a_name, row)) \
        .groupByKey() \
        .repartition(600) \
        .filter(lambda r: len(r[1].data) <= 150000) \
        .cache()
    tid_old_rdd_2 = tid_old_rdd.map(lambda row: (row.a_name, row)) \
        .groupByKey() \
        .repartition(600) \
        .filter(lambda r: len(r[1].data) <= 150000) \
        .cache()

    feature_new_list = tid_new_rdd_2.mapValues(
        lambda v: DynamicFeatureConstruction.get_some_feature(
            v, [_ for _ in range(25, 29)]))
    feature_old_list = tid_old_rdd_2.mapValues(
        lambda v: DynamicFeatureConstruction.get_some_feature(
            v, [_ for _ in range(25, 29)]))
    
    dynamic_risk_data = feature_old_list.join(
        feature_new_list
    ).mapValues(
        get_dynamic_risk
    ).map(
        lambda (k, v): dict({'company_name': k}, **v)
    ).collect()
    
    
    return dynamic_risk_data

def collect_to_driver(version, out_path, old_version, new_version):
    '''
    格式化输出
    '''
    src_time = time.time()
    dynamic_risk_data = json_normalize(spark_data_flow(old_version, new_version))
    os.system("rm /data5/antifraud/Jinli/data/prddata/prd_jinli_dynamic_feature_score_distribution_{version}.xlsx".format(version=version))
    dynamic_risk_data.to_excel("{path}/prd_jinli_dynamic_feature_score_distribution_{version}.xlsx".format(path=OUTPATH, version=version))
    des_time = time.time()    
    
    print 'SUCCESS !! \n 耗时：{0}'.format(des_time - src_time)
    
    
if __name__ == '__main__':  
    #输出参数
    VERSION = "20170519_1"
    OUTPATH = "/data5/antifraud/Jinli/data/prddata"
    
    collect_to_driver(
        version=VERSION,
        out_path=OUTPATH,
        old_version='20170117',
        new_version='20170403'
    )



NameError: name 'FeatureConstruction' is not defined

In [ ]:
'''
新版动态风险：需要2个时间节点的中间数据
'''
import datetime
import requests
import json
import time
import re
import os
from operator import itemgetter
from collections import Counter, defaultdict, OrderedDict

from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pandas.io.json import json_normalize
import networkx as nx
import numpy as np


def get_dynamic_risk(data):
    '''
    计算动态风险
    '''
    old_data, new_data = data
    
    def get_result(my_denominator, my_numerator):
        '''特殊计算规则，嘻嘻'''
        if not denominator and  numerator:
            result = 65535.
        elif not denominator and  not numerator:
            result = 0.
        else:
            result = numerator * 1. / denominator 
        return result

    #3度及3度以下自然人节点变动率
    denominator = (old_data['feature_16']['y_1'] + 
                               old_data['feature_16']['y_2'] +
                               old_data['feature_16']['y_3'])
    numerator = (new_data['feature_16']['y_1'] + 
                               new_data['feature_16']['y_2'] +
                               new_data['feature_16']['y_3']) - denominator
    a_1 = get_result(denominator, numerator)
        
    #对外投资公司数量变动率
    denominator = old_data['feature_17']['x']
    numerator = new_data['feature_17']['x'] - denominator
    a_4 = get_result(denominator, numerator)
    
    #公司分支机构数量变动率
    denominator = old_data['feature_18']['d']
    numerator = new_data['feature_18']['d'] - denominator
    a_5 = get_result(denominator, numerator)
    
    #利益一致行动法人的数量变动率
    denominator = old_data['feature_22']['d']
    numerator = new_data['feature_22']['d'] - denominator
    a_6 = get_result(denominator, numerator)
    
    #法定代表人变更次数变动率
    denominator = old_data['feature_6']['c_1']
    numerator = new_data['feature_6']['c_1'] - denominator
    b_1 = get_result(denominator, numerator)
    
    #股东变更次数变动率
    denominator = old_data['feature_6']['c_2']
    numerator = new_data['feature_6']['c_2'] - denominator
    b_2 = get_result(denominator, numerator)
    
    #注册资本变更次数变动率
    denominator = old_data['feature_6']['c_3']
    numerator = new_data['feature_6']['c_3'] - denominator
    b_3 = get_result(denominator, numerator)
    
    #大专及大专以下或不限专业招聘人数变动率
    denominator = old_data['feature_7']['e_1']
    numerator = new_data['feature_7']['e_1'] - denominator
    c_1 = get_result(denominator, numerator)
    
    #3度及3度以下核心自然人（前3）控制节点总数变动率
    denominator = old_data['feature_1']['r_4']
    numerator = new_data['feature_1']['r_4'] - denominator
    d_2 = get_result(denominator, numerator)
    
    
    return dict(
        feature_26={
            'a_1': a_1,
            'a_4': a_4,
            'a_5': a_5,
            'a_6': a_6,
            'b_1': b_1,
            'b_2': b_2,
            'b_3': b_3,
            'c_1': c_1,
            'd_2': d_2
        }
    )
    
def spark_data_flow(tid_old_version, tid_new_version):
    '''
    dataflow
    '''
    #输入
    #算法设计直接用rdd
    tid_old_df = spark.read.json(
        "/user/antifraud/hongjing2/dataflow/step_one/prd/common_static_feature_distribution_v2/{version}".format(version=tid_old_version))
    feature_old_rdd = tid_old_df.rdd.map(lambda r: (r.bbd_qyxx_id, r))
    tid_new_df = spark.read.json(
        "/user/antifraud/hongjing2/dataflow/step_one/prd/common_static_feature_distribution_v2/{version}".format(version=tid_new_version))
    feature_new_rdd = tid_new_df.rdd.map(lambda r: (r.bbd_qyxx_id, r))
        
    #最终计算流程
    dynamic_risk_data = feature_old_rdd.join(
        feature_new_rdd
    ).mapValues(
        get_dynamic_risk
    ).map(
        lambda (k, v): dict({'company_name': k}, **v)
    ).map(
        lambda data: json.dumps(data, ensure_ascii=False)
    )
    
    return dynamic_risk_data


    
    
if __name__ == '__main__':  
    #输出参数
    VERSION = "20170519_1"
    OUTPATH = "/data5/antifraud/Jinli/data/prddata"
    
    prd_rdd = spark_data_flow(
        tid_old_version='20170117',
        tid_new_version='20170117'
    ).cache()

    print prd_rdd.take(10)

In [2]:
'''
测试mysql存储有向属性图的性能
1、将某家公司的数据提取出来
2、生成属性图，序列化，写入mysql
3、读取属性图，反序列化，计算特征
'''

# 1
from operator import itemgetter
tidversion='20170410'
    
    
tid_df = spark.read.parquet("/user/antifraud/jinli/tiddata/relation_company_info_merge/{version}".format(version=tidversion))
tid_rdd = tid_df.rdd
    
#最终计算流程
tid_rdd_2 = tid_rdd.filter(lambda row: row.a == u'be29fc2b652d4eb7b1fdcfecfa6a9edf').map(lambda row: (row.a_name, row)).groupByKey() 
data = tid_rdd_2.collect()



# 2

import pickle

IP = '10.10.20.180'
USER =  'airflow'
PASSWORD = 'airflow'
DB_NAME = 'airflow'

def insert_table(table):
    '''
    连接mysql，执行一个SQL
    '''
    db = MySQLdb.connect(host=IP, user=USER, passwd=PASSWORD, db=DB_NAME)
    # 使用cursor()方法获取操作游标 
    cursor = db.cursor()
    # 使用execute方法执行SQL语句
    sql = """INSERT INTO {table}(bbd_qyxx_id,
         graph_obj)
         VALUES ('{id}', "{obj}" )""".format(table=table, id='be29fc2b652d4eb7b1fdcfecfa6a9edf', obj=pickle.dumps(dig).encode('string-escape'))
    try:
        # 执行SQL语句
        cursor.execute(sql)
        # 提交到数据库执行
        db.commit()
        print "插入{0}成功".format(table)
    except Exception as e:
        # 发生错误时回滚
        db.rollback()
        print e
    # 关闭数据库连接
    db.close()
    
    
insert_table('my_test_graph_object')


# 3

import pickle
import MySQLdb
import time 
import cProfile

IP = '10.10.20.180'
USER =  'airflow'
PASSWORD = 'airflow'
DB_NAME = 'airflow'

def select_table(table):
    '''
    连接mysql，执行一个SQL
    '''
    db = MySQLdb.connect(host=IP, user=USER, passwd=PASSWORD, db=DB_NAME)
    # 使用cursor()方法获取操作游标 
    cursor = db.cursor()
    # 使用execute方法执行SQL语句
    sql = """SELECT * FROM {table} LIMIT 1""".format(table=table)
    try:
        # 执行SQL语句
        cursor.execute(sql)
        # 查询数据
        pickled_obj = cursor.fetchall()
        
    except Exception as e:
        print e
    # 关闭数据库连接
    db.close()
    
    return pickled_obj



d = select_table('my_test_graph_object')
dig = pickle.loads(d[0][1])
src_time = time.time()
#feature_distribution = FeatureConstruction.get_some_feature(data[0][1], [_ for _ in range(1, 25)])
#cProfile.run("FeatureConstruction.get_some_feature(data[0][1], [_ for _ in range(1, 25)])")
des_time = time.time()

print des_time - src_time

feature_distribution

In [ ]:
'''
单个公司DEBUG
'''
tid_df = spark.read.parquet("/user/antifraud/jinli/tiddata/relation_company_info_merge")
tid_rdd = tid_df.where(tid_df.a_name == u'上海诺戎电子科技有限公司').rdd

#最终计算流程
tid_rdd_2 = tid_rdd.map(lambda row: (row.a, row)) \
    .groupByKey(400).filter(lambda r: len(r[1].data) <= 150000) \
    .cache()

#输出
VERSION = "20170301"
OUTPATH = "/data5/antifraud/Jinli/data/prddata"

tid_rdd_2.mapValues(
        FeatureConstruction.get_all_feature).map(itemgetter(1)).collect()

In [8]:
'''
BUG修复测试
'''
from pyspark.sql import functions as fun

tmp_df = spark.read.parquet("/user/antifraud/jinli/tiddata/relation_company_info_merge")
tid_1 = tmp_df.where(tmp_df.a == '5ef468297555411aba041a16c4fad4c5') \
.where(tmp_df.b_degree == 1).select(
    'b',
    tmp_df.b_estatus.alias('estatus')
)
tid_2 = tmp_df.where(tmp_df.a == '5ef468297555411aba041a16c4fad4c5') \
.where(tmp_df.c_degree == 1).select(
    'c',
    tmp_df.c_estatus.alias('estatus')
)




tid_2.union(tid_1).distinct().select(fun.sum('estatus')).show(100)

+------------+
|sum(estatus)|
+------------+
|           0|
+------------+



In [1]:
'''
小工具：用来看数据倾斜的，输出: [(企业ID，关联方数)]
'''

tid_df = spark.read.parquet("/user/antifraud/jinli/tiddata/relation_company_info_merge/20170403")
tid_rdd = tid_df.rdd
tid_rdd_2 = tid_rdd.map(lambda row: (row.a, row)).groupByKey(800).cache()
realation_strip = tid_rdd_2.mapValues(lambda x: len(x.data)).collect()

import operator
sorted(realation_strip, key=operator.itemgetter(1), reverse=True)[:10]

[(u'ce5277ab116a4ab89bdfab7bdfb7a63a', 32512),
 (u'be29fc2b652d4eb7b1fdcfecfa6a9edf', 25229),
 (u'79941af7fec84db1a06169da06bba181', 25229),
 (u'47c3144831574dceb0268b86974e371e', 24271),
 (u'964c8879b5bd48aca460530b5ef3b664', 23514),
 (u'68129f3e9c4f4d319160b8502d90348d', 23242),
 (u'acaebbfefd364f258b84c8e0457b8b8f', 22848),
 (u'3744963e8ca04715ac30a604799e6c68', 22242),
 (u'97d29bb53fa64a8c8e159a487d9c887b', 21333),
 (u'd5c13f432308484098ce1022b0bcfeed', 20937)]

In [6]:
df = spark.read.csv("/user/antifraud/jinli/rawdata/tar_company/raw_jl_company_list_20170514_1.data")
df.distinct().count()

1683

In [4]:
%%writefile /data5/antifraud/source/tmp_20170413/hongjing_jinli_static_kpi.py
# -*- coding: utf-8 -*-
import datetime
import requests
import json
import time
import re
import os
import sys
from operator import itemgetter
from collections import Counter, defaultdict, OrderedDict

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as fun
from pyspark.sql import types as tp
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pandas.io.json import json_normalize
import networkx as nx
import numpy as np



class FeatureConstruction(object):
    '''
    计算特征的函数集
    '''      

    def __fault_tolerant(func):
        '''
        一个用于容错的装饰器
        '''
        @classmethod
        def wappen(cls, *args, **kwargs):
            try:
                return func(cls, *args, **kwargs)
            except Exception, e:
                return (
                    "{func_name} has a errr : {excp}"
                ).format(func_name=func.__name__, excp=e)
        return wappen    
    
    @__fault_tolerant    
    def create_graph_udf(cls, relations, is_directed):
        '''
        根据关联方的结构创建有向、无向图
        '''
        def init_graph(edge_list, node_list, is_directed=0):
            #网络初始化
            G = nx.DiGraph() if is_directed == 1 else nx.Graph()    
            #增加带属性的节点
            for node in node_list:
                G.add_node(node[0], attr_dict=node[1])
            #增加带属性的边
            G.add_edges_from(edge_list)
            return G
            
        #生成一个图
        company_correlative_edges = [
            (row.b, row.c, {'is_invest': row.bc_relation}) for row in relations]
                                     
        company_correlative_nodes = [(
                row.b, 
                dict(
                    is_human=row.b_isperson,
                    is_black=row.b_is_black_company,
                    distance = row.b_degree,
                    name = row.b_name,
                    isSOcompany = row.b_isSOcompany,
                    esdate = row.b_regtime,
                    ktgg = row.b_ktgg,
                    zgcpwsw = row.b_zgcpwsw,
                    rmfygg = row.b_rmfygg,
                    lending = row.b_lending,
                    xzcf = row.b_xzcf,
                    zhixing = row.b_zhixing,
                    dishonesty = row.b_dishonesty,
                    jyyc = row.b_jyyc,
                    circxzcf = row.b_circxzcf,
                    opescope = row.b_opescope,
                    address = row.b_address,
                    estatus = row.b_estatus,
                    province = row.b_province
                ))  for row in relations] + [(
                row.c, 
                dict(
                    is_human=row.c_isperson,
                    is_black=row.c_is_black_company,
                    distance = row.c_degree,
                    name = row.c_name,
                    isSOcompany = row.c_isSOcompany,
                    esdate = row.c_regtime,
                    ktgg = row.c_ktgg,
                    zgcpwsw = row.c_zgcpwsw,
                    rmfygg = row.c_rmfygg,
                    lending = row.c_lending,
                    xzcf = row.c_xzcf,
                    zhixing = row.c_zhixing,
                    dishonesty = row.c_dishonesty,
                    jyyc = row.c_jyyc,
                    circxzcf = row.c_circxzcf,
                    opescope = row.c_opescope,
                    address = row.c_address,
                    estatus = row.c_estatus,
                    province = row.c_province
                )) for row in relations]
        
        if is_directed == 1:
            g = init_graph(
                company_correlative_edges,
                company_correlative_nodes, is_directed = 1) 
        else:
            g = init_graph(
                company_correlative_edges, 
                company_correlative_nodes, is_directed = 0)
        return g

    @__fault_tolerant
    def get_feature_1(cls):
        '''
        企业背景风险
        '''
        one_relation_set = [
            node for node, attr in cls.DIG.nodes_iter(data=True) if attr['distance'] == 1]
        shareholder = [
            src_node for src_node, des_node, edge_attr 
            in cls.DIG.edges_iter(one_relation_set, data=True) 
            if des_node == cls.tarcompany 
            and edge_attr['is_invest'] == 'INVEST']
        os_shareholder = [
            node for node in shareholder 
            if cls.DIG.node[node]['isSOcompany'] is True 
            and cls.DIG.node[node]['is_human'] == 0]
        anti_os_shareholder = [
            node for node in shareholder 
            if cls.DIG.node[node]['isSOcompany'] is False 
            and cls.DIG.node[node]['is_human'] == 0]
        nature_shareholder = [
            node for node in shareholder 
            if cls.DIG.node[node]['is_human'] == 1]
        is_ipo_tarcompany = 1 if cls.resultiterable.data[0].a_isIPOcompany is True else 0
    
        x = len(os_shareholder)
        y = len(anti_os_shareholder)
        z = len(nature_shareholder)
        w = len(shareholder)
        r_i = is_ipo_tarcompany
        r = (0.2*x + 0.5*y + z) * 100. * (2 - r_i) / (2*w + 0.001) 
        
        return dict(
            x=x,
            y=y,
            z=z,
            w=w,
            r_i=r_i,
            r=round(r, 2) if r else 100
        )


    @__fault_tolerant
    def get_feature_2(cls):
        '''
        资本风险
        '''
        x = cls.resultiterable.data[0].a_regcap 
        y = cls.resultiterable.data[0].a_realcap
        
        x = x / 10000 if x != 'NULL' and x is not None else 0
        y = y / 10000 if y != 'NULL' and y is not None else 0
        
        if 0 <= x < 500:
            c_i = 100
        elif 500 <= x < 1000:
            c_i = 60
        elif 1000 <= x < 5000:
            c_i = 30
        elif 5000 <= x:
            c_i = 10
        
        p_i = 1 if y / (x + 0.001) >= 0.5 else 0
        
        return  dict(
            x=x,
            y=y,
            c=c_i * (1 - p_i/2.)
        )
    
    @__fault_tolerant
    def get_feature_3(cls):
        '''
        公司地域风险
        '''
        province_black_num = cls.resultiterable.data[0].a_province_black_num
        province_leijinrong_num = cls.resultiterable.data[0].a_province_leijinrong_num
        province_black_num = province_black_num if province_black_num else 0
        province_leijinrong_num = province_leijinrong_num if province_leijinrong_num else 0
        
        return dict(
            j_1=province_black_num,
            j_2=province_leijinrong_num,
            j=round(province_black_num / (province_leijinrong_num + 0.0001), 4)
        )
    
    @__fault_tolerant
    def get_feature_4(cls):
        '''
        知识产权风险（专利和商标）
        '''
        k_1 = cls.resultiterable.data[0].a_zhuanli
        k_2 = cls.resultiterable.data[0].a_shangbiao 
        k_1 = k_1 if k_1 != 'NULL' and k_1 is not None else 0
        k_2 = k_2 if k_1 != 'NULL' and k_2 is not None else 0
        k_3 = k_1 + k_2
        
        if k_3 == 0:
            k = 100
        elif 1 <= k_3 < 5:
            k = 60
        elif 5 <= k_3 < 10:
            k = 30
        elif 10 <= k_3:
            k = 10
        
        return dict(
            k_1=k_1,
            k_2=k_2,
            k=k
        )
        
    @__fault_tolerant
    def get_feature_5(cls):
        '''
        域名备案风险
        '''
        icp = cls.resultiterable.data[0].a_ICP
        url = cls.resultiterable.data[0].a_url
        
        c_i = 1 if icp != 'NULL' and icp is not None else 0   
        try:
            if url is not None and url != 'NULL':
                status_code = requests.get(url, allow_redirects=False).status_code 
                p_i = 1 if status_code == 200 else 0
            else:
                p_i = 0
        except:
            p_i = 0
        
        l = (1 - c_i/2.) * (1 - p_i/2.) * 100
        
        return dict(
            c_i=c_i,
            p_i=p_i,
            l=l
        )
        
    @__fault_tolerant
    def get_feature_6(cls):
        '''
        工商变更风险
        '''
        def get_bgxx_symbol(bgxx_name):
            if u'法定代表人' in bgxx_name:
                return 'c_1'
            elif u'股东' in bgxx_name:
                return 'c_2'
            elif u'注册资本' in bgxx_name:
                return 'c_3'
            elif u'高管' in bgxx_name:
                return 'c_4'
            elif u'经营范围' in bgxx_name:
                return 'c_5'
            else:
                return 'c_6'
        
        default_result = OrderedDict([
                ('c_1', 0),
                ('c_2', 0),
                ('c_3', 0),
                ('c_4', 0),
                ('c_5', 0),
                ('c_6', 0),
                ('r', 0),
                ('z', 10)
            ]
        )
        
        if cls.resultiterable.data[0].a_bgxx is not None:
            for bgxx_name, bgxx_num in cls.resultiterable.data[0].a_bgxx.iteritems():
                default_result[get_bgxx_symbol(bgxx_name)] += int(bgxx_num)
            default_result['r'] = np.dot(default_result.values(), [2, 1, 1, 1, 2, 0, 0, 0])
        
            if default_result['r'] == 0:
                default_result['z'] = 10
            elif 1 <= default_result['r'] < 5:
                default_result['z'] = 30
            elif 5 <= default_result['r'] < 10:
                default_result['z'] = 60
            elif 10 <= default_result['r']:
                default_result['z'] = 100
                
        default_result.pop('c_6')
        
        return dict(default_result)
        
    @__fault_tolerant
    def get_feature_7(cls):
        '''
        人才结构风险
        '''        
        def get_recruit_num(each_recruit):
            if u'本科' in each_recruit:
                return 'e_2'
            elif u'硕士' in each_recruit:
                return 'e_3'
            elif u'博士' in each_recruit:
                return 'e_3'
            else:
                return 'e_1'
            
        default_result = OrderedDict([
                ('e_1', 0),
                ('e_2', 0),
                ('e_3', 0),
                ('e', 0),
                ('r', 0),
                ('z', 10)
            ]
        )
        
        if cls.resultiterable.data[0].a_recruit is not None:
            for education_name, education_num in cls.resultiterable.data[0].a_recruit.iteritems():
                default_result[get_recruit_num(education_name)] += int(education_num)
            default_result['e'] = sum(
                map(int ,cls.resultiterable.data[0].a_recruit.values()))
            default_result['r'] = round(
                np.dot(default_result.values(), [100., 60., 30., 0, 0, 0]) / default_result['e'], 2)
            
            if default_result['r'] == 0:
                default_result['z'] = 10
            elif 0 < default_result['r'] < 60:
                default_result['z'] = 30
            elif 60 <= default_result['r'] < 80:
                default_result['z'] = 60
            elif 80 <= default_result['r']:
                default_result['z'] = 100         
            
        return dict(default_result)
        
    @__fault_tolerant
    def get_feature_8(cls):
        '''
        公司运营持续风险
        '''
        esdate_relation_set = [
            (datetime.date.today() - attr['esdate']).days 
            for node, attr in cls.DIG.nodes_iter(data=True) 
            if attr['distance'] <= 3 
            and attr['is_human'] == 0 
            and attr['esdate'] is not None]
        t = round(np.average(esdate_relation_set), 2)
        
        if 0 <= t < 365:
            y = 100
        elif 365 <= t < 1825:
            y = 60
        elif 1825 <= t < 3650:
            y = 30
        elif 3650 <= t:
            y = 10
        else:
            y = 100
        return dict(
            t=t if t else 0,
            y=y
        )

        
    @__fault_tolerant
    def get_feature_9(cls):
        '''
        招标和中标风险
        '''
        n_1 = cls.resultiterable.data[0].a_zhaobiao
        n_2 = cls.resultiterable.data[0].a_zhongbiao
        n_1 = n_1 if n_1 is not None else 0
        n_2 = n_2 if n_2 is not None else 0
        
        n = n_1 + n_2
        
        if n == 0:
            n = 100
        elif 1 <= n < 5:
            n = 60
        elif 5<= n < 10:
            n = 30
        else:
            n = 10
        
        return  dict(
            n_1=n_1,
            n_2=n_2,
            n=n
        )
        
    @__fault_tolerant
    def get_feature_xgxx(cls):
        '''
        【计算基础数据】
        法律诉讼风险：开庭公告、裁判文书、法院公告、民间借贷
        行政处罚
        被执行风险
        异常经营风险：经营异常、吊销&注销
        银监会行政处罚
        ''' 
        def get_certain_distance_all_info(distance, document_types):
            all_array = []
            #处理某一个distance不存在节点的情况
            all_array.append([0]*len(document_types))
            for node, attr in cls.DIG.nodes_iter(data=True):
                if attr['is_human'] == 0  and attr['distance'] == distance:
                    each_array = map(lambda x: x if x else 0, 
                                                  [attr[each_document] 
                                                      for each_document in document_types])
                    all_array.append(each_array)
                else:
                    continue
            documents_num = np.sum(all_array, axis=0)
            return documents_num
        
        matrx = dict()
        xgxx_type = ['ktgg', 'zgcpwsw', 'rmfygg', 
                             'lending', 'xzcf', 'zhixing', 
                             'dishonesty', 'jyyc', 'circxzcf', 
                             'estatus']

        for each_distance in xrange(0, 4):
            xgxx_num_list = get_certain_distance_all_info(each_distance, 
                                                                                      xgxx_type)
            matrx[each_distance] = dict(zip(xgxx_type, xgxx_num_list))
            
        return matrx
    
    @__fault_tolerant
    def filter_xgxx_type(cls, tar_xgxx):
        '''
        过滤相关信息
        '''
        risk = dict()
        for each_distance, xgxx_statistics in cls.xgxx_distribution.iteritems():
            tar_xgxx_statistics = dict([
                    (each_xgxx_type, each_xgxx_num)
                    for each_xgxx_type, each_xgxx_num 
                    in xgxx_statistics.iteritems() 
                    if each_xgxx_type in tar_xgxx])
            risk[each_distance] = tar_xgxx_statistics
        return risk
    
    @__fault_tolerant
    def get_feature_10(cls):
        '''
        法律诉讼风险：开庭公告、裁判文书、法院公告、民间借贷
        '''
        tar_xgxx = ['ktgg', 'zgcpwsw', 'rmfygg', 'lending']
        risk = cls.filter_xgxx_type(tar_xgxx)
        
        risk['r'] = round(
            np.dot(
                map(sum, [
                        risk[each_distance].values() 
                        for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)
        
        if risk['r'] == 0:
            risk['z'] = 10
        elif 0 < risk['r'] <= 10:
            risk['z'] = 30
        elif 10 < risk['r'] <= 30:
            risk['z'] = 60
        elif 30 < risk['r']:
            risk['z'] = 100
        
        return risk
            
    @__fault_tolerant
    def get_feature_11(cls):
        '''
        行政处罚风险
        '''
        tar_xgxx = ['xzcf']
        risk = cls.filter_xgxx_type(tar_xgxx)
        
        risk['r'] = round(
            np.dot(
                map(sum, [risk[each_distance].values() 
                        for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)
        
        if risk['r'] == 0:
            risk['z'] = 10
        elif 0 < risk['r'] <= 10:
            risk['z'] = 30
        elif 10 < risk['r'] <= 30:
            risk['z'] = 60
        elif 30 < risk['r']:
            risk['z'] = 100
        
        return risk
    
    @__fault_tolerant
    def get_feature_12(cls):
        '''
        被执行风险：被执行、失信被执行
        '''
        tar_xgxx = ['zhixing', 'dishonesty']
        risk = cls.filter_xgxx_type(tar_xgxx)        
        
        risk['r'] = round(
            np.dot(
                map(
                    lambda x: x['zhixing'] + 2. * x['dishonesty'], 
                    [risk[each_distance] for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)
        
        if risk['r'] == 0:
            risk['z'] = 10
        elif 0 < risk['r'] <= 10:
            risk['z'] = 30
        elif 10 < risk['r'] <= 30:
            risk['z'] = 60
        elif 30 < risk['r']:
            risk['z'] = 100        
        
        return risk
        
    @__fault_tolerant
    def get_feature_13(cls):
        '''
        异常经营风险
        '''
        tar_xgxx = ['jyyc', 'estatus']
        risk = cls.filter_xgxx_type(tar_xgxx)        
        
        risk['r'] = round(
            np.dot(
                map(
                    lambda x: x['jyyc'] + 2. * x['estatus'], 
                    [risk[each_distance] for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)

        if risk['r'] == 0:
            risk['z'] = 10
        elif 0 < risk['r'] <= 10:
            risk['z'] = 30
        elif 10 < risk['r'] <= 30:
            risk['z'] = 60
        elif 30 < risk['r']:
            risk['z'] = 100
        
        return risk        
        
    @__fault_tolerant
    def get_feature_14(cls):
        '''
        银监会行政处罚
        '''
        tar_xgxx = ['circxzcf']
        risk = cls.filter_xgxx_type(tar_xgxx)        
        
        risk['r'] = round(
            np.dot(
                map(
                    lambda x: x['circxzcf'], 
                    [risk[each_distance] for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)

        if risk['r'] == 0:
            risk['z'] = 10
        elif 0 < risk['r'] <= 10:
            risk['z'] = 30
        elif 10 < risk['r'] <= 30:
            risk['z'] = 60
        elif 30 < risk['r']:
            risk['z'] = 100        
    
        return risk            
        
    @__fault_tolerant
    def get_feature_15(cls):
        '''
        实际控制人风险
        '''
        def get_degree_distribution(is_human):
            some_person_sets = [
                node
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if attr['is_human'] == is_human
                and attr['distance'] <= 3]
            person_out_degree = cls.DIG.out_degree(some_person_sets).values()
            if not person_out_degree:
                person_out_degree.append(0)
            return person_out_degree
        
        nature_person_distribution = get_degree_distribution(1)
        legal_person_distribution = get_degree_distribution(0)
        
        nature_max_control = max(nature_person_distribution)
        legal_max_control = max(legal_person_distribution)        
        nature_avg_control = round(np.average(nature_person_distribution), 2)
        legal_avg_control = round(np.average(legal_person_distribution), 2)
        
        total_legal_num = len([
                node 
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if attr['is_human'] == 0])
        
        risk = round(((
                    2*(nature_max_control + legal_max_control) + 
                    (nature_avg_control + legal_avg_control)) /
                (2*total_legal_num + 0.001)), 2)
        
        if 0 <= risk <= 0.5:
            r = 10
        elif 0.5 < risk <= 1:
            r = 30
        elif 1 < risk <= 3:
            r = 60  
        elif 3 < risk:
            r = 100
        else:
            r = 10
        
        return dict(
            x_1=nature_max_control,
            x_2=legal_max_control,
            y_1=nature_avg_control,
            y_2=legal_avg_control,
            z=total_legal_num,
            s=risk,
            r=r
        )
    
    @__fault_tolerant
    def get_feature_16(cls):
        '''
        公司扩张路径风险
        '''
        def get_node_set(is_human):
            return Counter([
                    attr['distance']
                    for node, attr in cls.DIG.nodes_iter(data=True)
                    if attr['is_human'] == is_human])
        
        nature_person_distribution = get_node_set(1)
        legal_person_distribution = get_node_set(0)
        
        nature_person_num = [
            nature_person_distribution.get(each_distance, 0)
            for each_distance in range(1, 4)]
        legal_person_num = [
            legal_person_distribution.get(each_distance, 0)
            for each_distance in range(1, 4)]
        
        risk = round(np.sum(
                np.divide(
                    [
                        np.divide(
                            np.sum(nature_person_num[:each_distance]), 
                            np.sum(legal_person_num[:each_distance]), 
                            dtype=float)
                        for each_distance in range(1, 4)], 
                    np.array([1, 2, 3], dtype=float))), 2)
        
        if 0 <= risk <= 1:
            r = 10
        elif 1 < risk <= 3:
            r = 30
        elif 3 < risk <= 5:
            r = 60  
        elif 5 < risk:
            r = 100
        else:
            r = 10
        
        return dict(
            x_1=legal_person_num[0],
            x_2=legal_person_num[1],
            x_3=legal_person_num[2],
            y_1=nature_person_num[0],
            y_2=nature_person_num[1],
            y_3=nature_person_num[2],
            z=risk if risk < 10000 else 0,
            r=r
        )


    @__fault_tolerant
    def get_feature_17(cls):
        '''
        关联方中心集聚风险
        '''
        one_relation_set = [
            node for node, attr in cls.DIG.nodes_iter(data=True) 
            if attr['distance'] == 1]
        legal_person_shareholder = len([
                src_node 
                for src_node, des_node, edge_attr 
                in cls.DIG.edges_iter(one_relation_set, data=True) 
                if des_node == cls.tarcompany 
                and edge_attr['is_invest'] == 'INVEST'
                and cls.DIG.node[src_node]['is_human'] == 0])
        legal_person_subsidiary = len([
                des_node 
                for src_node, des_node, edge_attr 
                in cls.DIG.edges_iter(one_relation_set, data=True) 
                if src_node == cls.tarcompany 
                and edge_attr['is_invest'] == 'INVEST'
                and cls.DIG.node[des_node]['is_human'] == 0])
        
        risk =(
            legal_person_subsidiary -
            legal_person_shareholder ) 
        
        if risk < 0:
            r = 10
        elif risk == 0:
            r = 30
        elif 0 < risk <= 3:
            r = 60
        elif 3 < risk:
            r = 100
        else:
            r = 100        
        
        return dict(
            x=legal_person_subsidiary,
            y=legal_person_shareholder,
            z=risk,
            r=r
        )

    @__fault_tolerant
    def get_feature_18(cls):
        '''
        分支机构过度扩张风险
        '''
        risk = cls.resultiterable.data[0].a_fzjg
        risk = risk if risk is not None else 0

        if  risk == 0:
            r = 10
        elif risk == 1:
            r = 30
        elif 1 < risk <= 3:
            r = 60  
        elif 3 < risk:
            r = 100
        else:
            r = 10
        
        return dict(
            d=risk if risk < 100 else 100, 
            z=r
        )
    
    @__fault_tolerant
    def get_feature_19(cls):
        '''
        关联方结构稳定风险
        '''
        def get_relation_num():
            return Counter([
                    attr['distance']
                    for node, attr in cls.DIG.nodes(data=True)])
        
        #目标企业各个度的节点的数量
        relations_num = get_relation_num()
        relation_three_num = relations_num.get(3, 0)
        relation_two_num = relations_num.get(2, 0)
        relation_one_num = relations_num.get(1, 0)
        relation_zero_num = relations_num.get(0, 1)
        
        x = np.array([
                relation_zero_num, 
                relation_one_num, 
                relation_two_num, 
                relation_three_num]).astype(float)
        
        y_2 = x[2] / (x[1]+x[2])
        y_3 = x[3] / (x[1]+x[2]+x[3])
        risk = y_2/2 + y_3/3
        

        
        if 0 <= risk <= 0.1:
            r = 10
        elif 0.1 < risk <= 0.3:
            r = 30
        elif 0.3 < risk <= 0.5:
            r = 60  
        elif 0.5 < risk:
            r = 100      
        else:
            r = 10
        
        return dict(
            x_1=x[1],
            x_2=x[2],
            x_3=x[3],
            y_2=y_2,
            y_3=y_3,
            z=risk if risk else 0,
            r=r
        )        
    
    @__fault_tolerant
    def get_feature_20(cls):
        '''
        潜在违规融资风险
        '''

        
        def get_relation_risk_num(keyword_type):
            return Counter([attr['distance']
                    for node, attr in cls.DIG.nodes_iter(data=True) 
                    if attr['is_human'] == 0 and 
                    attr['opescope'] == keyword_type])
        
        k_2_num = get_relation_risk_num('k_2')
        k_1_num = get_relation_risk_num('k_1')
        
        x = np.array([
                k_2_num.get(each_distance, 0) 
                for each_distance in xrange(1, 4)])
        y = np.array([
                k_1_num.get(each_distance, 0) 
                for each_distance in xrange(1, 4)])
    
        risk = round(
            np.sum(
                np.divide(np.add(x, y), [1, 2, 3], dtype=float)), 2)
    
        if  risk == 0:
            r = 10
        elif 0 < risk < 5:
            r = 30
        elif 5 <= risk < 10:
            r = 60  
        elif 10 <= risk:
            r = 100     
        else:
            r = 10
    
        return dict(
            x_1=x[0],
            x_2=x[1],
            x_3=x[2],
            y_1=y[0],
            y_2=y[1],
            y_3=y[2],
            g=r
        )
    
    @__fault_tolerant
    def get_feature_21(cls):
        '''
        关联方地址集中度风险
        '''
        legal_person_address = [
            attr['address'] 
            for node, attr in cls.DIG.nodes_iter(data=True) 
            if attr['distance'] <= 3 
            and attr['is_human'] == 0]
        c = Counter(
            filter(
                lambda x: x is not None and len(x) >= 21,  
                legal_person_address))
        n = c.most_common(1)
        n= n[0][1] if len(n) > 0 else 1
        risk = n -1
        
        if  risk == 0:
            z = 10
        elif risk == 1:
            z = 30
        elif risk == 2:
            z = 60  
        elif 2 < risk:
            z = 100             
        
        return dict(
            n=risk,
            y=risk,
            z=z
        )
    
    @__fault_tolerant
    def get_feature_22(cls):
        '''
        利益一致行动法人风险
        '''
        def is_similarity(str_1, str_2):
            '''
            判断两个字符串是否有连续2个字相同
            '''
            try:
                token_1 = [
                    str_1[index] + str_1[index+1] 
                    for index,data in enumerate(str_1) 
                    if index < len(str_1) - 1]
                is_similarity = sum([
                        1 for each_token in token_1 
                        if each_token in str_2])
                return True if is_similarity > 0 else False        
            except:
                return False
            
        #目标公司的名称frag
        tar_company = cls.resultiterable.data[0].a_name
        tar_company_frag = cls.resultiterable.data[0].a_namefrag
        #三度以内，所有关联方的节点集合(不包含自身)
        relation_set = [
                attr['name'] 
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if attr['distance'] <= 3]
        #relation_set.remove(tar_company)
        common_interests_list = [
            1 
            for node_name in relation_set 
            if is_similarity(tar_company_frag, node_name)]
        
        risk = sum(common_interests_list)
        if risk == 0:
            z = 10
        elif risk == 1:
            z = 30
        elif 1 < risk <= 3:
            z = 60
        elif 3 < risk:
            z = 100
        else:
            z = 10
        
        return dict(
            d=risk, 
            z=z
        )        
        
    @__fault_tolerant
    def get_feature_23(cls):
        '''
        关联方信誉风险
        '''
        def get_black_num(distance):
            '''
            某一度黑企业的总数
            '''
            return sum([
                    1
                    for node, attr in cls.DIG.nodes_iter(data=True)
                    if (attr['is_black'] and attr['distance'] == distance 
                            and attr['is_human'] == 0)
            ])
        
        black_relation_set =[
            get_black_num(each_distance) 
            for each_distance in range(1, 4)]
        risk = np.dot(black_relation_set, [1, 1/2., 1/3.])
        if risk == 0:
            z = 10
        elif 0 < risk < 0.5:
            z = 30
        elif 0.5 <= risk < 1:
            z = 60
        elif 1 <= risk:
            z =100
        else:
            z = 10
        
        return dict(
            b_1=black_relation_set[0],
            b_2=black_relation_set[1],
            b_3=black_relation_set[2],
            y=risk,
            z=z
        )
    
    @__fault_tolerant
    def get_feature_24(cls):
        '''
        短期逐利风险
        '''
        
        def get_max_established(distance, timedelta):
            '''
            获取在某distance关联方企业中，某企业在任意timedelta内投资成立公司数量的最大值
            '''
            #用于获取最大连续时间数，这里i的取值要仔细琢磨一下，这里输入一个时间差序列与时间差
            def get_date_density(difference_list, timedelta):
                time_density_list = []
                for index, date in enumerate(difference_list):
                    if date < 15:
                        s = 0
                        i = 1
                        while(s < 15 and i <= len(difference_list) - index):
                            i += 1
                            s = sum(difference_list[index:index+i])
                        time_density_list.append(i)
                    else:
                        continue
                return max(time_density_list) if len(time_density_list) >0 else 0            

            #distance所有节点集合
            relation_set = [
                node 
                for node, attr in cls.DIG.nodes(data=True) 
                if attr['distance'] == distance]
            investment_dict = defaultdict(lambda : [])     
            
            if len(cls.DIG.edge) > 1:
                for src_node, des_node, edge_attr in (
                        cls.DIG.edges_iter(relation_set, data=True)):
                    if (edge_attr.get('is_invest', 0) == 'INVEST' 
                            and cls.DIG.node[des_node]['distance'] == distance
                            and cls.DIG.node[des_node]['esdate'] is not None 
                            and cls.DIG.node[src_node]['is_human'] == 0):
                        #将所有节点投资的企业的成立时间加进列表中
                        investment_dict[src_node].append(cls.DIG.node[des_node]['esdate'])
             
            #目标企业所有节点所投资的企业时间密度字典
            all_date_density_dict = {}
            
            for node, date_list in investment_dict.iteritems():
                #构建按照时间先后排序的序列
                date_strp_list = sorted(date_list)
                if len(date_strp_list) > 1:
                    #构建时间差的序列，例如：[256, 4, 5, 1, 2, 33, 6, 5, 4, 73]
                    date_difference_list = [
                        (date_strp_list[index + 1] - date_strp_list[index]).days 
                        for index in range(len(date_strp_list) -1)]
                    #计算某法人节点在timedelta天之内有多少家公司成立
                    es_num = get_date_density(date_difference_list, timedelta)
                    if all_date_density_dict.has_key(es_num):
                        all_date_density_dict[es_num].append(node)
                    else:
                        all_date_density_dict[es_num] = [node]              
                else:
                    continue
            keys = all_date_density_dict.keys()        
            max_num = max(keys) if len(keys) > 0 else 0
            
            return max_num
        
        x = [
            get_max_established(each_distance, 180) 
            for each_distance in xrange(1, 4)]
        
        risk = round(
            np.sum(
                np.divide(x, [1, 2, 3], dtype=float)), 2)
        
        if  risk == 0:
            r = 10
        elif 0 < risk <= 0.7:
            r = 30
        elif 0.7 < risk <= 1:
            r = 60  
        elif 1 < risk:
            r = 100       
        else:
            r = 10
        
        return dict(
            x_1=x[0],
            x_2=x[1],
            x_3=x[2],
            z=r,
            d=risk
        )
    

    @classmethod
    def get_some_feature(cls, resultiterable, feature_nums):
        #创建类变量
        cls.resultiterable = resultiterable
        cls.tarcompany = cls.resultiterable.data[0].a
        cls.DIG = cls.create_graph_udf(cls.resultiterable, 1)
        cls.xgxx_distribution = cls.get_feature_xgxx()
        
        feature_list = [
            ('feature_{0}'.format(feature_index), 
                     eval('cls.get_feature_{0}()'.format(feature_index)))
            for feature_index in feature_nums]
        feature_list.append(('bbd_qyxx_id', cls.tarcompany))
        feature_list.append(('company_name', cls.resultiterable.data[0].a_name))
        
        return json.dumps(dict(feature_list), ensure_ascii=False)      #

    
def spark_data_flow(tidversion):
    '''
    dataflow
    '''
    conf = SparkConf()
    conf.setMaster('yarn-client')
    conf.set("spark.yarn.am.cores", 7)
    conf.set("spark.executor.memory", "50g")
    conf.set("spark.executor.instances", 20)
    conf.set("spark.executor.cores", 10)
    conf.set("spark.python.worker.memory", "2g")
    conf.set("spark.default.parallelism", 1000)
    conf.set("spark.sql.shuffle.partitions", 1000)
    conf.set("spark.broadcast.blockSize", 1024)
    conf.set("spark.executor.extraJavaOptions","-XX:+PrintGCDetails -XX:+PrintGCTimeStamps")    
    conf.set("spark.shuffle.file.buffer", '512k')
    
    spark = SparkSession \
        .builder \
        .appName("glf") \
        .config(conf = conf) \
        .enableHiveSupport() \
        .getOrCreate()  
    
    #输入
    #算法设计直接用rdd
    tid_df = spark.read.parquet("/user/antifraud/jinli/tiddata/relation_company_info_merge/{version}".format(version=tidversion))
    tid_rdd = tid_df.rdd
        
    #最终计算流程
    tid_rdd_2 = tid_rdd.map(lambda row: (row.a_name, row)) \
        .groupByKey() \
        .repartition(1000) \
        .filter(lambda r: len(r[1].data) <= 100000) \
        .cache()

    import time
    import signal
    def handler(signum, frame):
        raise AssertionError

    def fault_tolerant(func):
        def wappen(*args):
            try:
                return func(*args)
            except:
                return 'acer'
        return wappen
            
    @fault_tolerant
    def time_out(data):
        signal.signal(signal.SIGALRM, handler)
        signal.alarm(30)
        return FeatureConstruction.get_some_feature(data, [_ for _ in range(1, 25)])

    def calculate(data):
        return FeatureConstruction.get_some_feature(data, [_ for _ in range(1, 25)])

    feature_list = tid_rdd_2.mapValues(
        time_out
    ).map(
        itemgetter(1)
    )   
    
    return feature_list

def collect_to_driver(version, out_path, tidversion):
    '''
    格式化输出
    '''
    src_time = time.time()
    pd_df = spark_data_flow(tidversion)
    os.system("hadoop fs -rmr {path}/{version}/prd_jinli_feature_score_distribution_{version}".format(path=OUTPATH, version=version))
    pd_df.saveAsTextFile("{path}/{version}/prd_jinli_feature_score_distribution_{version}".format(path=OUTPATH, version=version))
    des_time = time.time()    
    
    print 'SUCCESS !! \n 耗时：{0}'.format(des_time - src_time)
    
    
if __name__ == '__main__':  
    #输入参数
    VERSION = "20170417"
    OUTPATH = "/user/antifraud/jinli/tmpdata"    

    collect_to_driver(
        version=VERSION,
        out_path=OUTPATH,
        tidversion='20170403'
    )

    

Overwriting /data5/antifraud/source/tmp_20170413/hongjing_jinli_static_kpi.py


In [3]:
%%writefile /data5/antifraud/source/tmp_20170413/hongjing_jinli_dynamic_kpi.py
# -*- coding: utf-8 -*-
'''
动态风险：需要2个时间节点的中间数据
'''
import datetime
import requests
import json
import time
import re
import os
import sys
from operator import itemgetter
from collections import Counter, defaultdict, OrderedDict

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as fun
from pyspark.sql import types as tp
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pandas.io.json import json_normalize
import networkx as nx
import numpy as np



class FeatureConstruction(object):
    '''
    计算特征的函数集
    '''      

    def __fault_tolerant(func):
        '''
        一个用于容错的装饰器
        '''
        @classmethod
        def wappen(cls, *args, **kwargs):
            try:
                return func(cls, *args, **kwargs)
            except Exception, e:
                return (
                    "{func_name} has a errr : {excp}"
                ).format(func_name=func.__name__, excp=e)
        return wappen    
    
    @__fault_tolerant    
    def create_graph_udf(cls, relations, is_directed):
        '''
        根据关联方的结构创建有向、无向图
        '''
        def init_graph(edge_list, node_list, is_directed=0):
            #网络初始化
            G = nx.DiGraph() if is_directed == 1 else nx.Graph()    
            #增加带属性的节点
            for node in node_list:
                G.add_node(node[0], attr_dict=node[1])
            #增加带属性的边
            G.add_edges_from(edge_list)
            return G
            
        #生成一个图
        company_correlative_edges = [
            (row.b, row.c, {'is_invest': row.bc_relation}) for row in relations]
                                     
        company_correlative_nodes = [(
                row.b, 
                dict(
                    is_human=row.b_isperson,
                    is_black=row.b_is_black_company,
                    distance = row.b_degree,
                    name = row.b_name,
                    isSOcompany = row.b_isSOcompany,
                    esdate = row.b_regtime,
                    ktgg = row.b_ktgg,
                    zgcpwsw = row.b_zgcpwsw,
                    rmfygg = row.b_rmfygg,
                    lending = row.b_lending,
                    xzcf = row.b_xzcf,
                    zhixing = row.b_zhixing,
                    dishonesty = row.b_dishonesty,
                    jyyc = row.b_jyyc,
                    circxzcf = row.b_circxzcf,
                    opescope = row.b_opescope,
                    address = row.b_address,
                    estatus = row.b_estatus,
                    province = row.b_province
                ))  for row in relations] + [(
                row.c, 
                dict(
                    is_human=row.c_isperson,
                    is_black=row.c_is_black_company,
                    distance = row.c_degree,
                    name = row.c_name,
                    isSOcompany = row.c_isSOcompany,
                    esdate = row.c_regtime,
                    ktgg = row.c_ktgg,
                    zgcpwsw = row.c_zgcpwsw,
                    rmfygg = row.c_rmfygg,
                    lending = row.c_lending,
                    xzcf = row.c_xzcf,
                    zhixing = row.c_zhixing,
                    dishonesty = row.c_dishonesty,
                    jyyc = row.c_jyyc,
                    circxzcf = row.c_circxzcf,
                    opescope = row.c_opescope,
                    address = row.c_address,
                    estatus = row.c_estatus,
                    province = row.c_province
                )) for row in relations]
        
        if is_directed == 1:
            g = init_graph(
                company_correlative_edges,
                company_correlative_nodes, is_directed = 1) 
        else:
            g = init_graph(
                company_correlative_edges, 
                company_correlative_nodes, is_directed = 0)
        return g

    @__fault_tolerant
    def get_feature_1(cls):
        '''
        企业背景风险
        '''
        one_relation_set = [
            node for node, attr in cls.DIG.nodes_iter(data=True) if attr['distance'] == 1]
        shareholder = [
            src_node for src_node, des_node, edge_attr 
            in cls.DIG.edges_iter(one_relation_set, data=True) 
            if des_node == cls.tarcompany 
            and edge_attr['is_invest'] == 'INVEST']
        os_shareholder = [
            node for node in shareholder 
            if cls.DIG.node[node]['isSOcompany'] is True 
            and cls.DIG.node[node]['is_human'] == 0]
        anti_os_shareholder = [
            node for node in shareholder 
            if cls.DIG.node[node]['isSOcompany'] is False 
            and cls.DIG.node[node]['is_human'] == 0]
        nature_shareholder = [
            node for node in shareholder 
            if cls.DIG.node[node]['is_human'] == 1]
        is_ipo_tarcompany = 1 if cls.resultiterable.data[0].a_isIPOcompany is True else 0
    
        x = len(os_shareholder)
        y = len(anti_os_shareholder)
        z = len(nature_shareholder)
        w = len(shareholder)
        r_i = is_ipo_tarcompany
        r = (0.2*x + 0.5*y + z) * 100. * (2 - r_i) / (2*w + 0.001) 
        
        return dict(
            x=x,
            y=y,
            z=z,
            w=w,
            r_i=r_i,
            r=round(r, 2) if r else 100
        )


    @__fault_tolerant
    def get_feature_2(cls):
        '''
        资本风险
        '''
        x = cls.resultiterable.data[0].a_regcap 
        y = cls.resultiterable.data[0].a_realcap
        
        x = x / 10000 if x != 'NULL' and x is not None else 0
        y = y / 10000 if y != 'NULL' and y is not None else 0
        
        if 0 <= x < 500:
            c_i = 100
        elif 500 <= x < 1000:
            c_i = 60
        elif 1000 <= x < 5000:
            c_i = 30
        elif 5000 <= x:
            c_i = 10
        
        p_i = 1 if y / (x + 0.001) >= 0.5 else 0
        
        return  dict(
            x=x,
            y=y,
            c=c_i * (1 - p_i/2.)
        )
    
    @__fault_tolerant
    def get_feature_3(cls):
        '''
        公司地域风险
        '''
        province_black_num = cls.resultiterable.data[0].a_province_black_num
        province_leijinrong_num = cls.resultiterable.data[0].a_province_leijinrong_num
        province_black_num = province_black_num if province_black_num else 0
        province_leijinrong_num = province_leijinrong_num if province_leijinrong_num else 0
        
        return dict(
            j_1=province_black_num,
            j_2=province_leijinrong_num,
            j=round(province_black_num / (province_leijinrong_num + 0.0001), 4)
        )
    
    @__fault_tolerant
    def get_feature_4(cls):
        '''
        知识产权风险（专利和商标）
        '''
        k_1 = cls.resultiterable.data[0].a_zhuanli
        k_2 = cls.resultiterable.data[0].a_shangbiao 
        k_1 = k_1 if k_1 != 'NULL' and k_1 is not None else 0
        k_2 = k_2 if k_1 != 'NULL' and k_2 is not None else 0
        k_3 = k_1 + k_2
        
        if k_3 == 0:
            k = 100
        elif 1 <= k_3 < 5:
            k = 60
        elif 5 <= k_3 < 10:
            k = 30
        elif 10 <= k_3:
            k = 10
        
        return dict(
            k_1=k_1,
            k_2=k_2,
            k=k
        )
        
    @__fault_tolerant
    def get_feature_5(cls):
        '''
        域名备案风险
        '''
        icp = cls.resultiterable.data[0].a_ICP
        url = cls.resultiterable.data[0].a_url
        
        c_i = 1 if icp != 'NULL' and icp is not None else 0   
        try:
            if url is not None and url != 'NULL':
                status_code = requests.get(url, allow_redirects=False).status_code 
                p_i = 1 if status_code == 200 else 0
            else:
                p_i = 0
        except:
            p_i = 0
        
        l = (1 - c_i/2.) * (1 - p_i/2.) * 100
        
        return dict(
            c_i=c_i,
            p_i=p_i,
            l=l
        )
        
    @__fault_tolerant
    def get_feature_6(cls):
        '''
        工商变更风险
        '''
        def get_bgxx_symbol(bgxx_name):
            if u'法定代表人' in bgxx_name:
                return 'c_1'
            elif u'股东' in bgxx_name:
                return 'c_2'
            elif u'注册资本' in bgxx_name:
                return 'c_3'
            elif u'高管' in bgxx_name:
                return 'c_4'
            elif u'经营范围' in bgxx_name:
                return 'c_5'
            else:
                return 'c_6'
        
        default_result = OrderedDict([
                ('c_1', 0),
                ('c_2', 0),
                ('c_3', 0),
                ('c_4', 0),
                ('c_5', 0),
                ('c_6', 0),
                ('r', 0),
                ('z', 10)
            ]
        )
        
        if cls.resultiterable.data[0].a_bgxx is not None:
            for bgxx_name, bgxx_num in cls.resultiterable.data[0].a_bgxx.iteritems():
                default_result[get_bgxx_symbol(bgxx_name)] += int(bgxx_num)
            default_result['r'] = np.dot(default_result.values(), [2, 1, 1, 1, 2, 0, 0, 0])
        
            if default_result['r'] == 0:
                default_result['z'] = 10
            elif 1 <= default_result['r'] < 5:
                default_result['z'] = 30
            elif 5 <= default_result['r'] < 10:
                default_result['z'] = 60
            elif 10 <= default_result['r']:
                default_result['z'] = 100
                
        default_result.pop('c_6')
        
        return dict(default_result)
        
    @__fault_tolerant
    def get_feature_7(cls):
        '''
        人才结构风险
        '''        
        def get_recruit_num(each_recruit):
            if u'本科' in each_recruit:
                return 'e_2'
            elif u'硕士' in each_recruit:
                return 'e_3'
            elif u'博士' in each_recruit:
                return 'e_3'
            else:
                return 'e_1'
            
        default_result = OrderedDict([
                ('e_1', 0),
                ('e_2', 0),
                ('e_3', 0),
                ('e', 0),
                ('r', 0),
                ('z', 10)
            ]
        )
        
        if cls.resultiterable.data[0].a_recruit is not None:
            for education_name, education_num in cls.resultiterable.data[0].a_recruit.iteritems():
                default_result[get_recruit_num(education_name)] += int(education_num)
            default_result['e'] = sum(
                map(int ,cls.resultiterable.data[0].a_recruit.values()))
            default_result['r'] = round(
                np.dot(default_result.values(), [100., 60., 30., 0, 0, 0]) / default_result['e'], 2)
            
            if default_result['r'] == 0:
                default_result['z'] = 10
            elif 0 < default_result['r'] < 60:
                default_result['z'] = 30
            elif 60 <= default_result['r'] < 80:
                default_result['z'] = 60
            elif 80 <= default_result['r']:
                default_result['z'] = 100         
            
        return dict(default_result)
        
    @__fault_tolerant
    def get_feature_8(cls):
        '''
        公司运营持续风险
        '''
        esdate_relation_set = [
            (datetime.date.today() - attr['esdate']).days 
            for node, attr in cls.DIG.nodes_iter(data=True) 
            if attr['distance'] <= 3 
            and attr['is_human'] == 0 
            and attr['esdate'] is not None]
        t = round(np.average(esdate_relation_set), 2)
        
        if 0 <= t < 365:
            y = 100
        elif 365 <= t < 1825:
            y = 60
        elif 1825 <= t < 3650:
            y = 30
        elif 3650 <= t:
            y = 10
        else:
            y = 100
        return dict(
            t=t if t else 0,
            y=y
        )

        
    @__fault_tolerant
    def get_feature_9(cls):
        '''
        招标和中标风险
        '''
        n_1 = cls.resultiterable.data[0].a_zhaobiao
        n_2 = cls.resultiterable.data[0].a_zhongbiao
        n_1 = n_1 if n_1 is not None else 0
        n_2 = n_2 if n_2 is not None else 0
        
        n = n_1 + n_2
        
        if n == 0:
            n = 100
        elif 1 <= n < 5:
            n = 60
        elif 5<= n < 10:
            n = 30
        else:
            n = 10
        
        return  dict(
            n_1=n_1,
            n_2=n_2,
            n=n
        )
        
    @__fault_tolerant
    def get_feature_xgxx(cls):
        '''
        【计算基础数据】
        法律诉讼风险：开庭公告、裁判文书、法院公告、民间借贷
        行政处罚
        被执行风险
        异常经营风险：经营异常、吊销&注销
        银监会行政处罚
        ''' 
        def get_certain_distance_all_info(distance, document_types):
            all_array = []
            #处理某一个distance不存在节点的情况
            all_array.append([0]*len(document_types))
            for node, attr in cls.DIG.nodes_iter(data=True):
                if attr['is_human'] == 0  and attr['distance'] == distance:
                    each_array = map(lambda x: x if x else 0, 
                                                  [attr[each_document] 
                                                      for each_document in document_types])
                    all_array.append(each_array)
                else:
                    continue
            documents_num = np.sum(all_array, axis=0)
            return documents_num
        
        matrx = dict()
        xgxx_type = ['ktgg', 'zgcpwsw', 'rmfygg', 
                             'lending', 'xzcf', 'zhixing', 
                             'dishonesty', 'jyyc', 'circxzcf', 
                             'estatus']

        for each_distance in xrange(0, 4):
            xgxx_num_list = get_certain_distance_all_info(each_distance, 
                                                                                      xgxx_type)
            matrx[each_distance] = dict(zip(xgxx_type, xgxx_num_list))
            
        return matrx
    
    @__fault_tolerant
    def filter_xgxx_type(cls, tar_xgxx):
        '''
        过滤相关信息
        '''
        risk = dict()
        for each_distance, xgxx_statistics in cls.xgxx_distribution.iteritems():
            tar_xgxx_statistics = dict([
                    (each_xgxx_type, each_xgxx_num)
                    for each_xgxx_type, each_xgxx_num 
                    in xgxx_statistics.iteritems() 
                    if each_xgxx_type in tar_xgxx])
            risk[each_distance] = tar_xgxx_statistics
        return risk
    
    @__fault_tolerant
    def get_feature_10(cls):
        '''
        法律诉讼风险：开庭公告、裁判文书、法院公告、民间借贷
        '''
        tar_xgxx = ['ktgg', 'zgcpwsw', 'rmfygg', 'lending']
        risk = cls.filter_xgxx_type(tar_xgxx)
        
        risk['r'] = round(
            np.dot(
                map(sum, [
                        risk[each_distance].values() 
                        for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)
        
        if risk['r'] == 0:
            risk['z'] = 10
        elif 0 < risk['r'] <= 10:
            risk['z'] = 30
        elif 10 < risk['r'] <= 30:
            risk['z'] = 60
        elif 30 < risk['r']:
            risk['z'] = 100
        
        return risk
            
    @__fault_tolerant
    def get_feature_11(cls):
        '''
        行政处罚风险
        '''
        tar_xgxx = ['xzcf']
        risk = cls.filter_xgxx_type(tar_xgxx)
        
        risk['r'] = round(
            np.dot(
                map(sum, [risk[each_distance].values() 
                        for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)
        
        if risk['r'] == 0:
            risk['z'] = 10
        elif 0 < risk['r'] <= 10:
            risk['z'] = 30
        elif 10 < risk['r'] <= 30:
            risk['z'] = 60
        elif 30 < risk['r']:
            risk['z'] = 100
        
        return risk
    
    @__fault_tolerant
    def get_feature_12(cls):
        '''
        被执行风险：被执行、失信被执行
        '''
        tar_xgxx = ['zhixing', 'dishonesty']
        risk = cls.filter_xgxx_type(tar_xgxx)        
        
        risk['r'] = round(
            np.dot(
                map(
                    lambda x: x['zhixing'] + 2. * x['dishonesty'], 
                    [risk[each_distance] for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)
        
        if risk['r'] == 0:
            risk['z'] = 10
        elif 0 < risk['r'] <= 10:
            risk['z'] = 30
        elif 10 < risk['r'] <= 30:
            risk['z'] = 60
        elif 30 < risk['r']:
            risk['z'] = 100        
        
        return risk
        
    @__fault_tolerant
    def get_feature_13(cls):
        '''
        异常经营风险
        '''
        tar_xgxx = ['jyyc', 'estatus']
        risk = cls.filter_xgxx_type(tar_xgxx)        
        
        risk['r'] = round(
            np.dot(
                map(
                    lambda x: x['jyyc'] + 2. * x['estatus'], 
                    [risk[each_distance] for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)

        if risk['r'] == 0:
            risk['z'] = 10
        elif 0 < risk['r'] <= 10:
            risk['z'] = 30
        elif 10 < risk['r'] <= 30:
            risk['z'] = 60
        elif 30 < risk['r']:
            risk['z'] = 100
        
        return risk        
        
    @__fault_tolerant
    def get_feature_14(cls):
        '''
        银监会行政处罚
        '''
        tar_xgxx = ['circxzcf']
        risk = cls.filter_xgxx_type(tar_xgxx)        
        
        risk['r'] = round(
            np.dot(
                map(
                    lambda x: x['circxzcf'], 
                    [risk[each_distance] for each_distance in xrange(0, 4)]), 
                [1., 1/2., 1/3., 1/4.]), 2)

        if risk['r'] == 0:
            risk['z'] = 10
        elif 0 < risk['r'] <= 10:
            risk['z'] = 30
        elif 10 < risk['r'] <= 30:
            risk['z'] = 60
        elif 30 < risk['r']:
            risk['z'] = 100        
    
        return risk            
        
    @__fault_tolerant
    def get_feature_15(cls):
        '''
        实际控制人风险
        '''
        def get_degree_distribution(is_human):
            some_person_sets = [
                node
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if attr['is_human'] == is_human
                and attr['distance'] <= 3]
            person_out_degree = cls.DIG.out_degree(some_person_sets).values()
            if not person_out_degree:
                person_out_degree.append(0)
            return person_out_degree
        
        nature_person_distribution = get_degree_distribution(1)
        legal_person_distribution = get_degree_distribution(0)
        
        nature_max_control = max(nature_person_distribution)
        legal_max_control = max(legal_person_distribution)        
        nature_avg_control = round(np.average(nature_person_distribution), 2)
        legal_avg_control = round(np.average(legal_person_distribution), 2)
        
        total_legal_num = len([
                node 
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if attr['is_human'] == 0])
        
        risk = round(((
                    2*(nature_max_control + legal_max_control) + 
                    (nature_avg_control + legal_avg_control)) /
                (2*total_legal_num + 0.001)), 2)
        
        if 0 <= risk <= 0.5:
            r = 10
        elif 0.5 < risk <= 1:
            r = 30
        elif 1 < risk <= 3:
            r = 60  
        elif 3 < risk:
            r = 100
        else:
            r = 10
        
        return dict(
            x_1=nature_max_control,
            x_2=legal_max_control,
            y_1=nature_avg_control,
            y_2=legal_avg_control,
            z=total_legal_num,
            s=risk,
            r=r
        )
    
    @__fault_tolerant
    def get_feature_16(cls):
        '''
        公司扩张路径风险
        '''
        def get_node_set(is_human):
            return Counter([
                    attr['distance']
                    for node, attr in cls.DIG.nodes_iter(data=True)
                    if attr['is_human'] == is_human])
        
        nature_person_distribution = get_node_set(1)
        legal_person_distribution = get_node_set(0)
        
        nature_person_num = [
            nature_person_distribution.get(each_distance, 0)
            for each_distance in range(1, 4)]
        legal_person_num = [
            legal_person_distribution.get(each_distance, 0)
            for each_distance in range(1, 4)]
        
        risk = round(np.sum(
                np.divide(
                    [
                        np.divide(
                            np.sum(nature_person_num[:each_distance]), 
                            np.sum(legal_person_num[:each_distance]), 
                            dtype=float)
                        for each_distance in range(1, 4)], 
                    np.array([1, 2, 3], dtype=float))), 2)
        
        if 0 <= risk <= 1:
            r = 10
        elif 1 < risk <= 3:
            r = 30
        elif 3 < risk <= 5:
            r = 60  
        elif 5 < risk:
            r = 100
        else:
            r = 10
        
        return dict(
            x_1=legal_person_num[0],
            x_2=legal_person_num[1],
            x_3=legal_person_num[2],
            y_1=nature_person_num[0],
            y_2=nature_person_num[1],
            y_3=nature_person_num[2],
            z=risk if risk < 10000 else 0,
            r=r
        )

    @__fault_tolerant
    def get_feature_17(cls):
        '''
        关联方中心集聚风险
        '''
        one_relation_set = [
            node for node, attr in cls.DIG.nodes_iter(data=True) 
            if attr['distance'] == 1]
        legal_person_shareholder = len([
                src_node 
                for src_node, des_node, edge_attr 
                in cls.DIG.edges_iter(one_relation_set, data=True) 
                if des_node == cls.tarcompany 
                and edge_attr['is_invest'] == 'INVEST'
                and cls.DIG.node[src_node]['is_human'] == 0])
        legal_person_subsidiary = len([
                des_node 
                for src_node, des_node, edge_attr 
                in cls.DIG.edges_iter(one_relation_set, data=True) 
                if src_node == cls.tarcompany 
                and edge_attr['is_invest'] == 'INVEST'
                and cls.DIG.node[des_node]['is_human'] == 0])
        
        risk =(
            legal_person_subsidiary -
            legal_person_shareholder ) 
        
        if risk < 0:
            r = 10
        elif risk == 0:
            r = 30
        elif 0 < risk <= 3:
            r = 60
        elif 3 < risk:
            r = 100
        else:
            r = 100        
        
        return dict(
            x=legal_person_subsidiary,
            y=legal_person_shareholder,
            z=risk,
            r=r
        )

    @__fault_tolerant
    def get_feature_18(cls):
        '''
        分支机构过度扩张风险
        '''
        risk = cls.resultiterable.data[0].a_fzjg
        risk = risk if risk is not None else 0

        if  risk == 0:
            r = 10
        elif risk == 1:
            r = 30
        elif 1 < risk <= 3:
            r = 60  
        elif 3 < risk:
            r = 100
        else:
            r = 10
        
        return dict(
            d=risk if risk < 100 else 100, 
            z=r
        )    
    
    @__fault_tolerant
    def get_feature_19(cls):
        '''
        关联方结构稳定风险
        '''
        def get_relation_num():
            return Counter([
                    attr['distance']
                    for node, attr in cls.DIG.nodes(data=True)])
        
        #目标企业各个度的节点的数量
        relations_num = get_relation_num()
        relation_three_num = relations_num.get(3, 0)
        relation_two_num = relations_num.get(2, 0)
        relation_one_num = relations_num.get(1, 0)
        relation_zero_num = relations_num.get(0, 1)
        
        x = np.array([
                relation_zero_num, 
                relation_one_num, 
                relation_two_num, 
                relation_three_num]).astype(float)
        
        y_2 = x[2] / (x[1]+x[2])
        y_3 = x[3] / (x[1]+x[2]+x[3])
        risk = y_2/2 + y_3/3
        

        
        if 0 <= risk <= 0.1:
            r = 10
        elif 0.1 < risk <= 0.3:
            r = 30
        elif 0.3 < risk <= 0.5:
            r = 60  
        elif 0.5 < risk:
            r = 100      
        else:
            r = 10
        
        return dict(
            x_1=x[1],
            x_2=x[2],
            x_3=x[3],
            y_2=y_2,
            y_3=y_3,
            z=risk if risk else 0,
            r=r
        )        
    
    @__fault_tolerant
    def get_feature_20(cls):
        '''
        潜在违规融资风险
        '''

        
        def get_relation_risk_num(keyword_type):
            return Counter([attr['distance']
                    for node, attr in cls.DIG.nodes_iter(data=True) 
                    if attr['is_human'] == 0 and 
                    attr['opescope'] == keyword_type])
        
        k_2_num = get_relation_risk_num('k_2')
        k_1_num = get_relation_risk_num('k_1')
        
        x = np.array([
                k_2_num.get(each_distance, 0) 
                for each_distance in xrange(1, 4)])
        y = np.array([
                k_1_num.get(each_distance, 0) 
                for each_distance in xrange(1, 4)])
    
        risk = round(
            np.sum(
                np.divide(np.add(x, y), [1, 2, 3], dtype=float)), 2)
    
        if  risk == 0:
            r = 10
        elif 0 < risk < 5:
            r = 30
        elif 5 <= risk < 10:
            r = 60  
        elif 10 <= risk:
            r = 100     
        else:
            r = 10
    
        return dict(
            x_1=x[0],
            x_2=x[1],
            x_3=x[2],
            y_1=y[0],
            y_2=y[1],
            y_3=y[2],
            g=r
        )
    
    @__fault_tolerant
    def get_feature_21(cls):
        '''
        关联方地址集中度风险
        '''
        legal_person_address = [
            attr['address'] 
            for node, attr in cls.DIG.nodes_iter(data=True) 
            if attr['distance'] <= 3 
            and attr['is_human'] == 0]
        c = Counter(
            filter(
                lambda x: x is not None and len(x) >= 21,  
                legal_person_address))
        n = c.most_common(1)
        n= n[0][1] if len(n) > 0 else 1
        risk = n -1
        
        if  risk == 0:
            z = 10
        elif risk == 1:
            z = 30
        elif risk == 2:
            z = 60  
        elif 2 < risk:
            z = 100             
        
        return dict(
            n=risk,
            y=risk,
            z=z
        )
    
    @__fault_tolerant
    def get_feature_22(cls):
        '''
        利益一致行动法人风险
        '''
        def is_similarity(str_1, str_2):
            '''
            判断两个字符串是否有连续2个字相同
            '''
            try:
                token_1 = [
                    str_1[index] + str_1[index+1] 
                    for index,data in enumerate(str_1) 
                    if index < len(str_1) - 1]
                is_similarity = sum([
                        1 for each_token in token_1 
                        if each_token in str_2])
                return True if is_similarity > 0 else False        
            except:
                return False
            
        #目标公司的名称frag
        tar_company = cls.resultiterable.data[0].a_name
        tar_company_frag = cls.resultiterable.data[0].a_namefrag
        #三度以内，所有关联方的节点集合(不包含自身)
        relation_set = [
                attr['name'] 
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if attr['distance'] <= 3]
        #relation_set.remove(tar_company)
        common_interests_list = [
            1 
            for node_name in relation_set 
            if is_similarity(tar_company_frag, node_name)]
        
        risk = sum(common_interests_list)
        if risk == 0:
            z = 10
        elif risk == 1:
            z = 30
        elif 1 < risk <= 3:
            z = 60
        elif 3 < risk:
            z = 100
        else:
            z = 10
        
        return dict(
            d=risk, 
            z=z
        )        
        
    @__fault_tolerant
    def get_feature_23(cls):
        '''
        关联方信誉风险
        '''
        def get_black_num(distance):
            '''
            某一度黑企业的总数
            '''
            return sum([
                    1
                    for node, attr in cls.DIG.nodes_iter(data=True)
                    if (attr['is_black'] and attr['distance'] == distance 
                            and attr['is_human'] == 0)
            ])
        
        black_relation_set =[
            get_black_num(each_distance) 
            for each_distance in range(1, 4)]
        risk = np.dot(black_relation_set, [1, 1/2., 1/3.])
        if risk == 0:
            z = 10
        elif 0 < risk < 0.5:
            z = 30
        elif 0.5 <= risk < 1:
            z = 60
        elif 1 <= risk:
            z =100
        else:
            z = 10
        
        return dict(
            b_1=black_relation_set[0],
            b_2=black_relation_set[1],
            b_3=black_relation_set[2],
            y=risk,
            z=z
        )
    
    @__fault_tolerant
    def get_feature_24(cls):
        '''
        短期逐利风险
        '''
        
        def get_max_established(distance, timedelta):
            '''
            获取在某distance关联方企业中，某企业在任意timedelta内投资成立公司数量的最大值
            '''
            #用于获取最大连续时间数，这里i的取值要仔细琢磨一下，这里输入一个时间差序列与时间差
            def get_date_density(difference_list, timedelta):
                time_density_list = []
                for index, date in enumerate(difference_list):
                    if date < 15:
                        s = 0
                        i = 1
                        while(s < 15 and i <= len(difference_list) - index):
                            i += 1
                            s = sum(difference_list[index:index+i])
                        time_density_list.append(i)
                    else:
                        continue
                return max(time_density_list) if len(time_density_list) >0 else 0            

            #distance所有节点集合
            relation_set = [
                node 
                for node, attr in cls.DIG.nodes(data=True) 
                if attr['distance'] == distance]
            investment_dict = defaultdict(lambda : [])     
            
            if len(cls.DIG.edge) > 1:
                for src_node, des_node, edge_attr in (
                        cls.DIG.edges_iter(relation_set, data=True)):
                    if (edge_attr.get('is_invest', 0) == 'INVEST' 
                            and cls.DIG.node[des_node]['distance'] == distance
                            and cls.DIG.node[des_node]['esdate'] is not None 
                            and cls.DIG.node[src_node]['is_human'] == 0):
                        #将所有节点投资的企业的成立时间加进列表中
                        investment_dict[src_node].append(cls.DIG.node[des_node]['esdate'])
             
            #目标企业所有节点所投资的企业时间密度字典
            all_date_density_dict = {}
            
            for node, date_list in investment_dict.iteritems():
                #构建按照时间先后排序的序列
                date_strp_list = sorted(date_list)
                if len(date_strp_list) > 1:
                    #构建时间差的序列，例如：[256, 4, 5, 1, 2, 33, 6, 5, 4, 73]
                    date_difference_list = [
                        (date_strp_list[index + 1] - date_strp_list[index]).days 
                        for index in range(len(date_strp_list) -1)]
                    #计算某法人节点在timedelta天之内有多少家公司成立
                    es_num = get_date_density(date_difference_list, timedelta)
                    if all_date_density_dict.has_key(es_num):
                        all_date_density_dict[es_num].append(node)
                    else:
                        all_date_density_dict[es_num] = [node]              
                else:
                    continue
            keys = all_date_density_dict.keys()        
            max_num = max(keys) if len(keys) > 0 else 0
            
            return max_num
        
        x = [
            get_max_established(each_distance, 180) 
            for each_distance in xrange(1, 4)]
        
        risk = round(
            np.sum(
                np.divide(x, [1, 2, 3], dtype=float)), 2)
        
        if  risk == 0:
            r = 10
        elif 0 < risk <= 0.7:
            r = 30
        elif 0.7 < risk <= 1:
            r = 60  
        elif 1 < risk:
            r = 100       
        else:
            r = 10
        
        return dict(
            x_1=x[0],
            x_2=x[1],
            x_3=x[2],
            z=r,
            d=risk
        )
    

    @classmethod
    def get_some_feature(cls, resultiterable, feature_nums):
        #创建类变量
        cls.resultiterable = resultiterable
        cls.tarcompany = cls.resultiterable.data[0].a
        cls.DIG = cls.create_graph_udf(cls.resultiterable, 1)
        cls.xgxx_distribution = cls.get_feature_xgxx()
        
        feature_list = [
            ('feature_{0}'.format(feature_index), 
                     eval('cls.get_feature_{0}()'.format(feature_index)))
            for feature_index in feature_nums]
        feature_list.append(('bbd_qyxx_id', cls.tarcompany))
        feature_list.append(('company_name', cls.resultiterable.data[0].a_name))
        
        return json.dumps(dict(feature_list), ensure_ascii=False)      #

class DynamicFeatureConstruction(FeatureConstruction):
    '''
    计算特征的函数集
    '''      

    def __fault_tolerant(func):
        '''
        一个用于容错的装饰器
        '''
        @classmethod
        def wappen(cls, *args, **kwargs):
            try:
                return func(cls, *args, **kwargs)
            except Exception, e:
                return (
                    "{func_name} has a errr : {excp}"
                ).format(func_name=func.__name__, excp=e)
        return wappen    
    
    @classmethod
    def get_feature_25(cls):
        '''
        平台稳态运营风险
        '''
        nature_person_num = sum([
                1
                for node, attr in  cls.DIG.nodes_iter(data=True)
                if (attr['is_human'] == 1
                        and attr['distance'] <= 3)
        ])
        legal_person_num = sum([
                1
                for node, attr in  cls.DIG.nodes_iter(data=True)
                if (attr['is_human'] == 0
                        and attr['distance'] <= 3)
        ])
        return dict(
            nature_person_num=nature_person_num,
            legal_person_num=legal_person_num
        )
    
    @classmethod
    def get_feature_26(cls):
        '''
        平台核心资本运作风险
        '''
        some_person_sets = [
            node
            for node, attr in cls.DIG.nodes_iter(data=True) 
            if (attr['is_human'] == 1
                    and attr['distance'] <= 3)]
        person_out_degree = cls.DIG.out_degree(
            some_person_sets).values()
        person_out_degree.sort(reverse=True)
        if not person_out_degree:
            person_out_degree.append(0)

        return dict(
            kernel_control_num=sum(person_out_degree[:3])
        )
    
    @classmethod
    def get_feature_27(cls):
        '''
        可持续性风险
        '''
        def is_similarity(str_1, str_2):
            '''
            判断两个字符串是否有连续2个字相同
            '''
            try:
                token_1 = [
                    str_1[index] + str_1[index+1] 
                    for index,data in enumerate(str_1) 
                    if index < len(str_1) - 1]
                is_similarity = sum([
                        1 for each_token in token_1 
                        if each_token in str_2])
                return True if is_similarity > 0 else False        
            except:
                return False
            
        #目标公司的名称frag
        tar_company = cls.resultiterable.data[0].a_name
        tar_company_frag = cls.resultiterable.data[0].a_namefrag
        #三度以内，所有关联方的节点集合(不包含自身)
        relation_set = [
                attr['name']
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if attr['distance'] <= 3]
        #relation_set.remove(tar_company)
        common_interests_num = sum([
                1 
                for node_name in relation_set 
                if is_similarity(tar_company_frag, node_name)
        ])
        total_legal_person_num = sum([
                1
                for node, attr in cls.DIG.nodes_iter(data=True) 
                if (attr['distance'] <= 3 and 
                       attr['is_human'] == 0)
        ])
        if total_legal_person_num:
            ratio = round(
                common_interests_num / total_legal_person_num, 2
            )
        else:
            ratio = 0
            
        return dict(
            ratio=ratio
        )
        
    @classmethod
    def get_feature_28(cls):
        '''
        平台跨区域舞弊风险
        '''
        province_list = filter(
            None, [
                attr['province']
                for node, attr in cls.DIG.nodes_iter(data=True)
                if (attr['distance'] <= 3 and 
                       attr['is_human'] == 0)]
        )
        province_distribution = Counter(province_list)
        province_top4_num = sum(
            map(lambda (k, v): v, province_distribution.most_common(4)))
        
        return dict(
            province_top4_num=province_top4_num
        )
    
    
    @classmethod
    def get_some_feature(cls, resultiterable, feature_nums):
        #创建类变量
        cls.resultiterable = resultiterable
        cls.tarcompany = cls.resultiterable.data[0].a
        cls.DIG = cls.create_graph_udf(cls.resultiterable, 1)
        cls.G = cls.create_graph_udf(cls.resultiterable, 0)
        
        feature_list = [
            ('feature_{0}'.format(feature_index), 
                     eval('cls.get_feature_{0}()'.format(feature_index)))
            for feature_index in feature_nums]
        feature_list.append(('bbd_qyxx_id', cls.tarcompany))
        feature_list.append(('company_name', cls.resultiterable.data[0].a_name))
        
        return dict(feature_list) #json.dumps(dict(feature_list), ensure_ascii=False)      

    
def get_dynamic_risk(data):
    '''
    计算动态风险
    '''
    old_data, new_data = data
    #平台稳态运营风险
    if old_data['feature_25']['nature_person_num']:
        nature_person_risk = ((new_data['feature_25']['nature_person_num'] - 
                                            old_data['feature_25']['nature_person_num']) / 
                                            old_data['feature_25']['nature_person_num'])
    else:
        nature_person_risk = 0.
    if old_data['feature_25']['legal_person_num']:
        legal_person_risk = ((new_data['feature_25']['legal_person_num'] - 
                                          old_data['feature_25']['legal_person_num']) / 
                                          old_data['feature_25']['legal_person_num'])
    else:
        legal_person_risk = 0.
    feature_25 = 5. * (nature_person_risk + legal_person_risk)
    
    #平台核心资本运作风险
    if old_data['feature_26']['kernel_control_num']:
        feature_26 = (15. * 
                              new_data['feature_26']['kernel_control_num'] / 
                              old_data['feature_26']['kernel_control_num'])
    else:
        feature_26 = 0.
        
    #可持续性风险
    if old_data['feature_27']['ratio']:
        feature_27 = (4. * 
                              new_data['feature_27']['ratio'] / 
                              old_data['feature_27']['ratio'])
    else:
        feature_27 = 0.
        
    #平台跨区域舞弊风险
    if old_data['feature_28']['province_top4_num']:
        feature_28 = (15. * 
                              new_data['feature_28']['province_top4_num'] / 
                              old_data['feature_28']['province_top4_num'])
    else:
        feature_28 = 0.
        
    return dict(
        feature_25={'p': round(feature_25, 2)},
        feature_26={'h': round(feature_26, 2)},
        feature_27={'c': round(feature_27, 2)},
        feature_28={'k': round(feature_28, 2)}
    )
    
def spark_data_flow(tid_old_version, tid_new_version):
    '''
    dataflow
    '''
    conf = SparkConf()
    conf.setMaster('yarn-client')
    conf.set("spark.yarn.am.cores", 7)
    conf.set("spark.executor.memory", "50g")
    conf.set("spark.executor.instances", 20)
    conf.set("spark.executor.cores", 10)
    conf.set("spark.python.worker.memory", "2g")
    conf.set("spark.default.parallelism", 1000)
    conf.set("spark.sql.shuffle.partitions", 1000)
    conf.set("spark.broadcast.blockSize", 1024)
    conf.set("spark.executor.extraJavaOptions","-XX:+PrintGCDetails -XX:+PrintGCTimeStamps")    
    conf.set("spark.shuffle.file.buffer", '512k')
    
    spark = SparkSession \
        .builder \
        .appName("glf") \
        .config(conf = conf) \
        .enableHiveSupport() \
        .getOrCreate()  
    
    #输入
    #算法设计直接用rdd
    tid_old_df = spark.read.parquet(
        "/user/antifraud/jinli/tiddata/relation_company_info_merge/{version}".format(version=tid_old_version))
    tid_old_rdd = tid_old_df.rdd
    tid_new_df = spark.read.parquet(
        "/user/antifraud/jinli/tiddata/relation_company_info_merge/{version}".format(version=tid_new_version))
    tid_new_rdd = tid_new_df.rdd    
        
    #最终计算流程
    tid_new_rdd_2 = tid_new_rdd.map(lambda row: (row.a_name, row)) \
        .groupByKey() \
        .repartition(600) \
        .filter(lambda r: len(r[1].data) <= 150000) \
        .cache()
    tid_old_rdd_2 = tid_old_rdd.map(lambda row: (row.a_name, row)) \
        .groupByKey() \
        .repartition(600) \
        .filter(lambda r: len(r[1].data) <= 150000) \
        .cache()

    feature_new_list = tid_new_rdd_2.mapValues(
        lambda v: DynamicFeatureConstruction.get_some_feature(
            v, [_ for _ in range(25, 29)]))
    feature_old_list = tid_old_rdd_2.mapValues(
        lambda v: DynamicFeatureConstruction.get_some_feature(
            v, [_ for _ in range(25, 29)]))
    
    dynamic_risk_data = feature_old_list.join(
        feature_new_list
    ).mapValues(
        get_dynamic_risk
    ).map(
        lambda (k, v): dict({'company_name': k}, **v)
    ).map(
        lambda data: json.dumps(data, ensure_ascii=False)
    )

    return dynamic_risk_data

def collect_to_driver(version, out_path, old_version, new_version):
    '''
    格式化输出
    '''
    src_time = time.time()
    dynamic_risk_data = spark_data_flow(old_version, new_version)
    os.system("hadoop fs -rmr {path}/{version}/prd_jinli_dynamic_feature_score_distribution_{version}".format(path=OUTPATH, version=VERSION))
    dynamic_risk_data.saveAsTextFile("{path}/{version}/prd_jinli_dynamic_feature_score_distribution_{version}".format(path=OUTPATH, version=VERSION))

    des_time = time.time()    
    
    print 'SUCCESS !! \n 耗时：{0}'.format(des_time - src_time)
    
    
if __name__ == '__main__':  
    #输出参数
    VERSION = "20170417"
    OUTPATH = "/user/antifraud/jinli/tmpdata"
    
    collect_to_driver(
        version=VERSION,
        out_path=OUTPATH,
        old_version='20170117',
        new_version='20170403'
    )



Overwriting /data5/antifraud/source/tmp_20170413/hongjing_jinli_dynamic_kpi.py


In [255]:
import json
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql import Row

schema = StructType(
    [StructField('feature_19',FloatType(),True)])

rdd = spark.sparkContext.textFile("/user/antifraud/jinli/tmpdata/20170413/prd_jinli_feature_score_distribution_20170413")
rdd = rdd.map(
    json.loads
).filter(
    lambda r: isinstance(r['feature_19']['z'], float)
).map(
    lambda r: (Vectors.dense([round(np.nan_to_num(r['feature_19']['z']), 2)]), )
).cache()

#rdd.collect()
df = spark.createDataFrame(rdd, ['feature'])
df.show(100)

+-------+
|feature|
+-------+
| [0.42]|
| [0.46]|
| [0.55]|
| [0.58]|
| [0.25]|
| [0.16]|
|  [0.7]|
| [0.25]|
| [0.32]|
| [0.55]|
| [0.25]|
| [0.73]|
| [0.61]|
| [0.13]|
| [0.24]|
| [0.51]|
| [0.73]|
|  [0.5]|
| [0.56]|
| [0.44]|
| [0.59]|
|  [0.1]|
| [0.52]|
|  [0.7]|
| [0.15]|
| [0.45]|
|  [0.0]|
| [0.48]|
| [0.39]|
| [0.33]|
|  [0.0]|
| [0.66]|
| [0.38]|
| [0.45]|
| [0.57]|
| [0.17]|
| [0.25]|
| [0.38]|
| [0.61]|
| [0.45]|
| [0.65]|
|  [0.0]|
| [0.17]|
|  [0.5]|
|  [0.7]|
| [0.41]|
| [0.67]|
| [0.17]|
| [0.36]|
| [0.26]|
| [0.33]|
| [0.32]|
|  [0.2]|
|  [0.0]|
|  [0.7]|
| [0.25]|
|  [0.3]|
| [0.63]|
| [0.36]|
| [0.25]|
| [0.19]|
| [0.28]|
| [0.69]|
| [0.49]|
| [0.54]|
| [0.39]|
| [0.71]|
| [0.42]|
| [0.36]|
| [0.77]|
| [0.66]|
| [0.18]|
| [0.57]|
| [0.42]|
| [0.36]|
|  [0.7]|
|  [0.6]|
| [0.74]|
| [0.48]|
| [0.68]|
| [0.66]|
| [0.63]|
| [0.65]|
| [0.19]|
|  [0.3]|
| [0.21]|
| [0.65]|
| [0.25]|
| [0.36]|
|  [0.0]|
| [0.13]|
|  [0.0]|
| [0.75]|
| [0.38]|
| [0.36]|
| [0.24]|
| [0.69]|


In [257]:
standardScaler = StandardScaler(inputCol="feature", outputCol="scaled", withStd=True, withMean=True)
model = standardScaler.fit(df)
scaledData = model.transform(df)
scaledData.show(100)

+-------+--------------------+
|feature|              scaled|
+-------+--------------------+
| [0.42]|[0.04718465138528...|
| [0.46]|[0.2481008360802646]|
| [0.55]|[0.7001622516439626]|
| [0.58]|[0.8508493901651948]|
| [0.25]|[-0.8067091335683...|
| [0.16]|[-1.2587705491320...|
|  [0.7]|[1.4535979442501252]|
| [0.25]|[-0.8067091335683...|
| [0.32]|[-0.4551058103521...|
| [0.55]|[0.7001622516439626]|
| [0.25]|[-0.8067091335683...|
| [0.73]| [1.604285082771358]|
| [0.61]|[1.0015365286864275]|
| [0.13]|[-1.4094576876532...|
| [0.24]|[-0.8569381797421...|
| [0.51]|[0.49924606694898...|
| [0.73]| [1.604285082771358]|
|  [0.5]|[0.4490170207752413]|
| [0.56]|[0.7503912978177069]|
| [0.44]|[0.1476427437327761]|
| [0.59]| [0.901078436338939]|
|  [0.1]|[-1.560144826174527]|
| [0.52]|[0.5494751131227298]|
|  [0.7]|[1.4535979442501252]|
| [0.15]|[-1.3089995953058...|
| [0.45]|[0.19787178990652...|
|  [0.0]|[-2.0624352879119...|
| [0.48]|[0.3485589284277528]|
| [0.39]|[-0.1035024871359...|
| [0.33]

In [258]:
from sklearn import preprocessing

arry = df.rdd.map(lambda r: r.feature.toArray()).collect()
scaler = preprocessing.StandardScaler().fit(arry)
print scaler.transform(arry)[:100]

[[ 0.04718469]
 [ 0.24810106]
 [ 0.70016289]
 [ 0.85085016]
 [-0.80670987]
 [-1.25877169]
 [ 1.45359926]
 [-0.80670987]
 [-0.45510622]
 [ 0.70016289]
 [-0.80670987]
 [ 1.60428654]
 [ 1.00153744]
 [-1.40945897]
 [-0.85693896]
 [ 0.49924652]
 [ 1.60428654]
 [ 0.44901743]
 [ 0.75039198]
 [ 0.14764288]
 [ 0.90107925]
 [-1.56014624]
 [ 0.54947561]
 [ 1.45359926]
 [-1.30900078]
 [ 0.19787197]
 [-2.06243716]
 [ 0.34855925]
 [-0.10350258]
 [-0.40487713]
 [-2.06243716]
 [ 1.2526829 ]
 [-0.15373167]
 [ 0.19787197]
 [ 0.80062107]
 [-1.2085426 ]
 [-0.80670987]
 [-0.15373167]
 [ 1.00153744]
 [ 0.19787197]
 [ 1.20245381]
 [-2.06243716]
 [-1.2085426 ]
 [ 0.44901743]
 [ 1.45359926]
 [-0.0030444 ]
 [ 1.30291199]
 [-1.2085426 ]
 [-0.25418986]
 [-0.75648077]
 [-0.40487713]
 [-0.45510622]
 [-1.05785533]
 [-2.06243716]
 [ 1.45359926]
 [-0.80670987]
 [-0.55556441]
 [ 1.10199562]
 [-0.25418986]
 [-0.80670987]
 [-1.10808442]
 [-0.65602259]
 [ 1.40337017]
 [ 0.39878834]
 [ 0.6499338 ]
 [-0.10350258]
 [ 1.50382

In [ ]:
df = df.filter()
scaler_a = StandardScaler(inputCol="_1", outputCol="scaledFeature",
                        withStd=True, withMean=False)
scalerModel = scaler_a.fit(df)
scaledData = scalerModel.transform(df)

In [ ]:
.map(
    lambda r: (Vectors.sparse(1, [(1, round(np.nan_to_num(r['feature_19']['z']), 2))]), )
).cache()

scaler_a = StandardScaler(inputCol="_1", outputCol="scaledFeature",
                        withStd=True, withMean=False)
scalerModel = scaler_a.fit(df)
scaledData = scalerModel.transform(df)

In [ ]:
scaler = StandardScaler(inputCol="_1", outputCol="scaledFeature",
                        withStd=True, withMean=False)
scalerModel = scaler.fit(df)
scaledData = scalerModel.transform(df)

In [119]:
rdd = spark.sparkContext.textFile("/user/antifraud/jinli/tmpdata/20170413/prd_jinli_feature_score_distribution_20170413")
rdd.map(
    json.loads
).take(1)

[{u'bbd_qyxx_id': u'97e922cfd9f24068b6a289df82ab9a71',
  u'company_name': u'\u5c71\u897f\u946b\u9686\u5229\u6e90\u6295\u8d44\u6709\u9650\u516c\u53f8',
  u'feature_1': {u'r': 99.98, u'r_i': 0, u'w': 3, u'x': 0, u'y': 0, u'z': 3},
  u'feature_10': {u'0': {u'ktgg': 0,
    u'lending': 0,
    u'rmfygg': 0,
    u'zgcpwsw': 0},
   u'1': {u'ktgg': 0, u'lending': 0, u'rmfygg': 0, u'zgcpwsw': 0},
   u'2': {u'ktgg': 0, u'lending': 0, u'rmfygg': 0, u'zgcpwsw': 0},
   u'3': {u'ktgg': 0, u'lending': 0, u'rmfygg': 0, u'zgcpwsw': 0},
   u'r': 0.0,
   u'z': 10},
  u'feature_11': {u'0': {u'xzcf': 0},
   u'1': {u'xzcf': 0},
   u'2': {u'xzcf': 0},
   u'3': {u'xzcf': 0},
   u'r': 0.0,
   u'z': 10},
  u'feature_12': {u'0': {u'dishonesty': 0, u'zhixing': 0},
   u'1': {u'dishonesty': 0, u'zhixing': 0},
   u'2': {u'dishonesty': 0, u'zhixing': 0},
   u'3': {u'dishonesty': 0, u'zhixing': 0},
   u'r': 0.0,
   u'z': 10},
  u'feature_13': {u'0': {u'estatus': 0, u'jyyc': 0},
   u'1': {u'estatus': 0, u'jyyc': 0},
   

In [130]:
dataFrame = spark.read.format("libsvm").load("/user/antifraud/source/tmp_test/tmp_file/sample_libsvm_data.txt")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(dataFrame)

# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(dataFrame)
scaledData.take(1)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.feature import StandardScaler

dataFrame = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(dataFrame)

# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(dataFrame)
scaledData.show()

In [47]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

data = [(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),),
        (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
        (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
df = spark.createDataFrame(data, ["features"])
pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(df)
result = model.transform(df).select("pcaFeatures")
result.show(truncate=False)

In [46]:
from pyspark.sql import types as tp, functions as fun

def get_num(col):
    import re
    try:
        return re.search('[\d\.\,]+', col).group()
    except:
        return None
get_num_udf = fun.udf(get_num, tp.StringType())

df = spark.read.csv("/user/antifraud/source/tmp_test/tmp_file/res.csv", sep=',')
df = df.withColumn('_c7', get_num_udf('_c5'))
df = df.withColumn('_c8', get_num_udf('_c6'))
df.repartition(1).write.csv("/user/antifraud/source/tmp_test/tmp_file/res2.csv", sep='\t')

In [ ]:
df = spark.read.parquet("/user/antifraud/hongjing2/dataflow/step_two/tid/nf_feature_preprocessing/20170518")
df.take(10)

In [6]:
df = spark.read.parquet(
    '/user/antifraud/hongjing2/dataflow/step_one/tid/common_company_info_merge_v2/20170627'
)

df.select('c_is_black_company').where(df.c_is_black_company.).show()

+------------------+
|c_is_black_company|
+------------------+
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
|             false|
+------------------+
only showing top 20 rows



In [3]:
raw = spark.read.csv(
    "/user/antifraud/jinli/rawdata/tar_company/raw_jl_company_list_20171015_1.data"
).withColumnRenamed("_c0", "company_name")

raw.show()

+------------------+
|      company_name|
+------------------+
|      北京花果信息技术有限公司|
|      北京远山利丰科贸有限公司|
|  北京镒金恒睿投资中心（有限合伙）|
|        十月投资控股有限公司|
|        北京光灿传媒有限公司|
|    北京首科文创投资管理有限公司|
|      北京成龙华天早餐有限公司|
|      北京极客韦尔科技有限公司|
|      北京汲古文化科技有限公司|
|    宁波首科燕园科技发展有限公司|
|  北京海科健融医疗投资管理有限公司|
|    北京文信时空文化发展有限公司|
|宁波首科淮海投资合伙企业（有限合伙）|
|      中瑞富信资产管理有限公司|
|      北京圣云奥海科技有限公司|
|    博思智胜（北京）咨询有限公司|
|发现创合（北京）文化传播有限责任公司|
|     北京恒基发投资有限责任公司|
|    北京锐圣尚坤科技发展有限公司|
|      北京镒金投资管理有限公司|
+------------------+
only showing top 20 rows



In [1]:
df = spark.read.parquet("/user/antifraud/hongjing2/dataflow/step_three/prd/all_company_info/20170924")

df.show()

+--------------------+--------------------+--------------------+----------+--------------------+--------+----+------+--------+------------+--------------------+---------+------------+--------------------+--------------------+
|        company_name|         bbd_qyxx_id|           risk_tags|risk_index|    risk_composition|province|city|county|is_black|company_type|           xgxx_info|risk_rank|data_version|          gmt_create|          gmt_update|
+--------------------+--------------------+--------------------+----------+--------------------+--------+----+------+--------+------------+--------------------+---------+------------+--------------------+--------------------+
|        上海枘琦信息科技有限公司|003c81736bc0467cb...|{"--": {"静态关联方风险"...|      30.0|{"--": {"静态关联方风险"...|     上海市| 上海市|   奉贤区|    null|        新兴金融|{"行政处罚": 0, "分支机构...|     持续监控|    20170924|2017-10-11 18:37:...|2017-10-11 18:37:...|
|     广西龙佳旺酒店投资有限责任公司|01eb7b691ad1457b9...|{"--": {"静态关联方风险"...|      30.1|{"--": {"静态关联方风险"...|

In [7]:
df.join(
    raw,
    'company_name'
).select(
    'company_name',
    'risk_index'
).write.csv(
    '/user/antifraud/source/tmp_test/tmp_file/tmp_20171015'
)

In [6]:
from pyspark.sql import Row

company = [
    u'丹东港集团有限公司',
    u'乐视网信息技术（北京）股份有限公司'
]

tar_df = spark.sparkContext.parallelize(company).map(lambda r: Row(company_name=r)).toDF()

basic_df = spark.sql(
    '''
    select company_name,bbd_qyxx_id from dw.qyxx_basic where dt='20171030'
    '''
)

tar_df.join(
    basic_df,
    'company_name'
).take(10)

[Row(company_name=u'\u4e50\u89c6\u7f51\u4fe1\u606f\u6280\u672f\uff08\u5317\u4eac\uff09\u80a1\u4efd\u6709\u9650\u516c\u53f8', bbd_qyxx_id=u'ba369113a4c244608fb3541a4a5e6074'),
 Row(company_name=u'\u4e39\u4e1c\u6e2f\u96c6\u56e2\u6709\u9650\u516c\u53f8', bbd_qyxx_id=u'ea17b8bac2bc48b093e52b754265a127')]

In [14]:
version_list = [
    '20170117'
    ,'20170221'
    ,'20170325'
    ,'20170424'
    ,'20170518'
    ,'20170627'
    ,'20170720'
    ,'20170825'
    ,'20170925'
    ,'20171025'
]

for version in  version_list:
    

In [14]:
df = spark.sql(
    '''
    select * from ods.xgxx_relation where dt=20171204
    '''
)

df.where(
    df.bbd_xgxx_id == '056e6b1d04cd8743a93601d08565d9d9'
).show()

+---------+--------------------+--------------------+---------+-------+--------------------+--------+
|       id|         bbd_qyxx_id|         bbd_xgxx_id|bbd_table|id_type|         create_time|      dt|
+---------+--------------------+--------------------+---------+-------+--------------------+--------+
|159236463|d289e76df0534ec2b...|056e6b1d04cd8743a...|     ktgg|      0|2016-08-04 10:14:...|20171204|
|159236465|a802b9a2f43b4f68a...|056e6b1d04cd8743a...|     ktgg|      0|2016-08-04 10:14:...|20171204|
|159236466|7a7be27abff74a038...|056e6b1d04cd8743a...|     ktgg|      0|2016-08-04 10:14:...|20171204|
|159236467|996f4bbd84df4073b...|056e6b1d04cd8743a...|     ktgg|      1|2016-08-04 10:14:...|20171204|
+---------+--------------------+--------------------+---------+-------+--------------------+--------+



In [16]:
df = spark.read.parquet('/user/wanxiang/tmpdata/tid_xgxx_relation_df/20171117')
df.where(
    df.bbd_xgxx_id == '056e6b1d04cd8743a93601d08565d9d9'
).show()

+---+--------------------+--------------------+---------+-------+--------+----------+---------------+
| id|         bbd_qyxx_id|         bbd_xgxx_id|bbd_table|id_type|      dt|event_time|event_timestamp|
+---+--------------------+--------------------+---------+-------+--------+----------+---------------+
|   |d289e76df0534ec2b...|056e6b1d04cd8743a...|     ktgg|      0|20171117|2016-06-24|     1466697600|
+---+--------------------+--------------------+---------+-------+--------+----------+---------------+



In [18]:
df = spark.read.csv("/user/wanxiang/step_two/20171204/event_edge")

df.where(
    df._c3 == '056e6b1d04cd8743a93601d08565d9d9'
).show()

+--------------------+----------+---+--------------------+----+
|                 _c0|       _c1|_c2|                 _c3| _c4|
+--------------------+----------+---+--------------------+----+
|d289e76df0534ec2b...|1512462194|  0|056e6b1d04cd8743a...|KTGG|
|7a7be27abff74a038...|1512462194|  0|056e6b1d04cd8743a...|KTGG|
|996f4bbd84df4073b...|1512462194|  1|056e6b1d04cd8743a...|KTGG|
|a802b9a2f43b4f68a...|1512462194|  0|056e6b1d04cd8743a...|KTGG|
+--------------------+----------+---+--------------------+----+



In [2]:
df = spark.read.parquet("/user/antifraud/hongjing2/dataflow/step_two/tid/nf_feature_preprocessing/20180227")
df.take(3)

[Row(features=DenseVector([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 5000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 104.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]), bbd_qyxx_id=u'07ca6b04163e4807b212fb9b96b48f87', company_name=u'\u8fdc\u76db\u519c\u7267\u79d1\u6280\uff08\u6d77\u5357\uff09\u6709\u9650\u516c\u53f8', scaledFeatures=DenseVector([0.0008, -0.1215, -0.0651, -0.1261, 0.0, 0.0, 0.0, -0.0328, -0.0781, -0.1616, -0.1825, -0.0607, -0.1455, 0.0, -0.1965, -0.0279, -0.0349, -0.1828, -0.0447, -0.1252, -0.1857, -0.0221, -0.2233, -0.0522, -0.2197, 1.0626, -0.1962, -0.1999, -0.7406, -0.2111, -1.272, -1.1416, -1.6513, 0.0258, -0.0033, -0.2311, -0.1021, -0.0414, -0.0728, -0.2198, -0.1111, -0.1661, -0.2198, -0.3353, -0.15, -0.3425, -0.1347, -0.2773, -0.3826, -0.5324, 0.0, 0.0, 0.0, 0.0, -0.9742, -1.0223, -0.4698, -0.072

In [1]:
df = spark.read.json("/user/antifraud/hongjing2/dataflow/step_two/prd/nf_feature_risk_score/20180227")
df.show()

+------------------+------------------------+------------------+----------------------------+---------------------------+--------------------+--------------------+------------------+
|  GM_behavior_risk|GM_company_strength_risk|    GM_credit_risk|GM_dynamic_relationship_risk|GM_static_relationship_risk|         bbd_qyxx_id|        company_name|       total_score|
+------------------+------------------------+------------------+----------------------------+---------------------------+--------------------+--------------------+------------------+
| 46.98555658016041|       63.31955352863837|46.910781430052324|           49.99924375714049|         43.039050757196605|4d0d6afe64bd40748...|      永安市安立信企业管理有限公司| 43.34999493748262|
|51.924864700431236|      38.944979187839515|44.672319498943686|          49.353070100043986|          63.49531613502696|4d0d6e2e15fb4be4b...|       温州心净莲文化发展有限公司| 33.40753364228069|
|  52.6784356378773|       69.89132298589371| 84.06120723053986|          48.53213896

In [8]:
XGXX_RELATION = '20180305'
raw_qyxx_fzjg_merge = spark.sql(
    '''
    SELECT
    bbd_branch_id b,
    bbd_qyxx_id c,
    name b_name,
    company_name c_name,
    'BRANCH' bc_relation,
    '分支机构' role_name,
    '-' ratio,
    regno_or_creditcode,
    regno, 
    name
    FROM
    dw.qyxx_fzjg_merge
    WHERE
    dt='{version}'
    '''.format(version=XGXX_RELATION)
)
raw_qyxx_fzjg_merge.show()

+--------------------+--------------------+--------------------+------------------+-----------+---------+-----+-------------------+-----------------+--------------------+
|                   b|                   c|              b_name|            c_name|bc_relation|role_name|ratio|regno_or_creditcode|            regno|                name|
+--------------------+--------------------+--------------------+------------------+-----------+---------+-----+-------------------+-----------------+--------------------+
|4b747cf333044a4ea...|82e3b988d9784d3fb...|  西安市木可焊接科技有限公司第一分公司|     西安市木可焊接科技有限公司|     BRANCH|     分支机构|    -| 91610139MA6TX14L4G|  610139200000809|  西安市木可焊接科技有限公司第一分公司|
|9e6300948e004b0f9...|f47ed50a4903448c8...|重庆群联物业管理有限责任公司南坪直...|    重庆群联物业管理有限责任公司|     BRANCH|     分支机构|    -|                   |  500108300020536|重庆群联物业管理有限责任公司南坪直...|
|e5a4d96186f344e88...|f7f47551076f430ea...|    浙江民政礼仪用品经营服务公司分部|    浙江民政礼仪用品经营服务公司|     BRANCH|     分支机构|    -|      3301001703411|    3301001703

In [12]:
import sys
import os
import re
from functools import partial
import hashlib

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql import functions as fun, types as tp

def filter_comma(col):
    '''ID中逗号或为空值，则将该记录删除'''
    if not col or ',' in col or u'\uff0c' in col:
        return False
    else:
        return True

def filter_chinaese(col):
    '''字段中只要包含中文，将其过滤'''
    if col:
        match = re.search(ur'[\u4e00-\u9fa5]', col)
        return False if match else True
    else:
        return False

def get_id(src, des, relation, position):
    '''
    生成规则：md5（起点name+终点id+关系类型+关系名）
    由于源ID有中文，因此这里需要做容错
    '''
    try:
        role_id = hashlib.md5(src.encode('utf-8') + 
                              des.encode('utf-8') +
                              relation.encode('utf-8') +
                              position.encode('utf-8'))
        return role_id.hexdigest()
    except:
        return ''
        
def get_branch_id(regno_or_creditcode, 
                  regno, 
                  name):
    '''
    生成规则：md5（起点name+终点id+关系类型+关系名）
    由于源ID有中文，因此这里需要做容错
    '''
    try:
        role_id = hashlib.md5(regno_or_creditcode.encode('utf-8') + 
                              regno.encode('utf-8') +
                              name.encode('utf-8') +
                              "qyxx_fzjg_merge")
        return role_id.hexdigest()
    except:
        return ''

def is_invest(col):
    if 'INVEST' in col:
        return True
    else:
        return False

def get_role_label(label):
    return 'Entity;Role;{0}'.format(label.lower().capitalize())

def get_chinese(string):
    '''获得中文字符串'''
    try:
        result = re.findall(ur'[\u4e00-\u9fa5]', string)
        if result:
            return ''.join(result)
        else:
            return '-'
    except:
        return '-'


filter_chinaese_udf = fun.udf(filter_chinaese, tp.BooleanType())
filter_comma_udf = fun.udf(filter_comma, tp.BooleanType())
get_role_label_udf = fun.udf(get_role_label, 
                             tp.StringType())
get_isinvest_label_udf = fun.udf(
    partial(get_role_label, 'Isinvest'), tp.StringType())
get_id_udf = fun.udf(get_id, tp.StringType())
get_branch_id_udf = fun.udf(get_branch_id, tp.StringType())
get_isinvest_id_udf = fun.udf(
    partial(get_id, relation='Isinvest', position=''), tp.StringType())
is_invest_udf = fun.udf(is_invest, tp.BooleanType())
get_chinese_udf = fun.udf(get_chinese, tp.StringType())

get_relation_label_3_udf = fun.udf(
        partial(lambda r: r, 'VIRTUAL'), tp.StringType())    


get_relation_label_3_udf = fun.udf(
    partial(lambda r: r, 'VIRTUAL'), tp.StringType())    

tid_qyxx_fzjg_merge = raw_qyxx_fzjg_merge.where(
        filter_comma_udf('b')
    ).where(
        filter_chinaese_udf('b')
    ).where(
        filter_comma_udf('c')
    ).where(
        filter_chinaese_udf('c')
    ).where(
        filter_comma_udf('b_name')
    ).where(
        filter_comma_udf('c_name')
    ).select(
        fun.when(
            raw_qyxx_fzjg_merge.b == 'None',
            get_branch_id_udf('regno_or_creditcode', 'regno', 'name')
        ).otherwise(
            raw_qyxx_fzjg_merge.b        
        ).alias('b'),
        'b_name',
        get_id_udf('b_name', 'c', 
                   'bc_relation', 'role_name').alias('bbd_role_id:ID'),
        'c',
        'c_name',
        'bc_relation',
        'role_name',
        'ratio',
    ).dropDuplicates(
        ['b', 'c', 'bc_relation']
    )

In [16]:
tmp = spark.read.parquet("/user/wanxiang/tmpdata/tid_qyxx_fzjg_merge/20180304")
tmp.count()

5028976

In [3]:
df = spark.sql(
    '''
    select * from dw.qyxx_basic where dt=20180305 and bbd_qyxx_id='5e1562de3f1a4386bad38a0e9128a437'
    '''
)

df.show()

+----+--------------------+--------------------+-------------+----------+------------+----------+-----------+-------------------+--------------+----------------+-------------------------+----------------+------------+----------------+--------------+------------------+-----------------+----------+----+------+--------------------+----------+----------------+------------------+-----------+----------+----------+--------------------+----------------+-----------+-------+--------------+----------------+--------------------+-------------+---------------+---------+---------------+-------------------+-----------+----------+----+----------------+------------+-------------+--------------------+---------------+---------------+-------------+--------+
|  id|         bbd_qyxx_id|             address|approval_date|bbd_dotime|    bbd_type|bbd_uptime|cancel_date|company_companytype|company_county|company_currency|company_enterprise_status|company_industry|company_name|company_province|company_regorg|    

In [10]:
kpi = spark.read.json("/user/antifraud/jinli/prddata/prd_jinli_feature_score_distribution_20180312_02")
kpi.select(
    kpi.feature_10.getField('0').getField('zgcpwsw_1')
).show()

+----------------------+
|feature_10.0.zgcpwsw_1|
+----------------------+
|                     0|
|                     0|
|                     0|
|                     0|
|                     0|
|                     0|
|                     0|
|                     0|
|                     1|
|                     0|
|                     0|
|                     0|
|                     0|
|                     0|
|                     1|
|                     0|
|                     0|
|                     1|
|                     0|
|                     0|
+----------------------+
only showing top 20 rows

